In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

!pip install --quiet torch torchvision webdataset tqdm pillow

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 kB 8.9 MB/s eta 0:00:00


In [2]:
from pathlib import Path
import yaml
import sys
import time
import importlib
import logging
from typing import Any, Dict
from tqdm import tqdm
import inspect

config_path = Path('/content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/config/training.yaml')

with config_path.open('r') as f:
    cfg = yaml.safe_load(f)

colab_root = Path(cfg['env_paths']['colab'])
local_root = Path(cfg['env_paths']['local'])
PROJECT_ROOT = colab_root if colab_root.exists() else local_root
if not PROJECT_ROOT.exists():
    raise FileNotFoundError(f"Project root non trovato: {PROJECT_ROOT}")

sys.path.insert(0, str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT / 'src'))


from importlib.util import spec_from_file_location, module_from_spec

utils_dir = PROJECT_ROOT / 'src' / 'utils'
src_file = utils_dir / 'training_utils.py'

spec = spec_from_file_location('utils.training_utils', str(src_file))
training_utils = module_from_spec(spec)
spec.loader.exec_module(training_utils)

sys.modules['utils.training_utils'] = training_utils

from utils.training_utils import TRAINER_REGISTRY

print(f"🔥 PROJECT_ROOT: {PROJECT_ROOT}")


🔥 PROJECT_ROOT: /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project


In [ ]:
for split in ['train','val','test']:
    rel = cfg['data'].get(split)
    if rel:
        cfg['data'][split] = str(PROJECT_ROOT / rel)

print("📂 Dataset paths:")
for split in ['train','val','test']:
    print(f"  • {split}: {cfg['data'][split]}")

sys.path.insert(0, str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT / 'src'))

📂 Dataset paths:
  • train: /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed/webdataset_2500/train/patches-0000.tar
  • val: /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed/webdataset_2500/val/patches-0000.tar
  • test: /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed/webdataset_2500/test/patches-0000.tar


In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)-8s | %(name)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger("LAUNCHER")
logger.info("✅ Logger inizializzato a livello INFO")

In [ ]:
trainer_modules = [
    "trainers.simclr",
    "trainers.moco_v2",
    "trainers.rotation",
    "trainers.jigsaw",
    "trainers.supervised",
    "trainers.transfer",
]
for module_name in trainer_modules:
    if module_name in sys.modules:
        importlib.reload(sys.modules[module_name])
    else:
        importlib.import_module(module_name)


In [ ]:
import inspect
import time

def launch_training(cfg: dict) -> None:
    """
    Lancia il training dei modelli specificati.
    Per ogni batch: stampa Loss, (Acc se supervised), % completamento, Elapsed, ETA su riga separata.
    Supporta trainer con signature batch-wise o (imgs, labels).
    """
    models_cfg = cfg.get('models', {})
    run_model = cfg.get('run_model', 'all').lower()
    if run_model == 'all':
        tasks = list(models_cfg.items())
    else:
        if run_model not in models_cfg:
            raise KeyError(f"Modello '{run_model}' non presente in cfg['models']")
        tasks = [(run_model, models_cfg[run_model])]

    for name, m_cfg in tasks:
        if name not in TRAINER_REGISTRY:
            raise KeyError(f"Nessun trainer registrato per '{name}'")
        trainer = TRAINER_REGISTRY[name](m_cfg, cfg['data'])
        device = getattr(trainer, 'device', 'n/a')
        epochs = int(m_cfg['training'].get('epochs', 0))
        batch_size = int(m_cfg['training'].get('batch_size', 0))

        print(f"Device: {device} 🚀  Inizio training per modello '{name}'")
        print(f"→ Model config: {m_cfg}")
        print(f"Epoche: {epochs} | Batch size: {batch_size}\n")

        has_validation = hasattr(trainer, 'validate_epoch')

        for epoch in range(1, epochs + 1):
            epoch_start = time.time()
            total_batches = getattr(trainer, 'batches_train', None)
            running_loss = 0.0
            running_correct = 0
            total_samples = 0

            print(f"--- Epoch {epoch}/{epochs} ---")
            for i, batch in enumerate(trainer.train_loader, start=1):
                sig = inspect.signature(trainer.train_step)
                if len(sig.parameters) == 1:
                    result = trainer.train_step(batch)
                else:
                    imgs, labels = batch
                    result = trainer.train_step(imgs, labels)

                if len(result) == 4:
                    _, loss, correct, bs = result
                else:
                    loss, bs = result
                    correct = 0

                running_loss += loss * bs
                running_correct += correct
                total_samples += bs
                avg_loss = running_loss / total_samples
                avg_acc = (running_correct / total_samples) if has_validation and total_samples else 0.0
                elapsed = time.time() - epoch_start
                pct = (i / total_batches) * 100 if total_batches else 0.0
                eta = (elapsed / i) * (total_batches - i) if total_batches and i else 0.0

                msg = (
                    f"  Batch {i}/{total_batches} ({pct:.1f}%) | "
                    f"Loss: {avg_loss:.4f}"
                )
                if has_validation:
                    msg += f" | Acc: {avg_acc:.3f}"
                msg += f" | Elapsed: {elapsed:.1f}s | ETA: {eta:.1f}s"
                print(msg)

            if has_validation:
                val_loss, val_acc = trainer.validate_epoch()
                print(f"Val -> Loss: {val_loss:.4f} | Acc: {val_acc:.3f}")
                trainer.post_epoch(epoch, val_acc)
            else:
                epoch_loss = running_loss / total_samples
                trainer.post_epoch(epoch, epoch_loss)

            duration = time.time() - epoch_start
            print(f"Epoch {epoch} completed in {duration:.1f}s\n")

        best = trainer.summary()
        if isinstance(best, tuple) and len(best) == 2:
            be, bm = best
            print(f"✅ Training per '{name}' completato. Best @ epoch {be} -> {bm:.3f}")


In [ ]:
launch_training(cfg)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Device: cuda 🚀  Inizio training per modello 'supervised'
→ Model config: {'backbone': 'resnet50', 'pretrained': False, 'training': {'epochs': 50, 'batch_size': 32, 'optimizer': 'adam', 'learning_rate': '1e-4', 'weight_decay': '1e-5'}}
Epoche: 50 | Batch size: 32

--- Epoch 1/50 ---
  Batch 1/47 (2.1%) | Loss: 1.6185 | Acc: 0.281 | Elapsed: 0.9s | ETA: 39.9s
  Batch 2/47 (4.3%) | Loss: 1.6390 | Acc: 0.266 | Elapsed: 1.2s | ETA: 26.2s
  Batch 3/47 (6.4%) | Loss: 1.6396 | Acc: 0.250 | Elapsed: 1.5s | ETA: 21.5s
  Batch 4/47 (8.5%) | Loss: 1.6381 | Acc: 0.266 | Elapsed: 1.8s | ETA: 19.0s
  Batch 5/47 (10.6%) | Loss: 1.6054 | Acc: 0.287 | Elapsed: 2.1s | ETA: 17.4s
  Batch 6/47 (12.8%) | Loss: 1.6026 | Acc: 0.276 | Elapsed: 2.4s | ETA: 16.2s
  Batch 7/47 (14.9%) | Loss: 1.6059 | Acc: 0.268 | Elapsed: 2.7s | ETA: 15.3s
  Batch 8/47 (17.0%) | Loss: 1.5781 | Acc: 0.281 | Elapsed: 3.0s | ETA: 14.5s
  Batch 9/47 (19.1%) | Loss: 1.5671 | Acc: 0.309 | Elapsed: 3.3s | ETA: 13.8s
  Batch 10/47 (21.3

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Val -> Loss: 4.4590 | Acc: 0.228
Epoch 1 completed in 17.3s

--- Epoch 2/50 ---


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


...

Val -> Loss: 1.9671 | Acc: 0.493
Epoch 49 completed in 17.3s

--- Epoch 50/50 ---


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


  Batch 1/47 (2.1%) | Loss: 0.5584 | Acc: 0.656 | Elapsed: 0.8s | ETA: 37.6s
  Batch 2/47 (4.3%) | Loss: 0.4470 | Acc: 0.781 | Elapsed: 1.1s | ETA: 25.4s
  Batch 3/47 (6.4%) | Loss: 0.4321 | Acc: 0.792 | Elapsed: 1.5s | ETA: 21.6s
  Batch 4/47 (8.5%) | Loss: 0.4485 | Acc: 0.805 | Elapsed: 1.8s | ETA: 19.2s
  Batch 5/47 (10.6%) | Loss: 0.4529 | Acc: 0.794 | Elapsed: 2.1s | ETA: 17.7s
  Batch 6/47 (12.8%) | Loss: 0.4191 | Acc: 0.812 | Elapsed: 2.4s | ETA: 16.6s
  Batch 7/47 (14.9%) | Loss: 0.4578 | Acc: 0.804 | Elapsed: 2.7s | ETA: 15.7s
  Batch 8/47 (17.0%) | Loss: 0.4592 | Acc: 0.805 | Elapsed: 3.1s | ETA: 14.9s
  Batch 9/47 (19.1%) | Loss: 0.4691 | Acc: 0.806 | Elapsed: 3.4s | ETA: 14.3s
  Batch 10/47 (21.3%) | Loss: 0.4584 | Acc: 0.816 | Elapsed: 3.7s | ETA: 13.7s
  Batch 11/47 (23.4%) | Loss: 0.4456 | Acc: 0.810 | Elapsed: 4.0s | ETA: 13.2s
  Batch 12/47 (25.5%) | Loss: 0.4445 | Acc: 0.807 | Elapsed: 4.4s | ETA: 12.7s
  Batch 13/47 (27.7%) | Loss: 0.4401 | Acc: 0.810 | Elapsed: 4.7s

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Val -> Loss: 1.6349 | Acc: 0.585
Epoch 50 completed in 17.1s

✅ Training per 'supervised' completato. Best @ epoch 46 -> 0.614


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Device: cuda 🚀  Inizio training per modello 'supervised'
→ Model config: {'backbone': 'resnet50', 'pretrained': False, 'training': {'epochs': 50, 'batch_size': 32, 'optimizer': 'adam', 'learning_rate': '1e-4', 'weight_decay': '1e-5'}}
Epoche: 50 | Batch size: 32

--- Epoch 1/50 ---
  Batch 1/47 (2.1%) | Loss: 1.6185 | Acc: 0.281 | Elapsed: 0.9s | ETA: 39.9s
  Batch 2/47 (4.3%) | Loss: 1.6390 | Acc: 0.266 | Elapsed: 1.2s | ETA: 26.2s
  Batch 3/47 (6.4%) | Loss: 1.6396 | Acc: 0.250 | Elapsed: 1.5s | ETA: 21.5s
  Batch 4/47 (8.5%) | Loss: 1.6381 | Acc: 0.266 | Elapsed: 1.8s | ETA: 19.0s
  Batch 5/47 (10.6%) | Loss: 1.6054 | Acc: 0.287 | Elapsed: 2.1s | ETA: 17.4s
  Batch 6/47 (12.8%) | Loss: 1.6026 | Acc: 0.276 | Elapsed: 2.4s | ETA: 16.2s
  Batch 7/47 (14.9%) | Loss: 1.6059 | Acc: 0.268 | Elapsed: 2.7s | ETA: 15.3s
  Batch 8/47 (17.0%) | Loss: 1.5781 | Acc: 0.281 | Elapsed: 3.0s | ETA: 14.5s
  Batch 9/47 (19.1%) | Loss: 1.5671 | Acc: 0.309 | Elapsed: 3.3s | ETA: 13.8s
  Batch 10/47 (21.3%) | Loss: 1.5506 | Acc: 0.309 | Elapsed: 3.6s | ETA: 13.2s
  Batch 11/47 (23.4%) | Loss: 1.5557 | Acc: 0.304 | Elapsed: 3.9s | ETA: 12.7s
  Batch 12/47 (25.5%) | Loss: 1.5391 | Acc: 0.318 | Elapsed: 4.2s | ETA: 12.2s
  Batch 13/47 (27.7%) | Loss: 1.5382 | Acc: 0.315 | Elapsed: 4.5s | ETA: 11.7s
  Batch 14/47 (29.8%) | Loss: 1.5346 | Acc: 0.306 | Elapsed: 4.8s | ETA: 11.3s
  Batch 15/47 (31.9%) | Loss: 1.5304 | Acc: 0.302 | Elapsed: 5.1s | ETA: 10.9s
  Batch 16/47 (34.0%) | Loss: 1.5208 | Acc: 0.309 | Elapsed: 5.4s | ETA: 10.5s
  Batch 17/47 (36.2%) | Loss: 1.5201 | Acc: 0.307 | Elapsed: 5.7s | ETA: 10.1s
  Batch 18/47 (38.3%) | Loss: 1.5122 | Acc: 0.309 | Elapsed: 6.0s | ETA: 9.7s
  Batch 19/47 (40.4%) | Loss: 1.5080 | Acc: 0.306 | Elapsed: 6.3s | ETA: 9.4s
  Batch 20/47 (42.6%) | Loss: 1.5020 | Acc: 0.309 | Elapsed: 6.7s | ETA: 9.0s
  Batch 21/47 (44.7%) | Loss: 1.4880 | Acc: 0.314 | Elapsed: 7.0s | ETA: 8.6s
  Batch 22/47 (46.8%) | Loss: 1.4820 | Acc: 0.321 | Elapsed: 7.3s | ETA: 8.3s
  Batch 23/47 (48.9%) | Loss: 1.4698 | Acc: 0.329 | Elapsed: 7.6s | ETA: 7.9s
  Batch 24/47 (51.1%) | Loss: 1.4575 | Acc: 0.337 | Elapsed: 7.9s | ETA: 7.5s
  Batch 25/47 (53.2%) | Loss: 1.4460 | Acc: 0.345 | Elapsed: 8.2s | ETA: 7.2s
  Batch 26/47 (55.3%) | Loss: 1.4407 | Acc: 0.349 | Elapsed: 8.5s | ETA: 6.9s
  Batch 27/47 (57.4%) | Loss: 1.4273 | Acc: 0.359 | Elapsed: 8.8s | ETA: 6.5s
  Batch 28/47 (59.6%) | Loss: 1.4189 | Acc: 0.360 | Elapsed: 9.1s | ETA: 6.2s
  Batch 29/47 (61.7%) | Loss: 1.4126 | Acc: 0.367 | Elapsed: 9.4s | ETA: 5.8s
  Batch 30/47 (63.8%) | Loss: 1.4034 | Acc: 0.372 | Elapsed: 9.7s | ETA: 5.5s
  Batch 31/47 (66.0%) | Loss: 1.3924 | Acc: 0.379 | Elapsed: 10.0s | ETA: 5.2s
  Batch 32/47 (68.1%) | Loss: 1.3891 | Acc: 0.381 | Elapsed: 10.3s | ETA: 4.8s
  Batch 33/47 (70.2%) | Loss: 1.3795 | Acc: 0.387 | Elapsed: 10.6s | ETA: 4.5s
  Batch 34/47 (72.3%) | Loss: 1.3720 | Acc: 0.390 | Elapsed: 10.9s | ETA: 4.2s
  Batch 35/47 (74.5%) | Loss: 1.3804 | Acc: 0.390 | Elapsed: 11.2s | ETA: 3.9s
  Batch 36/47 (76.6%) | Loss: 1.3767 | Acc: 0.391 | Elapsed: 11.5s | ETA: 3.5s
  Batch 37/47 (78.7%) | Loss: 1.3720 | Acc: 0.394 | Elapsed: 11.8s | ETA: 3.2s
  Batch 38/47 (80.9%) | Loss: 1.3669 | Acc: 0.394 | Elapsed: 12.1s | ETA: 2.9s
  Batch 39/47 (83.0%) | Loss: 1.3646 | Acc: 0.396 | Elapsed: 12.5s | ETA: 2.6s
  Batch 40/47 (85.1%) | Loss: 1.3604 | Acc: 0.397 | Elapsed: 12.8s | ETA: 2.2s
  Batch 41/47 (87.2%) | Loss: 1.3617 | Acc: 0.398 | Elapsed: 13.1s | ETA: 1.9s
  Batch 42/47 (89.4%) | Loss: 1.3555 | Acc: 0.400 | Elapsed: 13.4s | ETA: 1.6s
  Batch 43/47 (91.5%) | Loss: 1.3489 | Acc: 0.403 | Elapsed: 13.7s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 1.3439 | Acc: 0.408 | Elapsed: 14.0s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 1.3377 | Acc: 0.410 | Elapsed: 14.3s | ETA: 0.6s
  Batch 46/47 (97.9%) | Loss: 1.3314 | Acc: 0.412 | Elapsed: 14.6s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 1.3321 | Acc: 0.412 | Elapsed: 14.7s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 4.4590 | Acc: 0.228
Epoch 1 completed in 17.3s

--- Epoch 2/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 1.2148 | Acc: 0.469 | Elapsed: 1.0s | ETA: 45.6s
  Batch 2/47 (4.3%) | Loss: 1.1944 | Acc: 0.500 | Elapsed: 1.3s | ETA: 29.3s
  Batch 3/47 (6.4%) | Loss: 1.1966 | Acc: 0.510 | Elapsed: 1.6s | ETA: 23.8s
  Batch 4/47 (8.5%) | Loss: 1.1519 | Acc: 0.547 | Elapsed: 1.9s | ETA: 20.7s
  Batch 5/47 (10.6%) | Loss: 1.1680 | Acc: 0.519 | Elapsed: 2.2s | ETA: 18.8s
  Batch 6/47 (12.8%) | Loss: 1.1427 | Acc: 0.542 | Elapsed: 2.5s | ETA: 17.4s
  Batch 7/47 (14.9%) | Loss: 1.1105 | Acc: 0.554 | Elapsed: 2.9s | ETA: 16.4s
  Batch 8/47 (17.0%) | Loss: 1.1040 | Acc: 0.535 | Elapsed: 3.2s | ETA: 15.5s
  Batch 9/47 (19.1%) | Loss: 1.0792 | Acc: 0.538 | Elapsed: 3.5s | ETA: 14.7s
  Batch 10/47 (21.3%) | Loss: 1.0766 | Acc: 0.541 | Elapsed: 3.8s | ETA: 14.0s
  Batch 11/47 (23.4%) | Loss: 1.0768 | Acc: 0.537 | Elapsed: 4.1s | ETA: 13.4s
  Batch 12/47 (25.5%) | Loss: 1.0831 | Acc: 0.523 | Elapsed: 4.4s | ETA: 12.9s
  Batch 13/47 (27.7%) | Loss: 1.0750 | Acc: 0.531 | Elapsed: 4.7s | ETA: 12.4s
  Batch 14/47 (29.8%) | Loss: 1.0757 | Acc: 0.525 | Elapsed: 5.0s | ETA: 11.9s
  Batch 15/47 (31.9%) | Loss: 1.0648 | Acc: 0.535 | Elapsed: 5.3s | ETA: 11.4s
  Batch 16/47 (34.0%) | Loss: 1.0629 | Acc: 0.529 | Elapsed: 5.7s | ETA: 11.0s
  Batch 17/47 (36.2%) | Loss: 1.0683 | Acc: 0.533 | Elapsed: 6.0s | ETA: 10.5s
  Batch 18/47 (38.3%) | Loss: 1.0683 | Acc: 0.535 | Elapsed: 6.3s | ETA: 10.1s
  Batch 19/47 (40.4%) | Loss: 1.0718 | Acc: 0.536 | Elapsed: 6.6s | ETA: 9.7s
  Batch 20/47 (42.6%) | Loss: 1.0600 | Acc: 0.547 | Elapsed: 6.9s | ETA: 9.3s
  Batch 21/47 (44.7%) | Loss: 1.0530 | Acc: 0.545 | Elapsed: 7.2s | ETA: 8.9s
  Batch 22/47 (46.8%) | Loss: 1.0440 | Acc: 0.545 | Elapsed: 7.5s | ETA: 8.5s
  Batch 23/47 (48.9%) | Loss: 1.0412 | Acc: 0.550 | Elapsed: 7.8s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 1.0391 | Acc: 0.547 | Elapsed: 8.1s | ETA: 7.8s
  Batch 25/47 (53.2%) | Loss: 1.0397 | Acc: 0.546 | Elapsed: 8.4s | ETA: 7.4s
  Batch 26/47 (55.3%) | Loss: 1.0365 | Acc: 0.549 | Elapsed: 8.8s | ETA: 7.1s
  Batch 27/47 (57.4%) | Loss: 1.0314 | Acc: 0.552 | Elapsed: 9.1s | ETA: 6.7s
  Batch 28/47 (59.6%) | Loss: 1.0407 | Acc: 0.551 | Elapsed: 9.4s | ETA: 6.4s
  Batch 29/47 (61.7%) | Loss: 1.0361 | Acc: 0.554 | Elapsed: 9.7s | ETA: 6.0s
  Batch 30/47 (63.8%) | Loss: 1.0313 | Acc: 0.553 | Elapsed: 10.0s | ETA: 5.7s
  Batch 31/47 (66.0%) | Loss: 1.0386 | Acc: 0.552 | Elapsed: 10.3s | ETA: 5.3s
  Batch 32/47 (68.1%) | Loss: 1.0352 | Acc: 0.556 | Elapsed: 10.6s | ETA: 5.0s
  Batch 33/47 (70.2%) | Loss: 1.0344 | Acc: 0.552 | Elapsed: 11.0s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 1.0395 | Acc: 0.548 | Elapsed: 11.3s | ETA: 4.3s
  Batch 35/47 (74.5%) | Loss: 1.0378 | Acc: 0.547 | Elapsed: 11.6s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 1.0338 | Acc: 0.550 | Elapsed: 11.9s | ETA: 3.6s
  Batch 37/47 (78.7%) | Loss: 1.0283 | Acc: 0.552 | Elapsed: 12.2s | ETA: 3.3s
  Batch 38/47 (80.9%) | Loss: 1.0358 | Acc: 0.551 | Elapsed: 12.6s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 1.0342 | Acc: 0.553 | Elapsed: 12.9s | ETA: 2.6s
  Batch 40/47 (85.1%) | Loss: 1.0357 | Acc: 0.551 | Elapsed: 13.2s | ETA: 2.3s
  Batch 41/47 (87.2%) | Loss: 1.0361 | Acc: 0.550 | Elapsed: 13.5s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 1.0375 | Acc: 0.552 | Elapsed: 13.8s | ETA: 1.6s
  Batch 43/47 (91.5%) | Loss: 1.0361 | Acc: 0.552 | Elapsed: 14.2s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 1.0301 | Acc: 0.555 | Elapsed: 14.5s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 1.0251 | Acc: 0.558 | Elapsed: 14.8s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 1.0254 | Acc: 0.557 | Elapsed: 15.1s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 1.0279 | Acc: 0.556 | Elapsed: 15.2s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 2.2637 | Acc: 0.384
Epoch 2 completed in 21.0s

--- Epoch 3/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.7138 | Acc: 0.750 | Elapsed: 0.8s | ETA: 37.2s
  Batch 2/47 (4.3%) | Loss: 0.8494 | Acc: 0.641 | Elapsed: 1.1s | ETA: 25.2s
  Batch 3/47 (6.4%) | Loss: 0.8312 | Acc: 0.667 | Elapsed: 1.5s | ETA: 21.4s
  Batch 4/47 (8.5%) | Loss: 0.8670 | Acc: 0.641 | Elapsed: 1.8s | ETA: 19.1s
  Batch 5/47 (10.6%) | Loss: 0.8862 | Acc: 0.613 | Elapsed: 2.1s | ETA: 17.5s
  Batch 6/47 (12.8%) | Loss: 1.0127 | Acc: 0.578 | Elapsed: 2.4s | ETA: 16.5s
  Batch 7/47 (14.9%) | Loss: 1.0001 | Acc: 0.567 | Elapsed: 2.7s | ETA: 15.7s
  Batch 8/47 (17.0%) | Loss: 1.0296 | Acc: 0.555 | Elapsed: 3.1s | ETA: 14.9s
  Batch 9/47 (19.1%) | Loss: 1.0272 | Acc: 0.549 | Elapsed: 3.4s | ETA: 14.2s
  Batch 10/47 (21.3%) | Loss: 1.0226 | Acc: 0.556 | Elapsed: 3.7s | ETA: 13.7s
  Batch 11/47 (23.4%) | Loss: 1.0182 | Acc: 0.557 | Elapsed: 4.0s | ETA: 13.1s
  Batch 12/47 (25.5%) | Loss: 0.9968 | Acc: 0.565 | Elapsed: 4.3s | ETA: 12.6s
  Batch 13/47 (27.7%) | Loss: 1.0285 | Acc: 0.553 | Elapsed: 4.7s | ETA: 12.2s
  Batch 14/47 (29.8%) | Loss: 1.0254 | Acc: 0.551 | Elapsed: 5.0s | ETA: 11.7s
  Batch 15/47 (31.9%) | Loss: 1.0266 | Acc: 0.548 | Elapsed: 5.3s | ETA: 11.3s
  Batch 16/47 (34.0%) | Loss: 1.0228 | Acc: 0.555 | Elapsed: 5.6s | ETA: 10.9s
  Batch 17/47 (36.2%) | Loss: 1.0229 | Acc: 0.555 | Elapsed: 5.9s | ETA: 10.5s
  Batch 18/47 (38.3%) | Loss: 1.0174 | Acc: 0.556 | Elapsed: 6.3s | ETA: 10.1s
  Batch 19/47 (40.4%) | Loss: 1.0179 | Acc: 0.553 | Elapsed: 6.6s | ETA: 9.7s
  Batch 20/47 (42.6%) | Loss: 1.0055 | Acc: 0.559 | Elapsed: 6.9s | ETA: 9.3s
  Batch 21/47 (44.7%) | Loss: 0.9980 | Acc: 0.562 | Elapsed: 7.2s | ETA: 9.0s
  Batch 22/47 (46.8%) | Loss: 1.0095 | Acc: 0.562 | Elapsed: 7.6s | ETA: 8.6s
  Batch 23/47 (48.9%) | Loss: 1.0092 | Acc: 0.561 | Elapsed: 7.9s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 1.0019 | Acc: 0.564 | Elapsed: 8.2s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 1.0061 | Acc: 0.561 | Elapsed: 8.6s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 1.0005 | Acc: 0.564 | Elapsed: 8.9s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 1.0038 | Acc: 0.561 | Elapsed: 9.2s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.9981 | Acc: 0.564 | Elapsed: 9.6s | ETA: 6.5s
  Batch 29/47 (61.7%) | Loss: 1.0037 | Acc: 0.560 | Elapsed: 9.9s | ETA: 6.2s
  Batch 30/47 (63.8%) | Loss: 1.0015 | Acc: 0.560 | Elapsed: 10.3s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.9998 | Acc: 0.561 | Elapsed: 10.6s | ETA: 5.5s
  Batch 32/47 (68.1%) | Loss: 1.0029 | Acc: 0.562 | Elapsed: 10.9s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.9982 | Acc: 0.565 | Elapsed: 11.3s | ETA: 4.8s
  Batch 34/47 (72.3%) | Loss: 0.9966 | Acc: 0.565 | Elapsed: 11.7s | ETA: 4.5s
  Batch 35/47 (74.5%) | Loss: 0.9886 | Acc: 0.567 | Elapsed: 12.0s | ETA: 4.1s
  Batch 36/47 (76.6%) | Loss: 0.9876 | Acc: 0.569 | Elapsed: 12.4s | ETA: 3.8s
  Batch 37/47 (78.7%) | Loss: 0.9875 | Acc: 0.569 | Elapsed: 12.7s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.9826 | Acc: 0.573 | Elapsed: 13.0s | ETA: 3.1s
  Batch 39/47 (83.0%) | Loss: 0.9850 | Acc: 0.573 | Elapsed: 13.4s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.9878 | Acc: 0.573 | Elapsed: 13.7s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.9830 | Acc: 0.575 | Elapsed: 14.0s | ETA: 2.1s
  Batch 42/47 (89.4%) | Loss: 0.9842 | Acc: 0.574 | Elapsed: 14.4s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.9845 | Acc: 0.577 | Elapsed: 14.7s | ETA: 1.4s
  Batch 44/47 (93.6%) | Loss: 0.9747 | Acc: 0.582 | Elapsed: 15.0s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.9711 | Acc: 0.583 | Elapsed: 15.3s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.9679 | Acc: 0.586 | Elapsed: 15.7s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.9709 | Acc: 0.586 | Elapsed: 15.7s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.5597 | Acc: 0.429
Epoch 3 completed in 21.8s

--- Epoch 4/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.7428 | Acc: 0.719 | Elapsed: 1.1s | ETA: 50.2s
  Batch 2/47 (4.3%) | Loss: 0.7804 | Acc: 0.734 | Elapsed: 1.4s | ETA: 31.9s
  Batch 3/47 (6.4%) | Loss: 0.7409 | Acc: 0.719 | Elapsed: 1.7s | ETA: 25.5s
  Batch 4/47 (8.5%) | Loss: 0.7355 | Acc: 0.719 | Elapsed: 2.1s | ETA: 22.4s
  Batch 5/47 (10.6%) | Loss: 0.7527 | Acc: 0.700 | Elapsed: 2.4s | ETA: 20.3s
  Batch 6/47 (12.8%) | Loss: 0.7783 | Acc: 0.682 | Elapsed: 2.7s | ETA: 18.7s
  Batch 7/47 (14.9%) | Loss: 0.7531 | Acc: 0.705 | Elapsed: 3.1s | ETA: 17.6s
  Batch 8/47 (17.0%) | Loss: 0.7810 | Acc: 0.691 | Elapsed: 3.4s | ETA: 16.6s
  Batch 9/47 (19.1%) | Loss: 0.7665 | Acc: 0.694 | Elapsed: 3.7s | ETA: 15.7s
  Batch 10/47 (21.3%) | Loss: 0.7742 | Acc: 0.678 | Elapsed: 4.1s | ETA: 15.0s
  Batch 11/47 (23.4%) | Loss: 0.8060 | Acc: 0.668 | Elapsed: 4.4s | ETA: 14.4s
  Batch 12/47 (25.5%) | Loss: 0.7949 | Acc: 0.677 | Elapsed: 4.7s | ETA: 13.7s
  Batch 13/47 (27.7%) | Loss: 0.7714 | Acc: 0.690 | Elapsed: 5.0s | ETA: 13.2s
  Batch 14/47 (29.8%) | Loss: 0.7809 | Acc: 0.685 | Elapsed: 5.4s | ETA: 12.7s
  Batch 15/47 (31.9%) | Loss: 0.7796 | Acc: 0.685 | Elapsed: 5.7s | ETA: 12.1s
  Batch 16/47 (34.0%) | Loss: 0.7804 | Acc: 0.684 | Elapsed: 6.0s | ETA: 11.7s
  Batch 17/47 (36.2%) | Loss: 0.7819 | Acc: 0.686 | Elapsed: 6.4s | ETA: 11.2s
  Batch 18/47 (38.3%) | Loss: 0.7867 | Acc: 0.684 | Elapsed: 6.7s | ETA: 10.8s
  Batch 19/47 (40.4%) | Loss: 0.8053 | Acc: 0.674 | Elapsed: 7.0s | ETA: 10.3s
  Batch 20/47 (42.6%) | Loss: 0.8089 | Acc: 0.672 | Elapsed: 7.3s | ETA: 9.9s
  Batch 21/47 (44.7%) | Loss: 0.8212 | Acc: 0.670 | Elapsed: 7.7s | ETA: 9.5s
  Batch 22/47 (46.8%) | Loss: 0.8274 | Acc: 0.666 | Elapsed: 8.0s | ETA: 9.1s
  Batch 23/47 (48.9%) | Loss: 0.8357 | Acc: 0.668 | Elapsed: 8.3s | ETA: 8.7s
  Batch 24/47 (51.1%) | Loss: 0.8344 | Acc: 0.669 | Elapsed: 8.6s | ETA: 8.3s
  Batch 25/47 (53.2%) | Loss: 0.8312 | Acc: 0.670 | Elapsed: 9.0s | ETA: 7.9s
  Batch 26/47 (55.3%) | Loss: 0.8295 | Acc: 0.673 | Elapsed: 9.3s | ETA: 7.5s
  Batch 27/47 (57.4%) | Loss: 0.8326 | Acc: 0.668 | Elapsed: 9.6s | ETA: 7.1s
  Batch 28/47 (59.6%) | Loss: 0.8333 | Acc: 0.663 | Elapsed: 9.9s | ETA: 6.7s
  Batch 29/47 (61.7%) | Loss: 0.8316 | Acc: 0.664 | Elapsed: 10.3s | ETA: 6.4s
  Batch 30/47 (63.8%) | Loss: 0.8287 | Acc: 0.666 | Elapsed: 10.6s | ETA: 6.0s
  Batch 31/47 (66.0%) | Loss: 0.8336 | Acc: 0.667 | Elapsed: 10.9s | ETA: 5.6s
  Batch 32/47 (68.1%) | Loss: 0.8322 | Acc: 0.666 | Elapsed: 11.3s | ETA: 5.3s
  Batch 33/47 (70.2%) | Loss: 0.8318 | Acc: 0.668 | Elapsed: 11.6s | ETA: 4.9s
  Batch 34/47 (72.3%) | Loss: 0.8353 | Acc: 0.664 | Elapsed: 11.9s | ETA: 4.6s
  Batch 35/47 (74.5%) | Loss: 0.8357 | Acc: 0.664 | Elapsed: 12.3s | ETA: 4.2s
  Batch 36/47 (76.6%) | Loss: 0.8364 | Acc: 0.661 | Elapsed: 12.6s | ETA: 3.8s
  Batch 37/47 (78.7%) | Loss: 0.8406 | Acc: 0.660 | Elapsed: 12.9s | ETA: 3.5s
  Batch 38/47 (80.9%) | Loss: 0.8398 | Acc: 0.659 | Elapsed: 13.3s | ETA: 3.1s
  Batch 39/47 (83.0%) | Loss: 0.8365 | Acc: 0.659 | Elapsed: 13.6s | ETA: 2.8s
  Batch 40/47 (85.1%) | Loss: 0.8388 | Acc: 0.655 | Elapsed: 13.9s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.8377 | Acc: 0.658 | Elapsed: 14.3s | ETA: 2.1s
  Batch 42/47 (89.4%) | Loss: 0.8364 | Acc: 0.658 | Elapsed: 14.6s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.8352 | Acc: 0.656 | Elapsed: 14.9s | ETA: 1.4s
  Batch 44/47 (93.6%) | Loss: 0.8388 | Acc: 0.653 | Elapsed: 15.2s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.8423 | Acc: 0.651 | Elapsed: 15.6s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.8415 | Acc: 0.651 | Elapsed: 15.9s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.8417 | Acc: 0.651 | Elapsed: 16.0s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.5079 | Acc: 0.464
Epoch 4 completed in 20.7s

--- Epoch 5/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.9448 | Acc: 0.625 | Elapsed: 0.8s | ETA: 37.0s
  Batch 2/47 (4.3%) | Loss: 0.7826 | Acc: 0.734 | Elapsed: 1.1s | ETA: 25.1s
  Batch 3/47 (6.4%) | Loss: 0.8560 | Acc: 0.677 | Elapsed: 1.5s | ETA: 21.4s
  Batch 4/47 (8.5%) | Loss: 0.8252 | Acc: 0.688 | Elapsed: 1.8s | ETA: 19.1s
  Batch 5/47 (10.6%) | Loss: 0.7697 | Acc: 0.694 | Elapsed: 2.1s | ETA: 17.5s
  Batch 6/47 (12.8%) | Loss: 0.7565 | Acc: 0.688 | Elapsed: 2.4s | ETA: 16.4s
  Batch 7/47 (14.9%) | Loss: 0.7375 | Acc: 0.701 | Elapsed: 2.7s | ETA: 15.6s
  Batch 8/47 (17.0%) | Loss: 0.7330 | Acc: 0.703 | Elapsed: 3.0s | ETA: 14.8s
  Batch 9/47 (19.1%) | Loss: 0.7339 | Acc: 0.698 | Elapsed: 3.4s | ETA: 14.2s
  Batch 10/47 (21.3%) | Loss: 0.7266 | Acc: 0.706 | Elapsed: 3.7s | ETA: 13.6s
  Batch 11/47 (23.4%) | Loss: 0.7393 | Acc: 0.693 | Elapsed: 4.0s | ETA: 13.1s
  Batch 12/47 (25.5%) | Loss: 0.7385 | Acc: 0.695 | Elapsed: 4.3s | ETA: 12.6s
  Batch 13/47 (27.7%) | Loss: 0.7677 | Acc: 0.685 | Elapsed: 4.6s | ETA: 12.1s
  Batch 14/47 (29.8%) | Loss: 0.7721 | Acc: 0.688 | Elapsed: 5.0s | ETA: 11.7s
  Batch 15/47 (31.9%) | Loss: 0.7753 | Acc: 0.685 | Elapsed: 5.3s | ETA: 11.2s
  Batch 16/47 (34.0%) | Loss: 0.7750 | Acc: 0.682 | Elapsed: 5.6s | ETA: 10.8s
  Batch 17/47 (36.2%) | Loss: 0.7671 | Acc: 0.684 | Elapsed: 5.9s | ETA: 10.4s
  Batch 18/47 (38.3%) | Loss: 0.8061 | Acc: 0.668 | Elapsed: 6.2s | ETA: 10.0s
  Batch 19/47 (40.4%) | Loss: 0.8040 | Acc: 0.664 | Elapsed: 6.5s | ETA: 9.6s
  Batch 20/47 (42.6%) | Loss: 0.8047 | Acc: 0.662 | Elapsed: 6.9s | ETA: 9.3s
  Batch 21/47 (44.7%) | Loss: 0.8081 | Acc: 0.662 | Elapsed: 7.2s | ETA: 8.9s
  Batch 22/47 (46.8%) | Loss: 0.8004 | Acc: 0.662 | Elapsed: 7.5s | ETA: 8.5s
  Batch 23/47 (48.9%) | Loss: 0.8083 | Acc: 0.658 | Elapsed: 7.8s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 0.8087 | Acc: 0.656 | Elapsed: 8.2s | ETA: 7.8s
  Batch 25/47 (53.2%) | Loss: 0.8093 | Acc: 0.656 | Elapsed: 8.5s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.8017 | Acc: 0.656 | Elapsed: 8.8s | ETA: 7.1s
  Batch 27/47 (57.4%) | Loss: 0.7998 | Acc: 0.657 | Elapsed: 9.1s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.7957 | Acc: 0.661 | Elapsed: 9.4s | ETA: 6.4s
  Batch 29/47 (61.7%) | Loss: 0.7915 | Acc: 0.663 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.8085 | Acc: 0.661 | Elapsed: 10.1s | ETA: 5.7s
  Batch 31/47 (66.0%) | Loss: 0.8016 | Acc: 0.665 | Elapsed: 10.4s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.8061 | Acc: 0.661 | Elapsed: 10.7s | ETA: 5.0s
  Batch 33/47 (70.2%) | Loss: 0.8114 | Acc: 0.659 | Elapsed: 11.1s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.8111 | Acc: 0.658 | Elapsed: 11.4s | ETA: 4.3s
  Batch 35/47 (74.5%) | Loss: 0.8165 | Acc: 0.655 | Elapsed: 11.7s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 0.8173 | Acc: 0.658 | Elapsed: 12.0s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.8184 | Acc: 0.660 | Elapsed: 12.3s | ETA: 3.3s
  Batch 38/47 (80.9%) | Loss: 0.8198 | Acc: 0.657 | Elapsed: 12.6s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.8178 | Acc: 0.659 | Elapsed: 13.0s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.8166 | Acc: 0.659 | Elapsed: 13.3s | ETA: 2.3s
  Batch 41/47 (87.2%) | Loss: 0.8132 | Acc: 0.662 | Elapsed: 13.6s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.8104 | Acc: 0.662 | Elapsed: 13.9s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.8079 | Acc: 0.663 | Elapsed: 14.3s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.8105 | Acc: 0.660 | Elapsed: 14.6s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.8077 | Acc: 0.661 | Elapsed: 14.9s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.8120 | Acc: 0.661 | Elapsed: 15.2s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.8117 | Acc: 0.661 | Elapsed: 15.3s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.5978 | Acc: 0.472
Epoch 5 completed in 19.6s

--- Epoch 6/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.6999 | Acc: 0.688 | Elapsed: 1.0s | ETA: 45.9s
  Batch 2/47 (4.3%) | Loss: 0.7360 | Acc: 0.688 | Elapsed: 1.3s | ETA: 29.4s
  Batch 3/47 (6.4%) | Loss: 0.7439 | Acc: 0.667 | Elapsed: 1.7s | ETA: 24.3s
  Batch 4/47 (8.5%) | Loss: 0.7335 | Acc: 0.703 | Elapsed: 2.0s | ETA: 21.2s
  Batch 5/47 (10.6%) | Loss: 0.7223 | Acc: 0.713 | Elapsed: 2.3s | ETA: 19.3s
  Batch 6/47 (12.8%) | Loss: 0.7077 | Acc: 0.729 | Elapsed: 2.6s | ETA: 17.9s
  Batch 7/47 (14.9%) | Loss: 0.7229 | Acc: 0.714 | Elapsed: 2.9s | ETA: 16.8s
  Batch 8/47 (17.0%) | Loss: 0.7866 | Acc: 0.703 | Elapsed: 3.3s | ETA: 15.9s
  Batch 9/47 (19.1%) | Loss: 0.8092 | Acc: 0.698 | Elapsed: 3.6s | ETA: 15.1s
  Batch 10/47 (21.3%) | Loss: 0.8428 | Acc: 0.681 | Elapsed: 3.9s | ETA: 14.4s
  Batch 11/47 (23.4%) | Loss: 0.8210 | Acc: 0.685 | Elapsed: 4.2s | ETA: 13.8s
  Batch 12/47 (25.5%) | Loss: 0.8283 | Acc: 0.674 | Elapsed: 4.5s | ETA: 13.2s
  Batch 13/47 (27.7%) | Loss: 0.8329 | Acc: 0.673 | Elapsed: 4.9s | ETA: 12.7s
  Batch 14/47 (29.8%) | Loss: 0.8280 | Acc: 0.670 | Elapsed: 5.2s | ETA: 12.2s
  Batch 15/47 (31.9%) | Loss: 0.8240 | Acc: 0.669 | Elapsed: 5.5s | ETA: 11.7s
  Batch 16/47 (34.0%) | Loss: 0.8336 | Acc: 0.664 | Elapsed: 5.8s | ETA: 11.2s
  Batch 17/47 (36.2%) | Loss: 0.8247 | Acc: 0.673 | Elapsed: 6.1s | ETA: 10.8s
  Batch 18/47 (38.3%) | Loss: 0.8197 | Acc: 0.672 | Elapsed: 6.4s | ETA: 10.4s
  Batch 19/47 (40.4%) | Loss: 0.8188 | Acc: 0.669 | Elapsed: 6.7s | ETA: 9.9s
  Batch 20/47 (42.6%) | Loss: 0.8147 | Acc: 0.673 | Elapsed: 7.1s | ETA: 9.5s
  Batch 21/47 (44.7%) | Loss: 0.8177 | Acc: 0.674 | Elapsed: 7.4s | ETA: 9.1s
  Batch 22/47 (46.8%) | Loss: 0.8081 | Acc: 0.680 | Elapsed: 7.7s | ETA: 8.8s
  Batch 23/47 (48.9%) | Loss: 0.8034 | Acc: 0.682 | Elapsed: 8.0s | ETA: 8.4s
  Batch 24/47 (51.1%) | Loss: 0.7994 | Acc: 0.682 | Elapsed: 8.3s | ETA: 8.0s
  Batch 25/47 (53.2%) | Loss: 0.7986 | Acc: 0.681 | Elapsed: 8.6s | ETA: 7.6s
  Batch 26/47 (55.3%) | Loss: 0.7915 | Acc: 0.681 | Elapsed: 9.0s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 0.7882 | Acc: 0.679 | Elapsed: 9.3s | ETA: 6.9s
  Batch 28/47 (59.6%) | Loss: 0.7778 | Acc: 0.685 | Elapsed: 9.6s | ETA: 6.5s
  Batch 29/47 (61.7%) | Loss: 0.7735 | Acc: 0.684 | Elapsed: 9.9s | ETA: 6.2s
  Batch 30/47 (63.8%) | Loss: 0.7730 | Acc: 0.683 | Elapsed: 10.2s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.7725 | Acc: 0.682 | Elapsed: 10.6s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.7768 | Acc: 0.681 | Elapsed: 10.9s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.7763 | Acc: 0.683 | Elapsed: 11.2s | ETA: 4.8s
  Batch 34/47 (72.3%) | Loss: 0.7914 | Acc: 0.680 | Elapsed: 11.5s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.7942 | Acc: 0.674 | Elapsed: 11.9s | ETA: 4.1s
  Batch 36/47 (76.6%) | Loss: 0.7972 | Acc: 0.674 | Elapsed: 12.2s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.7951 | Acc: 0.671 | Elapsed: 12.5s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.7951 | Acc: 0.673 | Elapsed: 12.8s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.7913 | Acc: 0.675 | Elapsed: 13.1s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.7958 | Acc: 0.673 | Elapsed: 13.5s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.7985 | Acc: 0.671 | Elapsed: 13.8s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.8023 | Acc: 0.668 | Elapsed: 14.1s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.8075 | Acc: 0.666 | Elapsed: 14.5s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.8062 | Acc: 0.666 | Elapsed: 14.8s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.8043 | Acc: 0.665 | Elapsed: 15.1s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.8084 | Acc: 0.662 | Elapsed: 15.4s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.8108 | Acc: 0.661 | Elapsed: 15.5s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 3.4632 | Acc: 0.386
Epoch 6 completed in 20.1s

--- Epoch 7/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.7231 | Acc: 0.719 | Elapsed: 0.9s | ETA: 39.8s
  Batch 2/47 (4.3%) | Loss: 1.0105 | Acc: 0.625 | Elapsed: 1.2s | ETA: 26.5s
  Batch 3/47 (6.4%) | Loss: 1.0320 | Acc: 0.615 | Elapsed: 1.5s | ETA: 22.4s
  Batch 4/47 (8.5%) | Loss: 0.9765 | Acc: 0.609 | Elapsed: 1.8s | ETA: 19.8s
  Batch 5/47 (10.6%) | Loss: 1.0082 | Acc: 0.600 | Elapsed: 2.2s | ETA: 18.2s
  Batch 6/47 (12.8%) | Loss: 0.9524 | Acc: 0.615 | Elapsed: 2.5s | ETA: 17.0s
  Batch 7/47 (14.9%) | Loss: 0.9548 | Acc: 0.616 | Elapsed: 2.8s | ETA: 16.0s
  Batch 8/47 (17.0%) | Loss: 0.9241 | Acc: 0.637 | Elapsed: 3.1s | ETA: 15.2s
  Batch 9/47 (19.1%) | Loss: 0.8890 | Acc: 0.649 | Elapsed: 3.4s | ETA: 14.5s
  Batch 10/47 (21.3%) | Loss: 0.8720 | Acc: 0.662 | Elapsed: 3.8s | ETA: 13.9s
  Batch 11/47 (23.4%) | Loss: 0.8656 | Acc: 0.659 | Elapsed: 4.1s | ETA: 13.3s
  Batch 12/47 (25.5%) | Loss: 0.9156 | Acc: 0.651 | Elapsed: 4.4s | ETA: 12.8s
  Batch 13/47 (27.7%) | Loss: 0.9034 | Acc: 0.651 | Elapsed: 4.7s | ETA: 12.3s
  Batch 14/47 (29.8%) | Loss: 0.9221 | Acc: 0.636 | Elapsed: 5.0s | ETA: 11.9s
  Batch 15/47 (31.9%) | Loss: 0.9221 | Acc: 0.631 | Elapsed: 5.4s | ETA: 11.4s
  Batch 16/47 (34.0%) | Loss: 0.9016 | Acc: 0.641 | Elapsed: 5.7s | ETA: 11.0s
  Batch 17/47 (36.2%) | Loss: 0.8975 | Acc: 0.640 | Elapsed: 6.0s | ETA: 10.6s
  Batch 18/47 (38.3%) | Loss: 0.8942 | Acc: 0.646 | Elapsed: 6.3s | ETA: 10.2s
  Batch 19/47 (40.4%) | Loss: 0.8874 | Acc: 0.650 | Elapsed: 6.6s | ETA: 9.8s
  Batch 20/47 (42.6%) | Loss: 0.8873 | Acc: 0.648 | Elapsed: 7.0s | ETA: 9.4s
  Batch 21/47 (44.7%) | Loss: 0.8731 | Acc: 0.653 | Elapsed: 7.3s | ETA: 9.0s
  Batch 22/47 (46.8%) | Loss: 0.8645 | Acc: 0.655 | Elapsed: 7.6s | ETA: 8.7s
  Batch 23/47 (48.9%) | Loss: 0.8626 | Acc: 0.662 | Elapsed: 7.9s | ETA: 8.3s
  Batch 24/47 (51.1%) | Loss: 0.8620 | Acc: 0.660 | Elapsed: 8.3s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 0.8587 | Acc: 0.662 | Elapsed: 8.6s | ETA: 7.6s
  Batch 26/47 (55.3%) | Loss: 0.8568 | Acc: 0.661 | Elapsed: 8.9s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 0.8479 | Acc: 0.662 | Elapsed: 9.2s | ETA: 6.9s
  Batch 28/47 (59.6%) | Loss: 0.8491 | Acc: 0.661 | Elapsed: 9.6s | ETA: 6.5s
  Batch 29/47 (61.7%) | Loss: 0.8369 | Acc: 0.665 | Elapsed: 9.9s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.8379 | Acc: 0.662 | Elapsed: 10.2s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.8359 | Acc: 0.659 | Elapsed: 10.6s | ETA: 5.5s
  Batch 32/47 (68.1%) | Loss: 0.8342 | Acc: 0.659 | Elapsed: 10.9s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.8407 | Acc: 0.659 | Elapsed: 11.2s | ETA: 4.8s
  Batch 34/47 (72.3%) | Loss: 0.8388 | Acc: 0.659 | Elapsed: 11.6s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.8358 | Acc: 0.660 | Elapsed: 11.9s | ETA: 4.1s
  Batch 36/47 (76.6%) | Loss: 0.8367 | Acc: 0.658 | Elapsed: 12.2s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.8383 | Acc: 0.657 | Elapsed: 12.5s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.8329 | Acc: 0.659 | Elapsed: 12.9s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.8317 | Acc: 0.656 | Elapsed: 13.2s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.8351 | Acc: 0.655 | Elapsed: 13.5s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.8364 | Acc: 0.652 | Elapsed: 13.8s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.8381 | Acc: 0.650 | Elapsed: 14.2s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.8286 | Acc: 0.655 | Elapsed: 14.5s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.8287 | Acc: 0.656 | Elapsed: 14.8s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.8244 | Acc: 0.658 | Elapsed: 15.1s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.8221 | Acc: 0.659 | Elapsed: 15.5s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.8223 | Acc: 0.659 | Elapsed: 15.5s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.2718 | Acc: 0.476
Epoch 7 completed in 18.1s

--- Epoch 8/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.6732 | Acc: 0.812 | Elapsed: 0.8s | ETA: 36.7s
  Batch 2/47 (4.3%) | Loss: 0.6528 | Acc: 0.750 | Elapsed: 1.1s | ETA: 25.0s
  Batch 3/47 (6.4%) | Loss: 0.6250 | Acc: 0.750 | Elapsed: 1.5s | ETA: 21.4s
  Batch 4/47 (8.5%) | Loss: 0.6083 | Acc: 0.773 | Elapsed: 1.8s | ETA: 19.1s
  Batch 5/47 (10.6%) | Loss: 0.6579 | Acc: 0.762 | Elapsed: 2.1s | ETA: 17.6s
  Batch 6/47 (12.8%) | Loss: 0.6673 | Acc: 0.760 | Elapsed: 2.4s | ETA: 16.6s
  Batch 7/47 (14.9%) | Loss: 0.6757 | Acc: 0.750 | Elapsed: 2.8s | ETA: 15.7s
  Batch 8/47 (17.0%) | Loss: 0.7115 | Acc: 0.734 | Elapsed: 3.1s | ETA: 15.0s
  Batch 9/47 (19.1%) | Loss: 0.7061 | Acc: 0.736 | Elapsed: 3.4s | ETA: 14.3s
  Batch 10/47 (21.3%) | Loss: 0.6884 | Acc: 0.741 | Elapsed: 3.7s | ETA: 13.8s
  Batch 11/47 (23.4%) | Loss: 0.6908 | Acc: 0.733 | Elapsed: 4.0s | ETA: 13.2s
  Batch 12/47 (25.5%) | Loss: 0.6848 | Acc: 0.732 | Elapsed: 4.4s | ETA: 12.8s
  Batch 13/47 (27.7%) | Loss: 0.7076 | Acc: 0.721 | Elapsed: 4.7s | ETA: 12.3s
  Batch 14/47 (29.8%) | Loss: 0.7067 | Acc: 0.717 | Elapsed: 5.0s | ETA: 11.9s
  Batch 15/47 (31.9%) | Loss: 0.7141 | Acc: 0.710 | Elapsed: 5.4s | ETA: 11.4s
  Batch 16/47 (34.0%) | Loss: 0.7045 | Acc: 0.719 | Elapsed: 5.7s | ETA: 11.0s
  Batch 17/47 (36.2%) | Loss: 0.7047 | Acc: 0.713 | Elapsed: 6.0s | ETA: 10.6s
  Batch 18/47 (38.3%) | Loss: 0.7257 | Acc: 0.708 | Elapsed: 6.3s | ETA: 10.2s
  Batch 19/47 (40.4%) | Loss: 0.7286 | Acc: 0.707 | Elapsed: 6.7s | ETA: 9.8s
  Batch 20/47 (42.6%) | Loss: 0.7298 | Acc: 0.709 | Elapsed: 7.0s | ETA: 9.4s
  Batch 21/47 (44.7%) | Loss: 0.7208 | Acc: 0.711 | Elapsed: 7.3s | ETA: 9.1s
  Batch 22/47 (46.8%) | Loss: 0.7125 | Acc: 0.716 | Elapsed: 7.6s | ETA: 8.7s
  Batch 23/47 (48.9%) | Loss: 0.7142 | Acc: 0.711 | Elapsed: 8.0s | ETA: 8.3s
  Batch 24/47 (51.1%) | Loss: 0.7024 | Acc: 0.719 | Elapsed: 8.3s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 0.6957 | Acc: 0.721 | Elapsed: 8.6s | ETA: 7.6s
  Batch 26/47 (55.3%) | Loss: 0.6932 | Acc: 0.720 | Elapsed: 8.9s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 0.6974 | Acc: 0.719 | Elapsed: 9.3s | ETA: 6.9s
  Batch 28/47 (59.6%) | Loss: 0.6934 | Acc: 0.721 | Elapsed: 9.6s | ETA: 6.5s
  Batch 29/47 (61.7%) | Loss: 0.6929 | Acc: 0.723 | Elapsed: 9.9s | ETA: 6.2s
  Batch 30/47 (63.8%) | Loss: 0.6885 | Acc: 0.727 | Elapsed: 10.2s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.6831 | Acc: 0.731 | Elapsed: 10.6s | ETA: 5.5s
  Batch 32/47 (68.1%) | Loss: 0.6918 | Acc: 0.727 | Elapsed: 10.9s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.6967 | Acc: 0.720 | Elapsed: 11.2s | ETA: 4.8s
  Batch 34/47 (72.3%) | Loss: 0.6997 | Acc: 0.715 | Elapsed: 11.6s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.6968 | Acc: 0.715 | Elapsed: 11.9s | ETA: 4.1s
  Batch 36/47 (76.6%) | Loss: 0.6915 | Acc: 0.717 | Elapsed: 12.2s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.6967 | Acc: 0.715 | Elapsed: 12.6s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.7058 | Acc: 0.711 | Elapsed: 12.9s | ETA: 3.1s
  Batch 39/47 (83.0%) | Loss: 0.7054 | Acc: 0.708 | Elapsed: 13.2s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.7040 | Acc: 0.709 | Elapsed: 13.5s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.7056 | Acc: 0.707 | Elapsed: 13.9s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.7059 | Acc: 0.707 | Elapsed: 14.2s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.7015 | Acc: 0.710 | Elapsed: 14.5s | ETA: 1.4s
  Batch 44/47 (93.6%) | Loss: 0.7000 | Acc: 0.711 | Elapsed: 14.9s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.7015 | Acc: 0.710 | Elapsed: 15.2s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.7005 | Acc: 0.711 | Elapsed: 15.5s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.7034 | Acc: 0.711 | Elapsed: 15.6s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.7159 | Acc: 0.493
Epoch 8 completed in 21.0s

--- Epoch 9/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.4790 | Acc: 0.812 | Elapsed: 0.8s | ETA: 36.9s
  Batch 2/47 (4.3%) | Loss: 0.5712 | Acc: 0.766 | Elapsed: 1.1s | ETA: 25.3s
  Batch 3/47 (6.4%) | Loss: 0.6803 | Acc: 0.729 | Elapsed: 1.5s | ETA: 21.5s
  Batch 4/47 (8.5%) | Loss: 0.7035 | Acc: 0.695 | Elapsed: 1.8s | ETA: 19.2s
  Batch 5/47 (10.6%) | Loss: 0.7341 | Acc: 0.681 | Elapsed: 2.1s | ETA: 17.6s
  Batch 6/47 (12.8%) | Loss: 0.7379 | Acc: 0.688 | Elapsed: 2.4s | ETA: 16.6s
  Batch 7/47 (14.9%) | Loss: 0.7278 | Acc: 0.705 | Elapsed: 2.7s | ETA: 15.7s
  Batch 8/47 (17.0%) | Loss: 0.7545 | Acc: 0.699 | Elapsed: 3.1s | ETA: 15.0s
  Batch 9/47 (19.1%) | Loss: 0.7180 | Acc: 0.715 | Elapsed: 3.4s | ETA: 14.3s
  Batch 10/47 (21.3%) | Loss: 0.7283 | Acc: 0.703 | Elapsed: 3.7s | ETA: 13.7s
  Batch 11/47 (23.4%) | Loss: 0.7129 | Acc: 0.710 | Elapsed: 4.0s | ETA: 13.2s
  Batch 12/47 (25.5%) | Loss: 0.7224 | Acc: 0.708 | Elapsed: 4.3s | ETA: 12.7s
  Batch 13/47 (27.7%) | Loss: 0.7392 | Acc: 0.700 | Elapsed: 4.7s | ETA: 12.2s
  Batch 14/47 (29.8%) | Loss: 0.7343 | Acc: 0.703 | Elapsed: 5.0s | ETA: 11.8s
  Batch 15/47 (31.9%) | Loss: 0.7454 | Acc: 0.700 | Elapsed: 5.3s | ETA: 11.3s
  Batch 16/47 (34.0%) | Loss: 0.7407 | Acc: 0.701 | Elapsed: 5.6s | ETA: 10.9s
  Batch 17/47 (36.2%) | Loss: 0.7406 | Acc: 0.695 | Elapsed: 6.0s | ETA: 10.5s
  Batch 18/47 (38.3%) | Loss: 0.7453 | Acc: 0.689 | Elapsed: 6.3s | ETA: 10.1s
  Batch 19/47 (40.4%) | Loss: 0.7476 | Acc: 0.691 | Elapsed: 6.6s | ETA: 9.7s
  Batch 20/47 (42.6%) | Loss: 0.7406 | Acc: 0.691 | Elapsed: 6.9s | ETA: 9.3s
  Batch 21/47 (44.7%) | Loss: 0.7329 | Acc: 0.693 | Elapsed: 7.2s | ETA: 9.0s
  Batch 22/47 (46.8%) | Loss: 0.7205 | Acc: 0.699 | Elapsed: 7.6s | ETA: 8.6s
  Batch 23/47 (48.9%) | Loss: 0.7112 | Acc: 0.701 | Elapsed: 7.9s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 0.7135 | Acc: 0.703 | Elapsed: 8.2s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 0.7160 | Acc: 0.703 | Elapsed: 8.5s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.7138 | Acc: 0.708 | Elapsed: 8.8s | ETA: 7.1s
  Batch 27/47 (57.4%) | Loss: 0.7105 | Acc: 0.709 | Elapsed: 9.2s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.7042 | Acc: 0.712 | Elapsed: 9.5s | ETA: 6.4s
  Batch 29/47 (61.7%) | Loss: 0.7093 | Acc: 0.711 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.7172 | Acc: 0.711 | Elapsed: 10.2s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.7164 | Acc: 0.708 | Elapsed: 10.5s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.7154 | Acc: 0.705 | Elapsed: 10.8s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.7118 | Acc: 0.707 | Elapsed: 11.2s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.7081 | Acc: 0.708 | Elapsed: 11.5s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.7080 | Acc: 0.708 | Elapsed: 11.8s | ETA: 4.1s
  Batch 36/47 (76.6%) | Loss: 0.7000 | Acc: 0.712 | Elapsed: 12.1s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.7076 | Acc: 0.707 | Elapsed: 12.5s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.7129 | Acc: 0.705 | Elapsed: 12.8s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.7107 | Acc: 0.706 | Elapsed: 13.1s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.7040 | Acc: 0.709 | Elapsed: 13.5s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.7024 | Acc: 0.709 | Elapsed: 13.8s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.7041 | Acc: 0.706 | Elapsed: 14.1s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.7116 | Acc: 0.703 | Elapsed: 14.4s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.7098 | Acc: 0.703 | Elapsed: 14.8s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.7132 | Acc: 0.702 | Elapsed: 15.1s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.7094 | Acc: 0.705 | Elapsed: 15.4s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.7135 | Acc: 0.704 | Elapsed: 15.5s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.4721 | Acc: 0.470
Epoch 9 completed in 19.6s

--- Epoch 10/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 1.1130 | Acc: 0.562 | Elapsed: 0.9s | ETA: 39.4s
  Batch 2/47 (4.3%) | Loss: 0.8310 | Acc: 0.672 | Elapsed: 1.2s | ETA: 26.2s
  Batch 3/47 (6.4%) | Loss: 0.8645 | Acc: 0.677 | Elapsed: 1.5s | ETA: 22.2s
  Batch 4/47 (8.5%) | Loss: 0.8596 | Acc: 0.672 | Elapsed: 1.8s | ETA: 19.7s
  Batch 5/47 (10.6%) | Loss: 0.8459 | Acc: 0.662 | Elapsed: 2.1s | ETA: 17.9s
  Batch 6/47 (12.8%) | Loss: 0.8084 | Acc: 0.661 | Elapsed: 2.5s | ETA: 16.8s
  Batch 7/47 (14.9%) | Loss: 0.7870 | Acc: 0.674 | Elapsed: 2.8s | ETA: 15.9s
  Batch 8/47 (17.0%) | Loss: 0.8004 | Acc: 0.664 | Elapsed: 3.1s | ETA: 15.1s
  Batch 9/47 (19.1%) | Loss: 0.8318 | Acc: 0.653 | Elapsed: 3.4s | ETA: 14.4s
  Batch 10/47 (21.3%) | Loss: 0.8106 | Acc: 0.659 | Elapsed: 3.7s | ETA: 13.9s
  Batch 11/47 (23.4%) | Loss: 0.8071 | Acc: 0.653 | Elapsed: 4.1s | ETA: 13.3s
  Batch 12/47 (25.5%) | Loss: 0.8159 | Acc: 0.654 | Elapsed: 4.4s | ETA: 12.8s
  Batch 13/47 (27.7%) | Loss: 0.8045 | Acc: 0.656 | Elapsed: 4.7s | ETA: 12.4s
  Batch 14/47 (29.8%) | Loss: 0.8097 | Acc: 0.656 | Elapsed: 5.1s | ETA: 11.9s
  Batch 15/47 (31.9%) | Loss: 0.7932 | Acc: 0.662 | Elapsed: 5.4s | ETA: 11.5s
  Batch 16/47 (34.0%) | Loss: 0.7735 | Acc: 0.672 | Elapsed: 5.7s | ETA: 11.0s
  Batch 17/47 (36.2%) | Loss: 0.7665 | Acc: 0.678 | Elapsed: 6.0s | ETA: 10.6s
  Batch 18/47 (38.3%) | Loss: 0.7637 | Acc: 0.672 | Elapsed: 6.4s | ETA: 10.3s
  Batch 19/47 (40.4%) | Loss: 0.7662 | Acc: 0.676 | Elapsed: 6.7s | ETA: 9.9s
  Batch 20/47 (42.6%) | Loss: 0.7615 | Acc: 0.678 | Elapsed: 7.0s | ETA: 9.5s
  Batch 21/47 (44.7%) | Loss: 0.7521 | Acc: 0.686 | Elapsed: 7.4s | ETA: 9.1s
  Batch 22/47 (46.8%) | Loss: 0.7598 | Acc: 0.679 | Elapsed: 7.7s | ETA: 8.7s
  Batch 23/47 (48.9%) | Loss: 0.7574 | Acc: 0.683 | Elapsed: 8.0s | ETA: 8.3s
  Batch 24/47 (51.1%) | Loss: 0.7438 | Acc: 0.693 | Elapsed: 8.3s | ETA: 8.0s
  Batch 25/47 (53.2%) | Loss: 0.7390 | Acc: 0.694 | Elapsed: 8.6s | ETA: 7.6s
  Batch 26/47 (55.3%) | Loss: 0.7276 | Acc: 0.701 | Elapsed: 9.0s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 0.7295 | Acc: 0.703 | Elapsed: 9.3s | ETA: 6.9s
  Batch 28/47 (59.6%) | Loss: 0.7337 | Acc: 0.700 | Elapsed: 9.6s | ETA: 6.5s
  Batch 29/47 (61.7%) | Loss: 0.7244 | Acc: 0.703 | Elapsed: 10.0s | ETA: 6.2s
  Batch 30/47 (63.8%) | Loss: 0.7177 | Acc: 0.704 | Elapsed: 10.3s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.7165 | Acc: 0.704 | Elapsed: 10.6s | ETA: 5.5s
  Batch 32/47 (68.1%) | Loss: 0.7158 | Acc: 0.705 | Elapsed: 10.9s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.7136 | Acc: 0.705 | Elapsed: 11.2s | ETA: 4.8s
  Batch 34/47 (72.3%) | Loss: 0.7217 | Acc: 0.702 | Elapsed: 11.6s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.7120 | Acc: 0.709 | Elapsed: 11.9s | ETA: 4.1s
  Batch 36/47 (76.6%) | Loss: 0.7234 | Acc: 0.707 | Elapsed: 12.2s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.7249 | Acc: 0.705 | Elapsed: 12.5s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.7214 | Acc: 0.706 | Elapsed: 12.9s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.7243 | Acc: 0.705 | Elapsed: 13.2s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.7202 | Acc: 0.706 | Elapsed: 13.5s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.7173 | Acc: 0.707 | Elapsed: 13.8s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.7122 | Acc: 0.709 | Elapsed: 14.1s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.7059 | Acc: 0.711 | Elapsed: 14.5s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.7059 | Acc: 0.712 | Elapsed: 14.8s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.7020 | Acc: 0.713 | Elapsed: 15.1s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.7009 | Acc: 0.711 | Elapsed: 15.4s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.7012 | Acc: 0.711 | Elapsed: 15.5s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.9303 | Acc: 0.429
Epoch 10 completed in 17.2s

--- Epoch 11/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.8005 | Acc: 0.656 | Elapsed: 1.0s | ETA: 46.0s
  Batch 2/47 (4.3%) | Loss: 0.7597 | Acc: 0.688 | Elapsed: 1.3s | ETA: 29.7s
  Batch 3/47 (6.4%) | Loss: 0.7153 | Acc: 0.708 | Elapsed: 1.7s | ETA: 24.5s
  Batch 4/47 (8.5%) | Loss: 0.8286 | Acc: 0.672 | Elapsed: 2.0s | ETA: 21.4s
  Batch 5/47 (10.6%) | Loss: 0.7750 | Acc: 0.700 | Elapsed: 2.3s | ETA: 19.3s
  Batch 6/47 (12.8%) | Loss: 0.7345 | Acc: 0.708 | Elapsed: 2.6s | ETA: 18.0s
  Batch 7/47 (14.9%) | Loss: 0.7128 | Acc: 0.719 | Elapsed: 3.0s | ETA: 16.9s
  Batch 8/47 (17.0%) | Loss: 0.7137 | Acc: 0.723 | Elapsed: 3.3s | ETA: 15.9s
  Batch 9/47 (19.1%) | Loss: 0.7672 | Acc: 0.698 | Elapsed: 3.6s | ETA: 15.2s
  Batch 10/47 (21.3%) | Loss: 0.7542 | Acc: 0.700 | Elapsed: 3.9s | ETA: 14.5s
  Batch 11/47 (23.4%) | Loss: 0.7672 | Acc: 0.693 | Elapsed: 4.2s | ETA: 13.9s
  Batch 12/47 (25.5%) | Loss: 0.7654 | Acc: 0.682 | Elapsed: 4.6s | ETA: 13.3s
  Batch 13/47 (27.7%) | Loss: 0.7590 | Acc: 0.680 | Elapsed: 4.9s | ETA: 12.8s
  Batch 14/47 (29.8%) | Loss: 0.7774 | Acc: 0.683 | Elapsed: 5.2s | ETA: 12.3s
  Batch 15/47 (31.9%) | Loss: 0.7835 | Acc: 0.683 | Elapsed: 5.5s | ETA: 11.8s
  Batch 16/47 (34.0%) | Loss: 0.7904 | Acc: 0.676 | Elapsed: 5.8s | ETA: 11.3s
  Batch 17/47 (36.2%) | Loss: 0.7831 | Acc: 0.676 | Elapsed: 6.2s | ETA: 10.9s
  Batch 18/47 (38.3%) | Loss: 0.7972 | Acc: 0.670 | Elapsed: 6.5s | ETA: 10.5s
  Batch 19/47 (40.4%) | Loss: 0.8071 | Acc: 0.661 | Elapsed: 6.8s | ETA: 10.0s
  Batch 20/47 (42.6%) | Loss: 0.7910 | Acc: 0.672 | Elapsed: 7.1s | ETA: 9.6s
  Batch 21/47 (44.7%) | Loss: 0.7926 | Acc: 0.671 | Elapsed: 7.5s | ETA: 9.2s
  Batch 22/47 (46.8%) | Loss: 0.7914 | Acc: 0.672 | Elapsed: 7.8s | ETA: 8.8s
  Batch 23/47 (48.9%) | Loss: 0.7812 | Acc: 0.678 | Elapsed: 8.1s | ETA: 8.5s
  Batch 24/47 (51.1%) | Loss: 0.7853 | Acc: 0.673 | Elapsed: 8.4s | ETA: 8.1s
  Batch 25/47 (53.2%) | Loss: 0.7751 | Acc: 0.675 | Elapsed: 8.7s | ETA: 7.7s
  Batch 26/47 (55.3%) | Loss: 0.7699 | Acc: 0.675 | Elapsed: 9.1s | ETA: 7.3s
  Batch 27/47 (57.4%) | Loss: 0.7779 | Acc: 0.672 | Elapsed: 9.4s | ETA: 7.0s
  Batch 28/47 (59.6%) | Loss: 0.7767 | Acc: 0.671 | Elapsed: 9.7s | ETA: 6.6s
  Batch 29/47 (61.7%) | Loss: 0.7721 | Acc: 0.675 | Elapsed: 10.0s | ETA: 6.2s
  Batch 30/47 (63.8%) | Loss: 0.7611 | Acc: 0.683 | Elapsed: 10.4s | ETA: 5.9s
  Batch 31/47 (66.0%) | Loss: 0.7558 | Acc: 0.684 | Elapsed: 10.7s | ETA: 5.5s
  Batch 32/47 (68.1%) | Loss: 0.7453 | Acc: 0.688 | Elapsed: 11.0s | ETA: 5.2s
  Batch 33/47 (70.2%) | Loss: 0.7376 | Acc: 0.691 | Elapsed: 11.3s | ETA: 4.8s
  Batch 34/47 (72.3%) | Loss: 0.7331 | Acc: 0.692 | Elapsed: 11.7s | ETA: 4.5s
  Batch 35/47 (74.5%) | Loss: 0.7344 | Acc: 0.692 | Elapsed: 12.0s | ETA: 4.1s
  Batch 36/47 (76.6%) | Loss: 0.7293 | Acc: 0.694 | Elapsed: 12.3s | ETA: 3.8s
  Batch 37/47 (78.7%) | Loss: 0.7336 | Acc: 0.692 | Elapsed: 12.6s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.7293 | Acc: 0.693 | Elapsed: 13.0s | ETA: 3.1s
  Batch 39/47 (83.0%) | Loss: 0.7270 | Acc: 0.693 | Elapsed: 13.3s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.7182 | Acc: 0.697 | Elapsed: 13.6s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.7208 | Acc: 0.694 | Elapsed: 13.9s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.7245 | Acc: 0.693 | Elapsed: 14.2s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.7265 | Acc: 0.693 | Elapsed: 14.6s | ETA: 1.4s
  Batch 44/47 (93.6%) | Loss: 0.7246 | Acc: 0.695 | Elapsed: 14.9s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.7189 | Acc: 0.698 | Elapsed: 15.2s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.7172 | Acc: 0.700 | Elapsed: 15.5s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.7176 | Acc: 0.700 | Elapsed: 15.6s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 2.1507 | Acc: 0.419
Epoch 11 completed in 17.7s

--- Epoch 12/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.7458 | Acc: 0.750 | Elapsed: 1.2s | ETA: 53.5s
  Batch 2/47 (4.3%) | Loss: 0.7450 | Acc: 0.688 | Elapsed: 1.5s | ETA: 33.2s
  Batch 3/47 (6.4%) | Loss: 0.7772 | Acc: 0.677 | Elapsed: 1.8s | ETA: 26.7s
  Batch 4/47 (8.5%) | Loss: 0.7370 | Acc: 0.672 | Elapsed: 2.1s | ETA: 23.0s
  Batch 5/47 (10.6%) | Loss: 0.7332 | Acc: 0.700 | Elapsed: 2.5s | ETA: 20.7s
  Batch 6/47 (12.8%) | Loss: 0.7177 | Acc: 0.708 | Elapsed: 2.8s | ETA: 19.0s
  Batch 7/47 (14.9%) | Loss: 0.7225 | Acc: 0.728 | Elapsed: 3.1s | ETA: 17.8s
  Batch 8/47 (17.0%) | Loss: 0.6801 | Acc: 0.742 | Elapsed: 3.4s | ETA: 16.7s
  Batch 9/47 (19.1%) | Loss: 0.6532 | Acc: 0.750 | Elapsed: 3.7s | ETA: 15.8s
  Batch 10/47 (21.3%) | Loss: 0.6468 | Acc: 0.750 | Elapsed: 4.1s | ETA: 15.1s
  Batch 11/47 (23.4%) | Loss: 0.6492 | Acc: 0.750 | Elapsed: 4.4s | ETA: 14.4s
  Batch 12/47 (25.5%) | Loss: 0.6601 | Acc: 0.750 | Elapsed: 4.7s | ETA: 13.7s
  Batch 13/47 (27.7%) | Loss: 0.6523 | Acc: 0.755 | Elapsed: 5.0s | ETA: 13.2s
  Batch 14/47 (29.8%) | Loss: 0.6778 | Acc: 0.746 | Elapsed: 5.4s | ETA: 12.6s
  Batch 15/47 (31.9%) | Loss: 0.6840 | Acc: 0.738 | Elapsed: 5.7s | ETA: 12.1s
  Batch 16/47 (34.0%) | Loss: 0.6862 | Acc: 0.730 | Elapsed: 6.0s | ETA: 11.6s
  Batch 17/47 (36.2%) | Loss: 0.6753 | Acc: 0.737 | Elapsed: 6.3s | ETA: 11.2s
  Batch 18/47 (38.3%) | Loss: 0.6901 | Acc: 0.729 | Elapsed: 6.6s | ETA: 10.7s
  Batch 19/47 (40.4%) | Loss: 0.6857 | Acc: 0.729 | Elapsed: 7.0s | ETA: 10.3s
  Batch 20/47 (42.6%) | Loss: 0.6858 | Acc: 0.725 | Elapsed: 7.3s | ETA: 9.8s
  Batch 21/47 (44.7%) | Loss: 0.6953 | Acc: 0.722 | Elapsed: 7.6s | ETA: 9.4s
  Batch 22/47 (46.8%) | Loss: 0.6915 | Acc: 0.720 | Elapsed: 7.9s | ETA: 9.0s
  Batch 23/47 (48.9%) | Loss: 0.6819 | Acc: 0.724 | Elapsed: 8.2s | ETA: 8.6s
  Batch 24/47 (51.1%) | Loss: 0.6760 | Acc: 0.729 | Elapsed: 8.6s | ETA: 8.2s
  Batch 25/47 (53.2%) | Loss: 0.6742 | Acc: 0.733 | Elapsed: 8.9s | ETA: 7.8s
  Batch 26/47 (55.3%) | Loss: 0.6713 | Acc: 0.733 | Elapsed: 9.2s | ETA: 7.4s
  Batch 27/47 (57.4%) | Loss: 0.6628 | Acc: 0.737 | Elapsed: 9.5s | ETA: 7.1s
  Batch 28/47 (59.6%) | Loss: 0.6557 | Acc: 0.741 | Elapsed: 9.9s | ETA: 6.7s
  Batch 29/47 (61.7%) | Loss: 0.6561 | Acc: 0.737 | Elapsed: 10.2s | ETA: 6.3s
  Batch 30/47 (63.8%) | Loss: 0.6546 | Acc: 0.740 | Elapsed: 10.5s | ETA: 6.0s
  Batch 31/47 (66.0%) | Loss: 0.6542 | Acc: 0.739 | Elapsed: 10.8s | ETA: 5.6s
  Batch 32/47 (68.1%) | Loss: 0.6535 | Acc: 0.739 | Elapsed: 11.2s | ETA: 5.2s
  Batch 33/47 (70.2%) | Loss: 0.6502 | Acc: 0.741 | Elapsed: 11.5s | ETA: 4.9s
  Batch 34/47 (72.3%) | Loss: 0.6519 | Acc: 0.741 | Elapsed: 11.8s | ETA: 4.5s
  Batch 35/47 (74.5%) | Loss: 0.6511 | Acc: 0.742 | Elapsed: 12.2s | ETA: 4.2s
  Batch 36/47 (76.6%) | Loss: 0.6563 | Acc: 0.737 | Elapsed: 12.5s | ETA: 3.8s
  Batch 37/47 (78.7%) | Loss: 0.6618 | Acc: 0.731 | Elapsed: 12.8s | ETA: 3.5s
  Batch 38/47 (80.9%) | Loss: 0.6609 | Acc: 0.730 | Elapsed: 13.1s | ETA: 3.1s
  Batch 39/47 (83.0%) | Loss: 0.6576 | Acc: 0.730 | Elapsed: 13.4s | ETA: 2.8s
  Batch 40/47 (85.1%) | Loss: 0.6610 | Acc: 0.731 | Elapsed: 13.8s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.6608 | Acc: 0.732 | Elapsed: 14.1s | ETA: 2.1s
  Batch 42/47 (89.4%) | Loss: 0.6617 | Acc: 0.731 | Elapsed: 14.4s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.6587 | Acc: 0.729 | Elapsed: 14.7s | ETA: 1.4s
  Batch 44/47 (93.6%) | Loss: 0.6591 | Acc: 0.729 | Elapsed: 15.1s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.6587 | Acc: 0.729 | Elapsed: 15.4s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.6657 | Acc: 0.729 | Elapsed: 15.7s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.6687 | Acc: 0.728 | Elapsed: 15.8s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.8645 | Acc: 0.505
Epoch 12 completed in 18.4s

--- Epoch 13/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.3943 | Acc: 0.844 | Elapsed: 0.8s | ETA: 37.2s
  Batch 2/47 (4.3%) | Loss: 0.5290 | Acc: 0.781 | Elapsed: 1.1s | ETA: 25.2s
  Batch 3/47 (6.4%) | Loss: 0.6694 | Acc: 0.708 | Elapsed: 1.5s | ETA: 21.5s
  Batch 4/47 (8.5%) | Loss: 0.7762 | Acc: 0.664 | Elapsed: 1.8s | ETA: 19.1s
  Batch 5/47 (10.6%) | Loss: 0.7366 | Acc: 0.694 | Elapsed: 2.1s | ETA: 17.6s
  Batch 6/47 (12.8%) | Loss: 0.7081 | Acc: 0.703 | Elapsed: 2.4s | ETA: 16.6s
  Batch 7/47 (14.9%) | Loss: 0.7346 | Acc: 0.683 | Elapsed: 2.7s | ETA: 15.7s
  Batch 8/47 (17.0%) | Loss: 0.7097 | Acc: 0.699 | Elapsed: 3.1s | ETA: 14.9s
  Batch 9/47 (19.1%) | Loss: 0.6917 | Acc: 0.708 | Elapsed: 3.4s | ETA: 14.3s
  Batch 10/47 (21.3%) | Loss: 0.6851 | Acc: 0.709 | Elapsed: 3.7s | ETA: 13.7s
  Batch 11/47 (23.4%) | Loss: 0.7014 | Acc: 0.699 | Elapsed: 4.0s | ETA: 13.2s
  Batch 12/47 (25.5%) | Loss: 0.6889 | Acc: 0.701 | Elapsed: 4.3s | ETA: 12.7s
  Batch 13/47 (27.7%) | Loss: 0.6877 | Acc: 0.704 | Elapsed: 4.7s | ETA: 12.2s
  Batch 14/47 (29.8%) | Loss: 0.7126 | Acc: 0.696 | Elapsed: 5.0s | ETA: 11.8s
  Batch 15/47 (31.9%) | Loss: 0.6974 | Acc: 0.706 | Elapsed: 5.3s | ETA: 11.3s
  Batch 16/47 (34.0%) | Loss: 0.6884 | Acc: 0.711 | Elapsed: 5.6s | ETA: 10.9s
  Batch 17/47 (36.2%) | Loss: 0.6724 | Acc: 0.717 | Elapsed: 6.0s | ETA: 10.5s
  Batch 18/47 (38.3%) | Loss: 0.6711 | Acc: 0.710 | Elapsed: 6.3s | ETA: 10.1s
  Batch 19/47 (40.4%) | Loss: 0.6750 | Acc: 0.704 | Elapsed: 6.6s | ETA: 9.7s
  Batch 20/47 (42.6%) | Loss: 0.6560 | Acc: 0.716 | Elapsed: 6.9s | ETA: 9.4s
  Batch 21/47 (44.7%) | Loss: 0.6547 | Acc: 0.719 | Elapsed: 7.3s | ETA: 9.0s
  Batch 22/47 (46.8%) | Loss: 0.6579 | Acc: 0.716 | Elapsed: 7.6s | ETA: 8.6s
  Batch 23/47 (48.9%) | Loss: 0.6517 | Acc: 0.719 | Elapsed: 7.9s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 0.6548 | Acc: 0.719 | Elapsed: 8.2s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 0.6676 | Acc: 0.716 | Elapsed: 8.5s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.6661 | Acc: 0.718 | Elapsed: 8.9s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 0.6659 | Acc: 0.719 | Elapsed: 9.2s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.6608 | Acc: 0.721 | Elapsed: 9.5s | ETA: 6.4s
  Batch 29/47 (61.7%) | Loss: 0.6652 | Acc: 0.721 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.6612 | Acc: 0.723 | Elapsed: 10.1s | ETA: 5.7s
  Batch 31/47 (66.0%) | Loss: 0.6582 | Acc: 0.724 | Elapsed: 10.5s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.6596 | Acc: 0.723 | Elapsed: 10.8s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.6851 | Acc: 0.713 | Elapsed: 11.1s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.6901 | Acc: 0.706 | Elapsed: 11.4s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.6845 | Acc: 0.707 | Elapsed: 11.8s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 0.6877 | Acc: 0.707 | Elapsed: 12.1s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.6858 | Acc: 0.706 | Elapsed: 12.4s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.6889 | Acc: 0.701 | Elapsed: 12.7s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.6870 | Acc: 0.702 | Elapsed: 13.1s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.6880 | Acc: 0.702 | Elapsed: 13.4s | ETA: 2.3s
  Batch 41/47 (87.2%) | Loss: 0.6891 | Acc: 0.702 | Elapsed: 13.7s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.6832 | Acc: 0.704 | Elapsed: 14.0s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.6772 | Acc: 0.707 | Elapsed: 14.4s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.6778 | Acc: 0.708 | Elapsed: 14.7s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.6776 | Acc: 0.710 | Elapsed: 15.0s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.6750 | Acc: 0.710 | Elapsed: 15.3s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.6784 | Acc: 0.709 | Elapsed: 15.4s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.3661 | Acc: 0.474
Epoch 13 completed in 18.7s

--- Epoch 14/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.6878 | Acc: 0.750 | Elapsed: 1.1s | ETA: 52.2s
  Batch 2/47 (4.3%) | Loss: 0.6861 | Acc: 0.719 | Elapsed: 1.5s | ETA: 34.0s
  Batch 3/47 (6.4%) | Loss: 0.7013 | Acc: 0.729 | Elapsed: 1.8s | ETA: 27.0s
  Batch 4/47 (8.5%) | Loss: 0.6987 | Acc: 0.711 | Elapsed: 2.2s | ETA: 23.4s
  Batch 5/47 (10.6%) | Loss: 0.7212 | Acc: 0.706 | Elapsed: 2.5s | ETA: 21.0s
  Batch 6/47 (12.8%) | Loss: 0.7117 | Acc: 0.714 | Elapsed: 2.8s | ETA: 19.2s
  Batch 7/47 (14.9%) | Loss: 0.6809 | Acc: 0.732 | Elapsed: 3.1s | ETA: 18.0s
  Batch 8/47 (17.0%) | Loss: 0.7165 | Acc: 0.723 | Elapsed: 3.5s | ETA: 16.9s
  Batch 9/47 (19.1%) | Loss: 0.7214 | Acc: 0.719 | Elapsed: 3.8s | ETA: 16.0s
  Batch 10/47 (21.3%) | Loss: 0.7316 | Acc: 0.709 | Elapsed: 4.1s | ETA: 15.2s
  Batch 11/47 (23.4%) | Loss: 0.7383 | Acc: 0.702 | Elapsed: 4.4s | ETA: 14.5s
  Batch 12/47 (25.5%) | Loss: 0.7248 | Acc: 0.711 | Elapsed: 4.7s | ETA: 13.8s
  Batch 13/47 (27.7%) | Loss: 0.7260 | Acc: 0.712 | Elapsed: 5.1s | ETA: 13.2s
  Batch 14/47 (29.8%) | Loss: 0.7202 | Acc: 0.710 | Elapsed: 5.4s | ETA: 12.7s
  Batch 15/47 (31.9%) | Loss: 0.7085 | Acc: 0.715 | Elapsed: 5.7s | ETA: 12.2s
  Batch 16/47 (34.0%) | Loss: 0.7033 | Acc: 0.715 | Elapsed: 6.0s | ETA: 11.7s
  Batch 17/47 (36.2%) | Loss: 0.6950 | Acc: 0.717 | Elapsed: 6.4s | ETA: 11.2s
  Batch 18/47 (38.3%) | Loss: 0.6797 | Acc: 0.722 | Elapsed: 6.7s | ETA: 10.8s
  Batch 19/47 (40.4%) | Loss: 0.6730 | Acc: 0.722 | Elapsed: 7.0s | ETA: 10.4s
  Batch 20/47 (42.6%) | Loss: 0.6727 | Acc: 0.722 | Elapsed: 7.4s | ETA: 9.9s
  Batch 21/47 (44.7%) | Loss: 0.6706 | Acc: 0.722 | Elapsed: 7.7s | ETA: 9.5s
  Batch 22/47 (46.8%) | Loss: 0.6729 | Acc: 0.722 | Elapsed: 8.0s | ETA: 9.1s
  Batch 23/47 (48.9%) | Loss: 0.6814 | Acc: 0.720 | Elapsed: 8.3s | ETA: 8.7s
  Batch 24/47 (51.1%) | Loss: 0.6751 | Acc: 0.721 | Elapsed: 8.7s | ETA: 8.3s
  Batch 25/47 (53.2%) | Loss: 0.6724 | Acc: 0.723 | Elapsed: 9.0s | ETA: 7.9s
  Batch 26/47 (55.3%) | Loss: 0.6721 | Acc: 0.725 | Elapsed: 9.3s | ETA: 7.5s
  Batch 27/47 (57.4%) | Loss: 0.6606 | Acc: 0.730 | Elapsed: 9.6s | ETA: 7.1s
  Batch 28/47 (59.6%) | Loss: 0.6565 | Acc: 0.733 | Elapsed: 10.0s | ETA: 6.8s
  Batch 29/47 (61.7%) | Loss: 0.6543 | Acc: 0.733 | Elapsed: 10.3s | ETA: 6.4s
  Batch 30/47 (63.8%) | Loss: 0.6459 | Acc: 0.735 | Elapsed: 10.6s | ETA: 6.0s
  Batch 31/47 (66.0%) | Loss: 0.6429 | Acc: 0.736 | Elapsed: 10.9s | ETA: 5.6s
  Batch 32/47 (68.1%) | Loss: 0.6433 | Acc: 0.734 | Elapsed: 11.2s | ETA: 5.3s
  Batch 33/47 (70.2%) | Loss: 0.6397 | Acc: 0.740 | Elapsed: 11.6s | ETA: 4.9s
  Batch 34/47 (72.3%) | Loss: 0.6546 | Acc: 0.732 | Elapsed: 11.9s | ETA: 4.5s
  Batch 35/47 (74.5%) | Loss: 0.6533 | Acc: 0.730 | Elapsed: 12.2s | ETA: 4.2s
  Batch 36/47 (76.6%) | Loss: 0.6493 | Acc: 0.731 | Elapsed: 12.5s | ETA: 3.8s
  Batch 37/47 (78.7%) | Loss: 0.6501 | Acc: 0.733 | Elapsed: 12.8s | ETA: 3.5s
  Batch 38/47 (80.9%) | Loss: 0.6499 | Acc: 0.734 | Elapsed: 13.2s | ETA: 3.1s
  Batch 39/47 (83.0%) | Loss: 0.6463 | Acc: 0.732 | Elapsed: 13.5s | ETA: 2.8s
  Batch 40/47 (85.1%) | Loss: 0.6425 | Acc: 0.736 | Elapsed: 13.8s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.6441 | Acc: 0.737 | Elapsed: 14.1s | ETA: 2.1s
  Batch 42/47 (89.4%) | Loss: 0.6428 | Acc: 0.737 | Elapsed: 14.5s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.6503 | Acc: 0.734 | Elapsed: 14.8s | ETA: 1.4s
  Batch 44/47 (93.6%) | Loss: 0.6545 | Acc: 0.732 | Elapsed: 15.1s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.6566 | Acc: 0.731 | Elapsed: 15.4s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.6573 | Acc: 0.730 | Elapsed: 15.8s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.6582 | Acc: 0.730 | Elapsed: 15.8s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.3382 | Acc: 0.544
Epoch 14 completed in 18.5s

--- Epoch 15/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.6698 | Acc: 0.656 | Elapsed: 0.8s | ETA: 37.6s
  Batch 2/47 (4.3%) | Loss: 0.6093 | Acc: 0.734 | Elapsed: 1.1s | ETA: 25.5s
  Batch 3/47 (6.4%) | Loss: 0.6274 | Acc: 0.729 | Elapsed: 1.5s | ETA: 21.7s
  Batch 4/47 (8.5%) | Loss: 0.6503 | Acc: 0.719 | Elapsed: 1.8s | ETA: 19.3s
  Batch 5/47 (10.6%) | Loss: 0.6614 | Acc: 0.719 | Elapsed: 2.1s | ETA: 17.7s
  Batch 6/47 (12.8%) | Loss: 0.6288 | Acc: 0.740 | Elapsed: 2.4s | ETA: 16.6s
  Batch 7/47 (14.9%) | Loss: 0.6450 | Acc: 0.728 | Elapsed: 2.8s | ETA: 15.7s
  Batch 8/47 (17.0%) | Loss: 0.6670 | Acc: 0.711 | Elapsed: 3.1s | ETA: 15.0s
  Batch 9/47 (19.1%) | Loss: 0.6769 | Acc: 0.705 | Elapsed: 3.4s | ETA: 14.3s
  Batch 10/47 (21.3%) | Loss: 0.6857 | Acc: 0.697 | Elapsed: 3.7s | ETA: 13.8s
  Batch 11/47 (23.4%) | Loss: 0.6806 | Acc: 0.702 | Elapsed: 4.0s | ETA: 13.2s
  Batch 12/47 (25.5%) | Loss: 0.6700 | Acc: 0.708 | Elapsed: 4.4s | ETA: 12.7s
  Batch 13/47 (27.7%) | Loss: 0.6597 | Acc: 0.712 | Elapsed: 4.7s | ETA: 12.2s
  Batch 14/47 (29.8%) | Loss: 0.6390 | Acc: 0.723 | Elapsed: 5.0s | ETA: 11.8s
  Batch 15/47 (31.9%) | Loss: 0.6383 | Acc: 0.717 | Elapsed: 5.3s | ETA: 11.3s
  Batch 16/47 (34.0%) | Loss: 0.6409 | Acc: 0.713 | Elapsed: 5.6s | ETA: 10.9s
  Batch 17/47 (36.2%) | Loss: 0.6372 | Acc: 0.715 | Elapsed: 6.0s | ETA: 10.5s
  Batch 18/47 (38.3%) | Loss: 0.6478 | Acc: 0.708 | Elapsed: 6.3s | ETA: 10.1s
  Batch 19/47 (40.4%) | Loss: 0.6419 | Acc: 0.711 | Elapsed: 6.6s | ETA: 9.7s
  Batch 20/47 (42.6%) | Loss: 0.6418 | Acc: 0.719 | Elapsed: 6.9s | ETA: 9.3s
  Batch 21/47 (44.7%) | Loss: 0.6479 | Acc: 0.716 | Elapsed: 7.2s | ETA: 9.0s
  Batch 22/47 (46.8%) | Loss: 0.6432 | Acc: 0.717 | Elapsed: 7.6s | ETA: 8.6s
  Batch 23/47 (48.9%) | Loss: 0.6338 | Acc: 0.723 | Elapsed: 7.9s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 0.6477 | Acc: 0.721 | Elapsed: 8.2s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 0.6444 | Acc: 0.721 | Elapsed: 8.5s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.6387 | Acc: 0.721 | Elapsed: 8.9s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 0.6405 | Acc: 0.721 | Elapsed: 9.2s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.6381 | Acc: 0.722 | Elapsed: 9.5s | ETA: 6.5s
  Batch 29/47 (61.7%) | Loss: 0.6362 | Acc: 0.722 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.6372 | Acc: 0.721 | Elapsed: 10.2s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.6324 | Acc: 0.724 | Elapsed: 10.5s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.6295 | Acc: 0.722 | Elapsed: 10.8s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.6362 | Acc: 0.720 | Elapsed: 11.2s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.6294 | Acc: 0.723 | Elapsed: 11.5s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.6300 | Acc: 0.722 | Elapsed: 11.8s | ETA: 4.1s
  Batch 36/47 (76.6%) | Loss: 0.6293 | Acc: 0.723 | Elapsed: 12.2s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.6312 | Acc: 0.725 | Elapsed: 12.5s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.6403 | Acc: 0.722 | Elapsed: 12.8s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.6360 | Acc: 0.724 | Elapsed: 13.1s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.6478 | Acc: 0.722 | Elapsed: 13.5s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.6549 | Acc: 0.720 | Elapsed: 13.8s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.6577 | Acc: 0.719 | Elapsed: 14.1s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.6591 | Acc: 0.720 | Elapsed: 14.4s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.6582 | Acc: 0.722 | Elapsed: 14.8s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.6637 | Acc: 0.719 | Elapsed: 15.1s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.6632 | Acc: 0.719 | Elapsed: 15.4s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.6632 | Acc: 0.719 | Elapsed: 15.5s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.3365 | Acc: 0.552
Epoch 15 completed in 20.1s

--- Epoch 16/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.4237 | Acc: 0.844 | Elapsed: 0.8s | ETA: 37.1s
  Batch 2/47 (4.3%) | Loss: 0.4924 | Acc: 0.797 | Elapsed: 1.1s | ETA: 25.1s
  Batch 3/47 (6.4%) | Loss: 0.4791 | Acc: 0.781 | Elapsed: 1.5s | ETA: 21.4s
  Batch 4/47 (8.5%) | Loss: 0.5084 | Acc: 0.766 | Elapsed: 1.8s | ETA: 19.1s
  Batch 5/47 (10.6%) | Loss: 0.5033 | Acc: 0.756 | Elapsed: 2.1s | ETA: 17.7s
  Batch 6/47 (12.8%) | Loss: 0.5365 | Acc: 0.745 | Elapsed: 2.4s | ETA: 16.7s
  Batch 7/47 (14.9%) | Loss: 0.5746 | Acc: 0.719 | Elapsed: 2.8s | ETA: 15.8s
  Batch 8/47 (17.0%) | Loss: 0.5536 | Acc: 0.742 | Elapsed: 3.1s | ETA: 15.0s
  Batch 9/47 (19.1%) | Loss: 0.5824 | Acc: 0.736 | Elapsed: 3.4s | ETA: 14.4s
  Batch 10/47 (21.3%) | Loss: 0.5969 | Acc: 0.725 | Elapsed: 3.7s | ETA: 13.8s
  Batch 11/47 (23.4%) | Loss: 0.5891 | Acc: 0.733 | Elapsed: 4.1s | ETA: 13.3s
  Batch 12/47 (25.5%) | Loss: 0.5897 | Acc: 0.740 | Elapsed: 4.4s | ETA: 12.8s
  Batch 13/47 (27.7%) | Loss: 0.6099 | Acc: 0.731 | Elapsed: 4.7s | ETA: 12.3s
  Batch 14/47 (29.8%) | Loss: 0.5845 | Acc: 0.746 | Elapsed: 5.0s | ETA: 11.8s
  Batch 15/47 (31.9%) | Loss: 0.5592 | Acc: 0.758 | Elapsed: 5.3s | ETA: 11.4s
  Batch 16/47 (34.0%) | Loss: 0.5784 | Acc: 0.756 | Elapsed: 5.7s | ETA: 11.0s
  Batch 17/47 (36.2%) | Loss: 0.5769 | Acc: 0.756 | Elapsed: 6.0s | ETA: 10.6s
  Batch 18/47 (38.3%) | Loss: 0.5689 | Acc: 0.755 | Elapsed: 6.3s | ETA: 10.2s
  Batch 19/47 (40.4%) | Loss: 0.5699 | Acc: 0.757 | Elapsed: 6.6s | ETA: 9.8s
  Batch 20/47 (42.6%) | Loss: 0.5977 | Acc: 0.753 | Elapsed: 6.9s | ETA: 9.4s
  Batch 21/47 (44.7%) | Loss: 0.5943 | Acc: 0.760 | Elapsed: 7.3s | ETA: 9.0s
  Batch 22/47 (46.8%) | Loss: 0.6006 | Acc: 0.757 | Elapsed: 7.6s | ETA: 8.6s
  Batch 23/47 (48.9%) | Loss: 0.5948 | Acc: 0.760 | Elapsed: 7.9s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 0.6112 | Acc: 0.753 | Elapsed: 8.2s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 0.6161 | Acc: 0.751 | Elapsed: 8.5s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.6162 | Acc: 0.755 | Elapsed: 8.9s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 0.6094 | Acc: 0.756 | Elapsed: 9.2s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.6170 | Acc: 0.753 | Elapsed: 9.5s | ETA: 6.4s
  Batch 29/47 (61.7%) | Loss: 0.6174 | Acc: 0.751 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.6266 | Acc: 0.748 | Elapsed: 10.1s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.6227 | Acc: 0.750 | Elapsed: 10.5s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.6168 | Acc: 0.752 | Elapsed: 10.8s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.6165 | Acc: 0.751 | Elapsed: 11.1s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.6085 | Acc: 0.756 | Elapsed: 11.5s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.6101 | Acc: 0.756 | Elapsed: 11.8s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 0.6091 | Acc: 0.757 | Elapsed: 12.1s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.6073 | Acc: 0.758 | Elapsed: 12.4s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.6000 | Acc: 0.762 | Elapsed: 12.8s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.6072 | Acc: 0.757 | Elapsed: 13.1s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.6040 | Acc: 0.756 | Elapsed: 13.4s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.6058 | Acc: 0.756 | Elapsed: 13.8s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.6057 | Acc: 0.755 | Elapsed: 14.1s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.6071 | Acc: 0.755 | Elapsed: 14.4s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.6118 | Acc: 0.753 | Elapsed: 14.7s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.6106 | Acc: 0.751 | Elapsed: 15.1s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.6109 | Acc: 0.749 | Elapsed: 15.4s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.6105 | Acc: 0.750 | Elapsed: 15.5s | ETA: 0.0s
Val -> Loss: 1.0916 | Acc: 0.600
Epoch 16 completed in 21.2s

--- Epoch 17/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.3576 | Acc: 0.781 | Elapsed: 0.8s | ETA: 38.5s
  Batch 2/47 (4.3%) | Loss: 0.4733 | Acc: 0.734 | Elapsed: 1.1s | ETA: 25.8s
  Batch 3/47 (6.4%) | Loss: 0.5410 | Acc: 0.729 | Elapsed: 1.5s | ETA: 21.8s
  Batch 4/47 (8.5%) | Loss: 0.5025 | Acc: 0.750 | Elapsed: 1.8s | ETA: 19.4s
  Batch 5/47 (10.6%) | Loss: 0.5739 | Acc: 0.731 | Elapsed: 2.1s | ETA: 17.8s
  Batch 6/47 (12.8%) | Loss: 0.5646 | Acc: 0.745 | Elapsed: 2.4s | ETA: 16.7s
  Batch 7/47 (14.9%) | Loss: 0.5862 | Acc: 0.741 | Elapsed: 2.8s | ETA: 15.8s
  Batch 8/47 (17.0%) | Loss: 0.5908 | Acc: 0.727 | Elapsed: 3.1s | ETA: 15.0s
  Batch 9/47 (19.1%) | Loss: 0.5845 | Acc: 0.736 | Elapsed: 3.4s | ETA: 14.3s
  Batch 10/47 (21.3%) | Loss: 0.6234 | Acc: 0.731 | Elapsed: 3.7s | ETA: 13.8s
  Batch 11/47 (23.4%) | Loss: 0.6299 | Acc: 0.733 | Elapsed: 4.0s | ETA: 13.2s
  Batch 12/47 (25.5%) | Loss: 0.6295 | Acc: 0.734 | Elapsed: 4.4s | ETA: 12.7s
  Batch 13/47 (27.7%) | Loss: 0.6185 | Acc: 0.738 | Elapsed: 4.7s | ETA: 12.2s
  Batch 14/47 (29.8%) | Loss: 0.6147 | Acc: 0.732 | Elapsed: 5.0s | ETA: 11.8s
  Batch 15/47 (31.9%) | Loss: 0.6042 | Acc: 0.735 | Elapsed: 5.3s | ETA: 11.3s
  Batch 16/47 (34.0%) | Loss: 0.6016 | Acc: 0.732 | Elapsed: 5.6s | ETA: 10.9s
  Batch 17/47 (36.2%) | Loss: 0.6107 | Acc: 0.730 | Elapsed: 6.0s | ETA: 10.5s
  Batch 18/47 (38.3%) | Loss: 0.6226 | Acc: 0.731 | Elapsed: 6.3s | ETA: 10.1s
  Batch 19/47 (40.4%) | Loss: 0.6107 | Acc: 0.738 | Elapsed: 6.6s | ETA: 9.7s
  Batch 20/47 (42.6%) | Loss: 0.6190 | Acc: 0.738 | Elapsed: 6.9s | ETA: 9.4s
  Batch 21/47 (44.7%) | Loss: 0.6200 | Acc: 0.740 | Elapsed: 7.2s | ETA: 9.0s
  Batch 22/47 (46.8%) | Loss: 0.6131 | Acc: 0.744 | Elapsed: 7.6s | ETA: 8.6s
  Batch 23/47 (48.9%) | Loss: 0.6191 | Acc: 0.740 | Elapsed: 7.9s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 0.6286 | Acc: 0.736 | Elapsed: 8.2s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 0.6291 | Acc: 0.736 | Elapsed: 8.5s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.6262 | Acc: 0.738 | Elapsed: 8.9s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 0.6217 | Acc: 0.741 | Elapsed: 9.2s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.6180 | Acc: 0.743 | Elapsed: 9.5s | ETA: 6.5s
  Batch 29/47 (61.7%) | Loss: 0.6134 | Acc: 0.745 | Elapsed: 9.9s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.6079 | Acc: 0.750 | Elapsed: 10.2s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.6097 | Acc: 0.748 | Elapsed: 10.5s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.6111 | Acc: 0.747 | Elapsed: 10.8s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.6204 | Acc: 0.741 | Elapsed: 11.2s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.6148 | Acc: 0.745 | Elapsed: 11.5s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.6175 | Acc: 0.744 | Elapsed: 11.8s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 0.6162 | Acc: 0.744 | Elapsed: 12.1s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.6198 | Acc: 0.742 | Elapsed: 12.5s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.6219 | Acc: 0.742 | Elapsed: 12.8s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.6184 | Acc: 0.743 | Elapsed: 13.1s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.6170 | Acc: 0.746 | Elapsed: 13.4s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.6240 | Acc: 0.742 | Elapsed: 13.8s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.6213 | Acc: 0.743 | Elapsed: 14.1s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.6189 | Acc: 0.745 | Elapsed: 14.4s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.6179 | Acc: 0.745 | Elapsed: 14.8s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.6126 | Acc: 0.748 | Elapsed: 15.1s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.6108 | Acc: 0.749 | Elapsed: 15.4s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.6143 | Acc: 0.748 | Elapsed: 15.5s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.4880 | Acc: 0.552
Epoch 17 completed in 19.0s

--- Epoch 18/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.5262 | Acc: 0.781 | Elapsed: 0.9s | ETA: 40.4s
  Batch 2/47 (4.3%) | Loss: 0.6586 | Acc: 0.719 | Elapsed: 1.2s | ETA: 26.8s
  Batch 3/47 (6.4%) | Loss: 0.5878 | Acc: 0.760 | Elapsed: 1.5s | ETA: 22.4s
  Batch 4/47 (8.5%) | Loss: 0.5735 | Acc: 0.758 | Elapsed: 1.9s | ETA: 20.0s
  Batch 5/47 (10.6%) | Loss: 0.5250 | Acc: 0.787 | Elapsed: 2.2s | ETA: 18.4s
  Batch 6/47 (12.8%) | Loss: 0.5500 | Acc: 0.792 | Elapsed: 2.5s | ETA: 17.2s
  Batch 7/47 (14.9%) | Loss: 0.6053 | Acc: 0.768 | Elapsed: 2.9s | ETA: 16.3s
  Batch 8/47 (17.0%) | Loss: 0.6375 | Acc: 0.762 | Elapsed: 3.2s | ETA: 15.5s
  Batch 9/47 (19.1%) | Loss: 0.6276 | Acc: 0.764 | Elapsed: 3.5s | ETA: 14.8s
  Batch 10/47 (21.3%) | Loss: 0.6024 | Acc: 0.772 | Elapsed: 3.8s | ETA: 14.2s
  Batch 11/47 (23.4%) | Loss: 0.6081 | Acc: 0.764 | Elapsed: 4.2s | ETA: 13.6s
  Batch 12/47 (25.5%) | Loss: 0.6086 | Acc: 0.768 | Elapsed: 4.5s | ETA: 13.1s
  Batch 13/47 (27.7%) | Loss: 0.6145 | Acc: 0.764 | Elapsed: 4.8s | ETA: 12.6s
  Batch 14/47 (29.8%) | Loss: 0.6125 | Acc: 0.768 | Elapsed: 5.2s | ETA: 12.2s
  Batch 15/47 (31.9%) | Loss: 0.6103 | Acc: 0.765 | Elapsed: 5.5s | ETA: 11.7s
  Batch 16/47 (34.0%) | Loss: 0.6190 | Acc: 0.764 | Elapsed: 5.8s | ETA: 11.3s
  Batch 17/47 (36.2%) | Loss: 0.6066 | Acc: 0.768 | Elapsed: 6.2s | ETA: 10.9s
  Batch 18/47 (38.3%) | Loss: 0.6146 | Acc: 0.766 | Elapsed: 6.5s | ETA: 10.4s
  Batch 19/47 (40.4%) | Loss: 0.6115 | Acc: 0.768 | Elapsed: 6.8s | ETA: 10.0s
  Batch 20/47 (42.6%) | Loss: 0.6043 | Acc: 0.770 | Elapsed: 7.1s | ETA: 9.6s
  Batch 21/47 (44.7%) | Loss: 0.5963 | Acc: 0.774 | Elapsed: 7.4s | ETA: 9.2s
  Batch 22/47 (46.8%) | Loss: 0.6076 | Acc: 0.768 | Elapsed: 7.8s | ETA: 8.8s
  Batch 23/47 (48.9%) | Loss: 0.6007 | Acc: 0.770 | Elapsed: 8.1s | ETA: 8.5s
  Batch 24/47 (51.1%) | Loss: 0.6256 | Acc: 0.757 | Elapsed: 8.4s | ETA: 8.1s
  Batch 25/47 (53.2%) | Loss: 0.6211 | Acc: 0.757 | Elapsed: 8.8s | ETA: 7.7s
  Batch 26/47 (55.3%) | Loss: 0.6150 | Acc: 0.761 | Elapsed: 9.1s | ETA: 7.3s
  Batch 27/47 (57.4%) | Loss: 0.6110 | Acc: 0.764 | Elapsed: 9.4s | ETA: 7.0s
  Batch 28/47 (59.6%) | Loss: 0.6102 | Acc: 0.763 | Elapsed: 9.7s | ETA: 6.6s
  Batch 29/47 (61.7%) | Loss: 0.6129 | Acc: 0.760 | Elapsed: 10.1s | ETA: 6.3s
  Batch 30/47 (63.8%) | Loss: 0.6136 | Acc: 0.761 | Elapsed: 10.4s | ETA: 5.9s
  Batch 31/47 (66.0%) | Loss: 0.6074 | Acc: 0.764 | Elapsed: 10.7s | ETA: 5.5s
  Batch 32/47 (68.1%) | Loss: 0.5996 | Acc: 0.769 | Elapsed: 11.0s | ETA: 5.2s
  Batch 33/47 (70.2%) | Loss: 0.5950 | Acc: 0.771 | Elapsed: 11.4s | ETA: 4.8s
  Batch 34/47 (72.3%) | Loss: 0.5967 | Acc: 0.767 | Elapsed: 11.7s | ETA: 4.5s
  Batch 35/47 (74.5%) | Loss: 0.5972 | Acc: 0.765 | Elapsed: 12.0s | ETA: 4.1s
  Batch 36/47 (76.6%) | Loss: 0.5997 | Acc: 0.761 | Elapsed: 12.3s | ETA: 3.8s
  Batch 37/47 (78.7%) | Loss: 0.5990 | Acc: 0.761 | Elapsed: 12.7s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.5975 | Acc: 0.762 | Elapsed: 13.0s | ETA: 3.1s
  Batch 39/47 (83.0%) | Loss: 0.6046 | Acc: 0.758 | Elapsed: 13.3s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.6001 | Acc: 0.761 | Elapsed: 13.6s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.5996 | Acc: 0.761 | Elapsed: 13.9s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.5978 | Acc: 0.762 | Elapsed: 14.3s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.5950 | Acc: 0.762 | Elapsed: 14.6s | ETA: 1.4s
  Batch 44/47 (93.6%) | Loss: 0.5955 | Acc: 0.763 | Elapsed: 14.9s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.5955 | Acc: 0.764 | Elapsed: 15.2s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.5977 | Acc: 0.764 | Elapsed: 15.6s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.5983 | Acc: 0.764 | Elapsed: 15.6s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.4419 | Acc: 0.509
Epoch 18 completed in 17.8s

--- Epoch 19/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.5385 | Acc: 0.750 | Elapsed: 1.0s | ETA: 47.9s
  Batch 2/47 (4.3%) | Loss: 0.5812 | Acc: 0.719 | Elapsed: 1.4s | ETA: 30.5s
  Batch 3/47 (6.4%) | Loss: 0.6571 | Acc: 0.708 | Elapsed: 1.7s | ETA: 24.9s
  Batch 4/47 (8.5%) | Loss: 0.6526 | Acc: 0.719 | Elapsed: 2.0s | ETA: 21.7s
  Batch 5/47 (10.6%) | Loss: 0.6066 | Acc: 0.744 | Elapsed: 2.3s | ETA: 19.6s
  Batch 6/47 (12.8%) | Loss: 0.6076 | Acc: 0.729 | Elapsed: 2.7s | ETA: 18.2s
  Batch 7/47 (14.9%) | Loss: 0.6591 | Acc: 0.701 | Elapsed: 3.0s | ETA: 17.0s
  Batch 8/47 (17.0%) | Loss: 0.6870 | Acc: 0.688 | Elapsed: 3.3s | ETA: 16.1s
  Batch 9/47 (19.1%) | Loss: 0.7321 | Acc: 0.667 | Elapsed: 3.6s | ETA: 15.3s
  Batch 10/47 (21.3%) | Loss: 0.7065 | Acc: 0.681 | Elapsed: 3.9s | ETA: 14.6s
  Batch 11/47 (23.4%) | Loss: 0.6942 | Acc: 0.685 | Elapsed: 4.3s | ETA: 14.0s
  Batch 12/47 (25.5%) | Loss: 0.6834 | Acc: 0.688 | Elapsed: 4.6s | ETA: 13.4s
  Batch 13/47 (27.7%) | Loss: 0.6646 | Acc: 0.700 | Elapsed: 4.9s | ETA: 12.9s
  Batch 14/47 (29.8%) | Loss: 0.6602 | Acc: 0.705 | Elapsed: 5.2s | ETA: 12.3s
  Batch 15/47 (31.9%) | Loss: 0.6557 | Acc: 0.710 | Elapsed: 5.6s | ETA: 11.8s
  Batch 16/47 (34.0%) | Loss: 0.6510 | Acc: 0.713 | Elapsed: 5.9s | ETA: 11.4s
  Batch 17/47 (36.2%) | Loss: 0.6617 | Acc: 0.711 | Elapsed: 6.2s | ETA: 10.9s
  Batch 18/47 (38.3%) | Loss: 0.6530 | Acc: 0.720 | Elapsed: 6.5s | ETA: 10.5s
  Batch 19/47 (40.4%) | Loss: 0.6547 | Acc: 0.720 | Elapsed: 6.8s | ETA: 10.1s
  Batch 20/47 (42.6%) | Loss: 0.6456 | Acc: 0.727 | Elapsed: 7.2s | ETA: 9.7s
  Batch 21/47 (44.7%) | Loss: 0.6366 | Acc: 0.731 | Elapsed: 7.5s | ETA: 9.3s
  Batch 22/47 (46.8%) | Loss: 0.6305 | Acc: 0.736 | Elapsed: 7.8s | ETA: 8.9s
  Batch 23/47 (48.9%) | Loss: 0.6330 | Acc: 0.736 | Elapsed: 8.1s | ETA: 8.5s
  Batch 24/47 (51.1%) | Loss: 0.6392 | Acc: 0.740 | Elapsed: 8.5s | ETA: 8.1s
  Batch 25/47 (53.2%) | Loss: 0.6346 | Acc: 0.740 | Elapsed: 8.8s | ETA: 7.7s
  Batch 26/47 (55.3%) | Loss: 0.6238 | Acc: 0.745 | Elapsed: 9.1s | ETA: 7.3s
  Batch 27/47 (57.4%) | Loss: 0.6208 | Acc: 0.745 | Elapsed: 9.4s | ETA: 7.0s
  Batch 28/47 (59.6%) | Loss: 0.6238 | Acc: 0.746 | Elapsed: 9.7s | ETA: 6.6s
  Batch 29/47 (61.7%) | Loss: 0.6232 | Acc: 0.747 | Elapsed: 10.1s | ETA: 6.2s
  Batch 30/47 (63.8%) | Loss: 0.6255 | Acc: 0.746 | Elapsed: 10.4s | ETA: 5.9s
  Batch 31/47 (66.0%) | Loss: 0.6269 | Acc: 0.746 | Elapsed: 10.7s | ETA: 5.5s
  Batch 32/47 (68.1%) | Loss: 0.6236 | Acc: 0.747 | Elapsed: 11.0s | ETA: 5.2s
  Batch 33/47 (70.2%) | Loss: 0.6252 | Acc: 0.747 | Elapsed: 11.3s | ETA: 4.8s
  Batch 34/47 (72.3%) | Loss: 0.6242 | Acc: 0.747 | Elapsed: 11.7s | ETA: 4.5s
  Batch 35/47 (74.5%) | Loss: 0.6208 | Acc: 0.748 | Elapsed: 12.0s | ETA: 4.1s
  Batch 36/47 (76.6%) | Loss: 0.6228 | Acc: 0.747 | Elapsed: 12.3s | ETA: 3.8s
  Batch 37/47 (78.7%) | Loss: 0.6234 | Acc: 0.747 | Elapsed: 12.6s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.6219 | Acc: 0.749 | Elapsed: 13.0s | ETA: 3.1s
  Batch 39/47 (83.0%) | Loss: 0.6225 | Acc: 0.748 | Elapsed: 13.3s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.6228 | Acc: 0.746 | Elapsed: 13.6s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.6172 | Acc: 0.749 | Elapsed: 14.0s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.6142 | Acc: 0.751 | Elapsed: 14.3s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.6193 | Acc: 0.752 | Elapsed: 14.6s | ETA: 1.4s
  Batch 44/47 (93.6%) | Loss: 0.6210 | Acc: 0.751 | Elapsed: 14.9s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.6188 | Acc: 0.751 | Elapsed: 15.2s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.6139 | Acc: 0.753 | Elapsed: 15.6s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.6153 | Acc: 0.753 | Elapsed: 15.6s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.4364 | Acc: 0.540
Epoch 19 completed in 17.3s

--- Epoch 20/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.3860 | Acc: 0.812 | Elapsed: 0.8s | ETA: 36.3s
  Batch 2/47 (4.3%) | Loss: 0.5839 | Acc: 0.766 | Elapsed: 1.1s | ETA: 24.8s
  Batch 3/47 (6.4%) | Loss: 0.5358 | Acc: 0.802 | Elapsed: 1.4s | ETA: 21.1s
  Batch 4/47 (8.5%) | Loss: 0.5690 | Acc: 0.781 | Elapsed: 1.8s | ETA: 18.9s
  Batch 5/47 (10.6%) | Loss: 0.6733 | Acc: 0.769 | Elapsed: 2.1s | ETA: 17.4s
  Batch 6/47 (12.8%) | Loss: 0.6498 | Acc: 0.755 | Elapsed: 2.4s | ETA: 16.4s
  Batch 7/47 (14.9%) | Loss: 0.6790 | Acc: 0.746 | Elapsed: 2.7s | ETA: 15.6s
  Batch 8/47 (17.0%) | Loss: 0.6858 | Acc: 0.734 | Elapsed: 3.0s | ETA: 14.8s
  Batch 9/47 (19.1%) | Loss: 0.6620 | Acc: 0.747 | Elapsed: 3.4s | ETA: 14.2s
  Batch 10/47 (21.3%) | Loss: 0.6403 | Acc: 0.750 | Elapsed: 3.7s | ETA: 13.7s
  Batch 11/47 (23.4%) | Loss: 0.6538 | Acc: 0.747 | Elapsed: 4.0s | ETA: 13.1s
  Batch 12/47 (25.5%) | Loss: 0.6444 | Acc: 0.747 | Elapsed: 4.3s | ETA: 12.6s
  Batch 13/47 (27.7%) | Loss: 0.6384 | Acc: 0.745 | Elapsed: 4.7s | ETA: 12.2s
  Batch 14/47 (29.8%) | Loss: 0.6453 | Acc: 0.730 | Elapsed: 5.0s | ETA: 11.7s
  Batch 15/47 (31.9%) | Loss: 0.6275 | Acc: 0.735 | Elapsed: 5.3s | ETA: 11.3s
  Batch 16/47 (34.0%) | Loss: 0.6190 | Acc: 0.740 | Elapsed: 5.6s | ETA: 10.9s
  Batch 17/47 (36.2%) | Loss: 0.6075 | Acc: 0.746 | Elapsed: 5.9s | ETA: 10.5s
  Batch 18/47 (38.3%) | Loss: 0.5973 | Acc: 0.750 | Elapsed: 6.3s | ETA: 10.1s
  Batch 19/47 (40.4%) | Loss: 0.6031 | Acc: 0.745 | Elapsed: 6.6s | ETA: 9.7s
  Batch 20/47 (42.6%) | Loss: 0.6049 | Acc: 0.742 | Elapsed: 6.9s | ETA: 9.3s
  Batch 21/47 (44.7%) | Loss: 0.6132 | Acc: 0.741 | Elapsed: 7.2s | ETA: 8.9s
  Batch 22/47 (46.8%) | Loss: 0.6216 | Acc: 0.743 | Elapsed: 7.5s | ETA: 8.6s
  Batch 23/47 (48.9%) | Loss: 0.6107 | Acc: 0.747 | Elapsed: 7.9s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 0.6030 | Acc: 0.753 | Elapsed: 8.2s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 0.6015 | Acc: 0.751 | Elapsed: 8.5s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.5958 | Acc: 0.754 | Elapsed: 8.8s | ETA: 7.1s
  Batch 27/47 (57.4%) | Loss: 0.5930 | Acc: 0.753 | Elapsed: 9.2s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.5894 | Acc: 0.756 | Elapsed: 9.5s | ETA: 6.4s
  Batch 29/47 (61.7%) | Loss: 0.5866 | Acc: 0.758 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.5921 | Acc: 0.755 | Elapsed: 10.1s | ETA: 5.7s
  Batch 31/47 (66.0%) | Loss: 0.5878 | Acc: 0.759 | Elapsed: 10.5s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.5911 | Acc: 0.758 | Elapsed: 10.8s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.5931 | Acc: 0.758 | Elapsed: 11.1s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.5938 | Acc: 0.758 | Elapsed: 11.4s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.5903 | Acc: 0.760 | Elapsed: 11.7s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 0.5908 | Acc: 0.760 | Elapsed: 12.1s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.5961 | Acc: 0.756 | Elapsed: 12.4s | ETA: 3.3s
  Batch 38/47 (80.9%) | Loss: 0.5910 | Acc: 0.756 | Elapsed: 12.7s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.5910 | Acc: 0.755 | Elapsed: 13.0s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.5983 | Acc: 0.752 | Elapsed: 13.3s | ETA: 2.3s
  Batch 41/47 (87.2%) | Loss: 0.5969 | Acc: 0.752 | Elapsed: 13.7s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.5908 | Acc: 0.755 | Elapsed: 14.0s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.5866 | Acc: 0.757 | Elapsed: 14.3s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.5908 | Acc: 0.756 | Elapsed: 14.6s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.5890 | Acc: 0.756 | Elapsed: 15.0s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.5903 | Acc: 0.753 | Elapsed: 15.3s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.5896 | Acc: 0.754 | Elapsed: 15.3s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.4131 | Acc: 0.530
Epoch 20 completed in 17.0s

--- Epoch 21/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.6671 | Acc: 0.719 | Elapsed: 0.8s | ETA: 36.8s
  Batch 2/47 (4.3%) | Loss: 0.7370 | Acc: 0.688 | Elapsed: 1.1s | ETA: 25.2s
  Batch 3/47 (6.4%) | Loss: 0.7104 | Acc: 0.688 | Elapsed: 1.5s | ETA: 21.6s
  Batch 4/47 (8.5%) | Loss: 0.7096 | Acc: 0.672 | Elapsed: 1.8s | ETA: 19.2s
  Batch 5/47 (10.6%) | Loss: 0.6560 | Acc: 0.700 | Elapsed: 2.1s | ETA: 17.7s
  Batch 6/47 (12.8%) | Loss: 0.5936 | Acc: 0.729 | Elapsed: 2.4s | ETA: 16.6s
  Batch 7/47 (14.9%) | Loss: 0.5661 | Acc: 0.746 | Elapsed: 2.8s | ETA: 15.7s
  Batch 8/47 (17.0%) | Loss: 0.5526 | Acc: 0.750 | Elapsed: 3.1s | ETA: 14.9s
  Batch 9/47 (19.1%) | Loss: 0.5292 | Acc: 0.764 | Elapsed: 3.4s | ETA: 14.3s
  Batch 10/47 (21.3%) | Loss: 0.5365 | Acc: 0.759 | Elapsed: 3.7s | ETA: 13.8s
  Batch 11/47 (23.4%) | Loss: 0.5421 | Acc: 0.753 | Elapsed: 4.0s | ETA: 13.2s
  Batch 12/47 (25.5%) | Loss: 0.5591 | Acc: 0.750 | Elapsed: 4.4s | ETA: 12.7s
  Batch 13/47 (27.7%) | Loss: 0.5644 | Acc: 0.752 | Elapsed: 4.7s | ETA: 12.3s
  Batch 14/47 (29.8%) | Loss: 0.5675 | Acc: 0.757 | Elapsed: 5.0s | ETA: 11.8s
  Batch 15/47 (31.9%) | Loss: 0.5550 | Acc: 0.760 | Elapsed: 5.3s | ETA: 11.4s
  Batch 16/47 (34.0%) | Loss: 0.5455 | Acc: 0.764 | Elapsed: 5.7s | ETA: 10.9s
  Batch 17/47 (36.2%) | Loss: 0.5429 | Acc: 0.767 | Elapsed: 6.0s | ETA: 10.5s
  Batch 18/47 (38.3%) | Loss: 0.5398 | Acc: 0.766 | Elapsed: 6.3s | ETA: 10.1s
  Batch 19/47 (40.4%) | Loss: 0.5371 | Acc: 0.768 | Elapsed: 6.6s | ETA: 9.8s
  Batch 20/47 (42.6%) | Loss: 0.5344 | Acc: 0.773 | Elapsed: 6.9s | ETA: 9.4s
  Batch 21/47 (44.7%) | Loss: 0.5333 | Acc: 0.774 | Elapsed: 7.3s | ETA: 9.0s
  Batch 22/47 (46.8%) | Loss: 0.5482 | Acc: 0.763 | Elapsed: 7.6s | ETA: 8.6s
  Batch 23/47 (48.9%) | Loss: 0.5468 | Acc: 0.761 | Elapsed: 7.9s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 0.5413 | Acc: 0.764 | Elapsed: 8.2s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 0.5335 | Acc: 0.769 | Elapsed: 8.6s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.5419 | Acc: 0.769 | Elapsed: 8.9s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 0.5371 | Acc: 0.771 | Elapsed: 9.2s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.5425 | Acc: 0.771 | Elapsed: 9.5s | ETA: 6.5s
  Batch 29/47 (61.7%) | Loss: 0.5405 | Acc: 0.775 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.5424 | Acc: 0.775 | Elapsed: 10.2s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.5515 | Acc: 0.774 | Elapsed: 10.5s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.5476 | Acc: 0.775 | Elapsed: 10.8s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.5517 | Acc: 0.773 | Elapsed: 11.1s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.5489 | Acc: 0.772 | Elapsed: 11.4s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.5479 | Acc: 0.772 | Elapsed: 11.8s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 0.5531 | Acc: 0.772 | Elapsed: 12.1s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.5577 | Acc: 0.769 | Elapsed: 12.4s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.5605 | Acc: 0.769 | Elapsed: 12.7s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.5599 | Acc: 0.768 | Elapsed: 13.0s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.5556 | Acc: 0.771 | Elapsed: 13.4s | ETA: 2.3s
  Batch 41/47 (87.2%) | Loss: 0.5573 | Acc: 0.771 | Elapsed: 13.7s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.5572 | Acc: 0.772 | Elapsed: 14.0s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.5658 | Acc: 0.770 | Elapsed: 14.3s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.5646 | Acc: 0.770 | Elapsed: 14.6s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.5617 | Acc: 0.772 | Elapsed: 15.0s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.5630 | Acc: 0.773 | Elapsed: 15.3s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.5628 | Acc: 0.773 | Elapsed: 15.3s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 2.0810 | Acc: 0.446
Epoch 21 completed in 17.1s

--- Epoch 22/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.4962 | Acc: 0.750 | Elapsed: 1.0s | ETA: 45.4s
  Batch 2/47 (4.3%) | Loss: 0.5773 | Acc: 0.750 | Elapsed: 1.3s | ETA: 29.4s
  Batch 3/47 (6.4%) | Loss: 0.5527 | Acc: 0.771 | Elapsed: 1.7s | ETA: 24.2s
  Batch 4/47 (8.5%) | Loss: 0.5380 | Acc: 0.781 | Elapsed: 2.0s | ETA: 21.3s
  Batch 5/47 (10.6%) | Loss: 0.5528 | Acc: 0.756 | Elapsed: 2.3s | ETA: 19.3s
  Batch 6/47 (12.8%) | Loss: 0.5606 | Acc: 0.745 | Elapsed: 2.6s | ETA: 18.0s
  Batch 7/47 (14.9%) | Loss: 0.5571 | Acc: 0.746 | Elapsed: 3.0s | ETA: 16.9s
  Batch 8/47 (17.0%) | Loss: 0.5898 | Acc: 0.738 | Elapsed: 3.3s | ETA: 15.9s
  Batch 9/47 (19.1%) | Loss: 0.6011 | Acc: 0.740 | Elapsed: 3.6s | ETA: 15.2s
  Batch 10/47 (21.3%) | Loss: 0.5946 | Acc: 0.744 | Elapsed: 3.9s | ETA: 14.5s
  Batch 11/47 (23.4%) | Loss: 0.5982 | Acc: 0.750 | Elapsed: 4.2s | ETA: 13.8s
  Batch 12/47 (25.5%) | Loss: 0.6100 | Acc: 0.745 | Elapsed: 4.6s | ETA: 13.3s
  Batch 13/47 (27.7%) | Loss: 0.5860 | Acc: 0.757 | Elapsed: 4.9s | ETA: 12.7s
  Batch 14/47 (29.8%) | Loss: 0.6022 | Acc: 0.748 | Elapsed: 5.2s | ETA: 12.2s
  Batch 15/47 (31.9%) | Loss: 0.5957 | Acc: 0.750 | Elapsed: 5.5s | ETA: 11.8s
  Batch 16/47 (34.0%) | Loss: 0.5980 | Acc: 0.746 | Elapsed: 5.8s | ETA: 11.3s
  Batch 17/47 (36.2%) | Loss: 0.5907 | Acc: 0.750 | Elapsed: 6.1s | ETA: 10.9s
  Batch 18/47 (38.3%) | Loss: 0.5785 | Acc: 0.757 | Elapsed: 6.5s | ETA: 10.4s
  Batch 19/47 (40.4%) | Loss: 0.5837 | Acc: 0.752 | Elapsed: 6.8s | ETA: 10.0s
  Batch 20/47 (42.6%) | Loss: 0.5805 | Acc: 0.755 | Elapsed: 7.1s | ETA: 9.6s
  Batch 21/47 (44.7%) | Loss: 0.5868 | Acc: 0.751 | Elapsed: 7.4s | ETA: 9.2s
  Batch 22/47 (46.8%) | Loss: 0.5943 | Acc: 0.747 | Elapsed: 7.8s | ETA: 8.8s
  Batch 23/47 (48.9%) | Loss: 0.5872 | Acc: 0.751 | Elapsed: 8.1s | ETA: 8.4s
  Batch 24/47 (51.1%) | Loss: 0.5768 | Acc: 0.758 | Elapsed: 8.4s | ETA: 8.0s
  Batch 25/47 (53.2%) | Loss: 0.5876 | Acc: 0.752 | Elapsed: 8.7s | ETA: 7.7s
  Batch 26/47 (55.3%) | Loss: 0.5859 | Acc: 0.751 | Elapsed: 9.0s | ETA: 7.3s
  Batch 27/47 (57.4%) | Loss: 0.5941 | Acc: 0.748 | Elapsed: 9.4s | ETA: 6.9s
  Batch 28/47 (59.6%) | Loss: 0.5892 | Acc: 0.750 | Elapsed: 9.7s | ETA: 6.6s
  Batch 29/47 (61.7%) | Loss: 0.5989 | Acc: 0.750 | Elapsed: 10.0s | ETA: 6.2s
  Batch 30/47 (63.8%) | Loss: 0.5971 | Acc: 0.752 | Elapsed: 10.3s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.5975 | Acc: 0.752 | Elapsed: 10.6s | ETA: 5.5s
  Batch 32/47 (68.1%) | Loss: 0.5904 | Acc: 0.757 | Elapsed: 11.0s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.5838 | Acc: 0.759 | Elapsed: 11.3s | ETA: 4.8s
  Batch 34/47 (72.3%) | Loss: 0.5777 | Acc: 0.763 | Elapsed: 11.6s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.5787 | Acc: 0.762 | Elapsed: 11.9s | ETA: 4.1s
  Batch 36/47 (76.6%) | Loss: 0.5758 | Acc: 0.763 | Elapsed: 12.2s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.5733 | Acc: 0.765 | Elapsed: 12.6s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.5727 | Acc: 0.766 | Elapsed: 12.9s | ETA: 3.1s
  Batch 39/47 (83.0%) | Loss: 0.5763 | Acc: 0.765 | Elapsed: 13.2s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.5697 | Acc: 0.769 | Elapsed: 13.5s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.5672 | Acc: 0.770 | Elapsed: 13.9s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.5672 | Acc: 0.769 | Elapsed: 14.2s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.5783 | Acc: 0.767 | Elapsed: 14.5s | ETA: 1.4s
  Batch 44/47 (93.6%) | Loss: 0.5734 | Acc: 0.769 | Elapsed: 14.8s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.5745 | Acc: 0.767 | Elapsed: 15.2s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.5743 | Acc: 0.766 | Elapsed: 15.5s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.5792 | Acc: 0.764 | Elapsed: 15.5s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 2.1029 | Acc: 0.507
Epoch 22 completed in 17.4s

--- Epoch 23/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.6194 | Acc: 0.750 | Elapsed: 0.8s | ETA: 35.6s
  Batch 2/47 (4.3%) | Loss: 0.5583 | Acc: 0.750 | Elapsed: 1.1s | ETA: 24.5s
  Batch 3/47 (6.4%) | Loss: 0.7997 | Acc: 0.708 | Elapsed: 1.4s | ETA: 21.0s
  Batch 4/47 (8.5%) | Loss: 0.7257 | Acc: 0.719 | Elapsed: 1.7s | ETA: 18.7s
  Batch 5/47 (10.6%) | Loss: 0.6546 | Acc: 0.756 | Elapsed: 2.1s | ETA: 17.3s
  Batch 6/47 (12.8%) | Loss: 0.6477 | Acc: 0.760 | Elapsed: 2.4s | ETA: 16.3s
  Batch 7/47 (14.9%) | Loss: 0.6535 | Acc: 0.777 | Elapsed: 2.7s | ETA: 15.5s
  Batch 8/47 (17.0%) | Loss: 0.6726 | Acc: 0.770 | Elapsed: 3.0s | ETA: 14.8s
  Batch 9/47 (19.1%) | Loss: 0.6413 | Acc: 0.774 | Elapsed: 3.3s | ETA: 14.1s
  Batch 10/47 (21.3%) | Loss: 0.7142 | Acc: 0.747 | Elapsed: 3.7s | ETA: 13.6s
  Batch 11/47 (23.4%) | Loss: 0.6943 | Acc: 0.753 | Elapsed: 4.0s | ETA: 13.1s
  Batch 12/47 (25.5%) | Loss: 0.7102 | Acc: 0.740 | Elapsed: 4.3s | ETA: 12.6s
  Batch 13/47 (27.7%) | Loss: 0.7127 | Acc: 0.733 | Elapsed: 4.6s | ETA: 12.1s
  Batch 14/47 (29.8%) | Loss: 0.6901 | Acc: 0.737 | Elapsed: 4.9s | ETA: 11.7s
  Batch 15/47 (31.9%) | Loss: 0.6985 | Acc: 0.729 | Elapsed: 5.3s | ETA: 11.2s
  Batch 16/47 (34.0%) | Loss: 0.7098 | Acc: 0.717 | Elapsed: 5.6s | ETA: 10.8s
  Batch 17/47 (36.2%) | Loss: 0.7155 | Acc: 0.713 | Elapsed: 5.9s | ETA: 10.4s
  Batch 18/47 (38.3%) | Loss: 0.7243 | Acc: 0.715 | Elapsed: 6.2s | ETA: 10.1s
  Batch 19/47 (40.4%) | Loss: 0.7286 | Acc: 0.712 | Elapsed: 6.6s | ETA: 9.7s
  Batch 20/47 (42.6%) | Loss: 0.7266 | Acc: 0.713 | Elapsed: 6.9s | ETA: 9.3s
  Batch 21/47 (44.7%) | Loss: 0.7182 | Acc: 0.716 | Elapsed: 7.2s | ETA: 8.9s
  Batch 22/47 (46.8%) | Loss: 0.7119 | Acc: 0.722 | Elapsed: 7.5s | ETA: 8.5s
  Batch 23/47 (48.9%) | Loss: 0.7137 | Acc: 0.721 | Elapsed: 7.8s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 0.7083 | Acc: 0.724 | Elapsed: 8.2s | ETA: 7.8s
  Batch 25/47 (53.2%) | Loss: 0.6974 | Acc: 0.729 | Elapsed: 8.5s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.6900 | Acc: 0.730 | Elapsed: 8.8s | ETA: 7.1s
  Batch 27/47 (57.4%) | Loss: 0.6802 | Acc: 0.735 | Elapsed: 9.1s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.6726 | Acc: 0.738 | Elapsed: 9.4s | ETA: 6.4s
  Batch 29/47 (61.7%) | Loss: 0.6706 | Acc: 0.734 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.6641 | Acc: 0.735 | Elapsed: 10.1s | ETA: 5.7s
  Batch 31/47 (66.0%) | Loss: 0.6602 | Acc: 0.736 | Elapsed: 10.4s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.6536 | Acc: 0.741 | Elapsed: 10.7s | ETA: 5.0s
  Batch 33/47 (70.2%) | Loss: 0.6521 | Acc: 0.741 | Elapsed: 11.1s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.6443 | Acc: 0.745 | Elapsed: 11.4s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.6470 | Acc: 0.744 | Elapsed: 11.7s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 0.6397 | Acc: 0.747 | Elapsed: 12.0s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.6352 | Acc: 0.748 | Elapsed: 12.4s | ETA: 3.3s
  Batch 38/47 (80.9%) | Loss: 0.6372 | Acc: 0.748 | Elapsed: 12.7s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.6488 | Acc: 0.744 | Elapsed: 13.0s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.6519 | Acc: 0.743 | Elapsed: 13.3s | ETA: 2.3s
  Batch 41/47 (87.2%) | Loss: 0.6447 | Acc: 0.748 | Elapsed: 13.7s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.6415 | Acc: 0.749 | Elapsed: 14.0s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.6411 | Acc: 0.749 | Elapsed: 14.3s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.6369 | Acc: 0.751 | Elapsed: 14.6s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.6400 | Acc: 0.748 | Elapsed: 14.9s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.6423 | Acc: 0.748 | Elapsed: 15.2s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.6451 | Acc: 0.747 | Elapsed: 15.3s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.6616 | Acc: 0.478
Epoch 23 completed in 17.0s

--- Epoch 24/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.7051 | Acc: 0.781 | Elapsed: 0.8s | ETA: 34.8s
  Batch 2/47 (4.3%) | Loss: 0.6678 | Acc: 0.734 | Elapsed: 1.1s | ETA: 24.1s
  Batch 3/47 (6.4%) | Loss: 0.5781 | Acc: 0.760 | Elapsed: 1.4s | ETA: 20.7s
  Batch 4/47 (8.5%) | Loss: 0.5976 | Acc: 0.742 | Elapsed: 1.7s | ETA: 18.6s
  Batch 5/47 (10.6%) | Loss: 0.6483 | Acc: 0.725 | Elapsed: 2.0s | ETA: 17.2s
  Batch 6/47 (12.8%) | Loss: 0.6255 | Acc: 0.734 | Elapsed: 2.4s | ETA: 16.2s
  Batch 7/47 (14.9%) | Loss: 0.6187 | Acc: 0.737 | Elapsed: 2.7s | ETA: 15.3s
  Batch 8/47 (17.0%) | Loss: 0.6088 | Acc: 0.742 | Elapsed: 3.0s | ETA: 14.6s
  Batch 9/47 (19.1%) | Loss: 0.5982 | Acc: 0.750 | Elapsed: 3.3s | ETA: 14.0s
  Batch 10/47 (21.3%) | Loss: 0.5970 | Acc: 0.741 | Elapsed: 3.6s | ETA: 13.5s
  Batch 11/47 (23.4%) | Loss: 0.5840 | Acc: 0.744 | Elapsed: 4.0s | ETA: 13.0s
  Batch 12/47 (25.5%) | Loss: 0.5729 | Acc: 0.747 | Elapsed: 4.3s | ETA: 12.5s
  Batch 13/47 (27.7%) | Loss: 0.5598 | Acc: 0.755 | Elapsed: 4.6s | ETA: 12.0s
  Batch 14/47 (29.8%) | Loss: 0.5578 | Acc: 0.759 | Elapsed: 4.9s | ETA: 11.6s
  Batch 15/47 (31.9%) | Loss: 0.5770 | Acc: 0.758 | Elapsed: 5.2s | ETA: 11.2s
  Batch 16/47 (34.0%) | Loss: 0.5640 | Acc: 0.762 | Elapsed: 5.6s | ETA: 10.8s
  Batch 17/47 (36.2%) | Loss: 0.5654 | Acc: 0.763 | Elapsed: 5.9s | ETA: 10.4s
  Batch 18/47 (38.3%) | Loss: 0.5606 | Acc: 0.767 | Elapsed: 6.2s | ETA: 10.0s
  Batch 19/47 (40.4%) | Loss: 0.5678 | Acc: 0.766 | Elapsed: 6.5s | ETA: 9.6s
  Batch 20/47 (42.6%) | Loss: 0.5579 | Acc: 0.772 | Elapsed: 6.9s | ETA: 9.3s
  Batch 21/47 (44.7%) | Loss: 0.5521 | Acc: 0.774 | Elapsed: 7.2s | ETA: 8.9s
  Batch 22/47 (46.8%) | Loss: 0.5523 | Acc: 0.771 | Elapsed: 7.5s | ETA: 8.5s
  Batch 23/47 (48.9%) | Loss: 0.5530 | Acc: 0.770 | Elapsed: 7.8s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 0.5528 | Acc: 0.770 | Elapsed: 8.2s | ETA: 7.8s
  Batch 25/47 (53.2%) | Loss: 0.5526 | Acc: 0.766 | Elapsed: 8.5s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.5531 | Acc: 0.767 | Elapsed: 8.8s | ETA: 7.1s
  Batch 27/47 (57.4%) | Loss: 0.5507 | Acc: 0.767 | Elapsed: 9.1s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.5531 | Acc: 0.763 | Elapsed: 9.5s | ETA: 6.4s
  Batch 29/47 (61.7%) | Loss: 0.5566 | Acc: 0.763 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.5660 | Acc: 0.761 | Elapsed: 10.1s | ETA: 5.7s
  Batch 31/47 (66.0%) | Loss: 0.5626 | Acc: 0.764 | Elapsed: 10.4s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.5647 | Acc: 0.760 | Elapsed: 10.8s | ETA: 5.0s
  Batch 33/47 (70.2%) | Loss: 0.5632 | Acc: 0.761 | Elapsed: 11.1s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.5617 | Acc: 0.762 | Elapsed: 11.4s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.5640 | Acc: 0.760 | Elapsed: 11.7s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 0.5616 | Acc: 0.762 | Elapsed: 12.0s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.5641 | Acc: 0.764 | Elapsed: 12.4s | ETA: 3.3s
  Batch 38/47 (80.9%) | Loss: 0.5682 | Acc: 0.764 | Elapsed: 12.7s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.5691 | Acc: 0.764 | Elapsed: 13.0s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.5665 | Acc: 0.765 | Elapsed: 13.3s | ETA: 2.3s
  Batch 41/47 (87.2%) | Loss: 0.5675 | Acc: 0.764 | Elapsed: 13.7s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.5643 | Acc: 0.766 | Elapsed: 14.0s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.5635 | Acc: 0.765 | Elapsed: 14.3s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.5668 | Acc: 0.761 | Elapsed: 14.6s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.5676 | Acc: 0.762 | Elapsed: 14.9s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.5698 | Acc: 0.760 | Elapsed: 15.3s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.5689 | Acc: 0.761 | Elapsed: 15.3s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.3001 | Acc: 0.563
Epoch 24 completed in 17.0s

--- Epoch 25/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.4612 | Acc: 0.781 | Elapsed: 0.8s | ETA: 35.3s
  Batch 2/47 (4.3%) | Loss: 0.4188 | Acc: 0.844 | Elapsed: 1.1s | ETA: 24.3s
  Batch 3/47 (6.4%) | Loss: 0.5073 | Acc: 0.802 | Elapsed: 1.4s | ETA: 20.9s
  Batch 4/47 (8.5%) | Loss: 0.4800 | Acc: 0.805 | Elapsed: 1.7s | ETA: 18.7s
  Batch 5/47 (10.6%) | Loss: 0.4555 | Acc: 0.825 | Elapsed: 2.1s | ETA: 17.2s
  Batch 6/47 (12.8%) | Loss: 0.5152 | Acc: 0.823 | Elapsed: 2.4s | ETA: 16.3s
  Batch 7/47 (14.9%) | Loss: 0.5031 | Acc: 0.826 | Elapsed: 2.7s | ETA: 15.5s
  Batch 8/47 (17.0%) | Loss: 0.5220 | Acc: 0.816 | Elapsed: 3.0s | ETA: 14.8s
  Batch 9/47 (19.1%) | Loss: 0.5391 | Acc: 0.809 | Elapsed: 3.4s | ETA: 14.2s
  Batch 10/47 (21.3%) | Loss: 0.5232 | Acc: 0.816 | Elapsed: 3.7s | ETA: 13.6s
  Batch 11/47 (23.4%) | Loss: 0.5387 | Acc: 0.807 | Elapsed: 4.0s | ETA: 13.1s
  Batch 12/47 (25.5%) | Loss: 0.5259 | Acc: 0.807 | Elapsed: 4.3s | ETA: 12.6s
  Batch 13/47 (27.7%) | Loss: 0.5278 | Acc: 0.798 | Elapsed: 4.7s | ETA: 12.2s
  Batch 14/47 (29.8%) | Loss: 0.5319 | Acc: 0.790 | Elapsed: 5.0s | ETA: 11.8s
  Batch 15/47 (31.9%) | Loss: 0.5419 | Acc: 0.787 | Elapsed: 5.3s | ETA: 11.4s
  Batch 16/47 (34.0%) | Loss: 0.5468 | Acc: 0.783 | Elapsed: 5.6s | ETA: 10.9s
  Batch 17/47 (36.2%) | Loss: 0.5399 | Acc: 0.785 | Elapsed: 6.0s | ETA: 10.5s
  Batch 18/47 (38.3%) | Loss: 0.5361 | Acc: 0.785 | Elapsed: 6.3s | ETA: 10.1s
  Batch 19/47 (40.4%) | Loss: 0.5362 | Acc: 0.789 | Elapsed: 6.6s | ETA: 9.7s
  Batch 20/47 (42.6%) | Loss: 0.5335 | Acc: 0.789 | Elapsed: 6.9s | ETA: 9.4s
  Batch 21/47 (44.7%) | Loss: 0.5325 | Acc: 0.793 | Elapsed: 7.3s | ETA: 9.0s
  Batch 22/47 (46.8%) | Loss: 0.5389 | Acc: 0.790 | Elapsed: 7.6s | ETA: 8.6s
  Batch 23/47 (48.9%) | Loss: 0.5375 | Acc: 0.789 | Elapsed: 7.9s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 0.5677 | Acc: 0.779 | Elapsed: 8.2s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 0.5815 | Acc: 0.770 | Elapsed: 8.5s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.5728 | Acc: 0.772 | Elapsed: 8.9s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 0.5731 | Acc: 0.772 | Elapsed: 9.2s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.5695 | Acc: 0.773 | Elapsed: 9.5s | ETA: 6.4s
  Batch 29/47 (61.7%) | Loss: 0.5648 | Acc: 0.777 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.5634 | Acc: 0.775 | Elapsed: 10.1s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.5622 | Acc: 0.776 | Elapsed: 10.5s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.5556 | Acc: 0.780 | Elapsed: 10.8s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.5618 | Acc: 0.777 | Elapsed: 11.1s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.5604 | Acc: 0.778 | Elapsed: 11.4s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.5595 | Acc: 0.779 | Elapsed: 11.8s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 0.5602 | Acc: 0.780 | Elapsed: 12.1s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.5549 | Acc: 0.783 | Elapsed: 12.4s | ETA: 3.3s
  Batch 38/47 (80.9%) | Loss: 0.5506 | Acc: 0.783 | Elapsed: 12.7s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.5485 | Acc: 0.782 | Elapsed: 13.0s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.5504 | Acc: 0.781 | Elapsed: 13.4s | ETA: 2.3s
  Batch 41/47 (87.2%) | Loss: 0.5541 | Acc: 0.779 | Elapsed: 13.7s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.5530 | Acc: 0.780 | Elapsed: 14.0s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.5563 | Acc: 0.778 | Elapsed: 14.3s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.5570 | Acc: 0.778 | Elapsed: 14.6s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.5538 | Acc: 0.778 | Elapsed: 15.0s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.5522 | Acc: 0.781 | Elapsed: 15.3s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.5524 | Acc: 0.781 | Elapsed: 15.4s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.3772 | Acc: 0.530
Epoch 25 completed in 17.9s

--- Epoch 26/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.4456 | Acc: 0.781 | Elapsed: 1.1s | ETA: 51.2s
  Batch 2/47 (4.3%) | Loss: 0.6453 | Acc: 0.734 | Elapsed: 1.4s | ETA: 32.1s
  Batch 3/47 (6.4%) | Loss: 0.6133 | Acc: 0.719 | Elapsed: 1.8s | ETA: 26.4s
  Batch 4/47 (8.5%) | Loss: 0.6411 | Acc: 0.727 | Elapsed: 2.1s | ETA: 22.8s
  Batch 5/47 (10.6%) | Loss: 0.6128 | Acc: 0.731 | Elapsed: 2.5s | ETA: 20.6s
  Batch 6/47 (12.8%) | Loss: 0.5940 | Acc: 0.740 | Elapsed: 2.8s | ETA: 19.0s
  Batch 7/47 (14.9%) | Loss: 0.5718 | Acc: 0.754 | Elapsed: 3.1s | ETA: 17.7s
  Batch 8/47 (17.0%) | Loss: 0.5691 | Acc: 0.750 | Elapsed: 3.4s | ETA: 16.7s
  Batch 9/47 (19.1%) | Loss: 0.5821 | Acc: 0.740 | Elapsed: 3.7s | ETA: 15.8s
  Batch 10/47 (21.3%) | Loss: 0.5579 | Acc: 0.756 | Elapsed: 4.1s | ETA: 15.0s
  Batch 11/47 (23.4%) | Loss: 0.5600 | Acc: 0.753 | Elapsed: 4.4s | ETA: 14.4s
  Batch 12/47 (25.5%) | Loss: 0.5519 | Acc: 0.758 | Elapsed: 4.7s | ETA: 13.7s
  Batch 13/47 (27.7%) | Loss: 0.5594 | Acc: 0.757 | Elapsed: 5.0s | ETA: 13.2s
  Batch 14/47 (29.8%) | Loss: 0.5515 | Acc: 0.766 | Elapsed: 5.4s | ETA: 12.6s
  Batch 15/47 (31.9%) | Loss: 0.5592 | Acc: 0.758 | Elapsed: 5.7s | ETA: 12.1s
  Batch 16/47 (34.0%) | Loss: 0.5453 | Acc: 0.770 | Elapsed: 6.0s | ETA: 11.6s
  Batch 17/47 (36.2%) | Loss: 0.5522 | Acc: 0.765 | Elapsed: 6.3s | ETA: 11.1s
  Batch 18/47 (38.3%) | Loss: 0.5449 | Acc: 0.769 | Elapsed: 6.6s | ETA: 10.7s
  Batch 19/47 (40.4%) | Loss: 0.5526 | Acc: 0.763 | Elapsed: 7.0s | ETA: 10.2s
  Batch 20/47 (42.6%) | Loss: 0.5548 | Acc: 0.766 | Elapsed: 7.3s | ETA: 9.8s
  Batch 21/47 (44.7%) | Loss: 0.5424 | Acc: 0.772 | Elapsed: 7.6s | ETA: 9.4s
  Batch 22/47 (46.8%) | Loss: 0.5405 | Acc: 0.774 | Elapsed: 7.9s | ETA: 9.0s
  Batch 23/47 (48.9%) | Loss: 0.5370 | Acc: 0.779 | Elapsed: 8.2s | ETA: 8.6s
  Batch 24/47 (51.1%) | Loss: 0.5322 | Acc: 0.781 | Elapsed: 8.6s | ETA: 8.2s
  Batch 25/47 (53.2%) | Loss: 0.5252 | Acc: 0.784 | Elapsed: 8.9s | ETA: 7.8s
  Batch 26/47 (55.3%) | Loss: 0.5215 | Acc: 0.786 | Elapsed: 9.2s | ETA: 7.4s
  Batch 27/47 (57.4%) | Loss: 0.5205 | Acc: 0.785 | Elapsed: 9.5s | ETA: 7.1s
  Batch 28/47 (59.6%) | Loss: 0.5209 | Acc: 0.786 | Elapsed: 9.8s | ETA: 6.7s
  Batch 29/47 (61.7%) | Loss: 0.5161 | Acc: 0.787 | Elapsed: 10.2s | ETA: 6.3s
  Batch 30/47 (63.8%) | Loss: 0.5235 | Acc: 0.782 | Elapsed: 10.5s | ETA: 5.9s
  Batch 31/47 (66.0%) | Loss: 0.5220 | Acc: 0.784 | Elapsed: 10.8s | ETA: 5.6s
  Batch 32/47 (68.1%) | Loss: 0.5217 | Acc: 0.784 | Elapsed: 11.1s | ETA: 5.2s
  Batch 33/47 (70.2%) | Loss: 0.5188 | Acc: 0.783 | Elapsed: 11.5s | ETA: 4.9s
  Batch 34/47 (72.3%) | Loss: 0.5177 | Acc: 0.784 | Elapsed: 11.8s | ETA: 4.5s
  Batch 35/47 (74.5%) | Loss: 0.5197 | Acc: 0.784 | Elapsed: 12.1s | ETA: 4.1s
  Batch 36/47 (76.6%) | Loss: 0.5194 | Acc: 0.786 | Elapsed: 12.4s | ETA: 3.8s
  Batch 37/47 (78.7%) | Loss: 0.5187 | Acc: 0.787 | Elapsed: 12.7s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.5204 | Acc: 0.787 | Elapsed: 13.1s | ETA: 3.1s
  Batch 39/47 (83.0%) | Loss: 0.5179 | Acc: 0.788 | Elapsed: 13.4s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.5139 | Acc: 0.790 | Elapsed: 13.7s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.5146 | Acc: 0.790 | Elapsed: 14.0s | ETA: 2.1s
  Batch 42/47 (89.4%) | Loss: 0.5180 | Acc: 0.789 | Elapsed: 14.4s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.5174 | Acc: 0.791 | Elapsed: 14.7s | ETA: 1.4s
  Batch 44/47 (93.6%) | Loss: 0.5223 | Acc: 0.788 | Elapsed: 15.0s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.5187 | Acc: 0.791 | Elapsed: 15.3s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.5191 | Acc: 0.791 | Elapsed: 15.7s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.5190 | Acc: 0.791 | Elapsed: 15.7s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.6021 | Acc: 0.565
Epoch 26 completed in 17.5s

--- Epoch 27/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.2931 | Acc: 0.906 | Elapsed: 0.8s | ETA: 36.7s
  Batch 2/47 (4.3%) | Loss: 0.4266 | Acc: 0.844 | Elapsed: 1.1s | ETA: 25.0s
  Batch 3/47 (6.4%) | Loss: 0.4179 | Acc: 0.844 | Elapsed: 1.5s | ETA: 21.4s
  Batch 4/47 (8.5%) | Loss: 0.4370 | Acc: 0.820 | Elapsed: 1.8s | ETA: 19.1s
  Batch 5/47 (10.6%) | Loss: 0.4567 | Acc: 0.812 | Elapsed: 2.1s | ETA: 17.5s
  Batch 6/47 (12.8%) | Loss: 0.4562 | Acc: 0.823 | Elapsed: 2.4s | ETA: 16.5s
  Batch 7/47 (14.9%) | Loss: 0.4538 | Acc: 0.826 | Elapsed: 2.7s | ETA: 15.7s
  Batch 8/47 (17.0%) | Loss: 0.4449 | Acc: 0.828 | Elapsed: 3.1s | ETA: 14.9s
  Batch 9/47 (19.1%) | Loss: 0.4636 | Acc: 0.819 | Elapsed: 3.4s | ETA: 14.3s
  Batch 10/47 (21.3%) | Loss: 0.4914 | Acc: 0.803 | Elapsed: 3.7s | ETA: 13.7s
  Batch 11/47 (23.4%) | Loss: 0.5237 | Acc: 0.793 | Elapsed: 4.0s | ETA: 13.2s
  Batch 12/47 (25.5%) | Loss: 0.5156 | Acc: 0.794 | Elapsed: 4.4s | ETA: 12.7s
  Batch 13/47 (27.7%) | Loss: 0.5293 | Acc: 0.788 | Elapsed: 4.7s | ETA: 12.2s
  Batch 14/47 (29.8%) | Loss: 0.5329 | Acc: 0.786 | Elapsed: 5.0s | ETA: 11.8s
  Batch 15/47 (31.9%) | Loss: 0.5386 | Acc: 0.783 | Elapsed: 5.3s | ETA: 11.4s
  Batch 16/47 (34.0%) | Loss: 0.5379 | Acc: 0.783 | Elapsed: 5.6s | ETA: 10.9s
  Batch 17/47 (36.2%) | Loss: 0.5313 | Acc: 0.789 | Elapsed: 6.0s | ETA: 10.5s
  Batch 18/47 (38.3%) | Loss: 0.5347 | Acc: 0.788 | Elapsed: 6.3s | ETA: 10.1s
  Batch 19/47 (40.4%) | Loss: 0.5315 | Acc: 0.788 | Elapsed: 6.6s | ETA: 9.7s
  Batch 20/47 (42.6%) | Loss: 0.5393 | Acc: 0.783 | Elapsed: 6.9s | ETA: 9.4s
  Batch 21/47 (44.7%) | Loss: 0.5386 | Acc: 0.784 | Elapsed: 7.3s | ETA: 9.0s
  Batch 22/47 (46.8%) | Loss: 0.5428 | Acc: 0.784 | Elapsed: 7.6s | ETA: 8.6s
  Batch 23/47 (48.9%) | Loss: 0.5381 | Acc: 0.787 | Elapsed: 7.9s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 0.5334 | Acc: 0.789 | Elapsed: 8.2s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 0.5294 | Acc: 0.789 | Elapsed: 8.5s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.5279 | Acc: 0.788 | Elapsed: 8.9s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 0.5245 | Acc: 0.789 | Elapsed: 9.2s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.5247 | Acc: 0.788 | Elapsed: 9.5s | ETA: 6.5s
  Batch 29/47 (61.7%) | Loss: 0.5269 | Acc: 0.787 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.5236 | Acc: 0.786 | Elapsed: 10.2s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.5253 | Acc: 0.785 | Elapsed: 10.5s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.5229 | Acc: 0.785 | Elapsed: 10.8s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.5198 | Acc: 0.785 | Elapsed: 11.1s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.5165 | Acc: 0.785 | Elapsed: 11.5s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.5147 | Acc: 0.787 | Elapsed: 11.8s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 0.5136 | Acc: 0.788 | Elapsed: 12.1s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.5105 | Acc: 0.787 | Elapsed: 12.4s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.5129 | Acc: 0.787 | Elapsed: 12.8s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.5107 | Acc: 0.788 | Elapsed: 13.1s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.5164 | Acc: 0.785 | Elapsed: 13.4s | ETA: 2.3s
  Batch 41/47 (87.2%) | Loss: 0.5190 | Acc: 0.785 | Elapsed: 13.7s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.5137 | Acc: 0.789 | Elapsed: 14.0s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.5144 | Acc: 0.790 | Elapsed: 14.4s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.5224 | Acc: 0.786 | Elapsed: 14.7s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.5257 | Acc: 0.785 | Elapsed: 15.0s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.5266 | Acc: 0.785 | Elapsed: 15.3s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.5264 | Acc: 0.785 | Elapsed: 15.4s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.1888 | Acc: 0.565
Epoch 27 completed in 17.1s

--- Epoch 28/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.5847 | Acc: 0.719 | Elapsed: 0.8s | ETA: 36.6s
  Batch 2/47 (4.3%) | Loss: 0.4800 | Acc: 0.781 | Elapsed: 1.1s | ETA: 25.0s
  Batch 3/47 (6.4%) | Loss: 0.4744 | Acc: 0.771 | Elapsed: 1.5s | ETA: 21.3s
  Batch 4/47 (8.5%) | Loss: 0.4443 | Acc: 0.797 | Elapsed: 1.8s | ETA: 19.0s
  Batch 5/47 (10.6%) | Loss: 0.4408 | Acc: 0.806 | Elapsed: 2.1s | ETA: 17.5s
  Batch 6/47 (12.8%) | Loss: 0.4302 | Acc: 0.823 | Elapsed: 2.4s | ETA: 16.5s
  Batch 7/47 (14.9%) | Loss: 0.4587 | Acc: 0.817 | Elapsed: 2.7s | ETA: 15.6s
  Batch 8/47 (17.0%) | Loss: 0.4572 | Acc: 0.824 | Elapsed: 3.1s | ETA: 14.9s
  Batch 9/47 (19.1%) | Loss: 0.4608 | Acc: 0.819 | Elapsed: 3.4s | ETA: 14.3s
  Batch 10/47 (21.3%) | Loss: 0.4692 | Acc: 0.822 | Elapsed: 3.7s | ETA: 13.7s
  Batch 11/47 (23.4%) | Loss: 0.4933 | Acc: 0.812 | Elapsed: 4.0s | ETA: 13.2s
  Batch 12/47 (25.5%) | Loss: 0.4881 | Acc: 0.807 | Elapsed: 4.3s | ETA: 12.7s
  Batch 13/47 (27.7%) | Loss: 0.5058 | Acc: 0.798 | Elapsed: 4.7s | ETA: 12.2s
  Batch 14/47 (29.8%) | Loss: 0.5022 | Acc: 0.795 | Elapsed: 5.0s | ETA: 11.8s
  Batch 15/47 (31.9%) | Loss: 0.4844 | Acc: 0.802 | Elapsed: 5.3s | ETA: 11.3s
  Batch 16/47 (34.0%) | Loss: 0.4728 | Acc: 0.811 | Elapsed: 5.6s | ETA: 10.9s
  Batch 17/47 (36.2%) | Loss: 0.4766 | Acc: 0.809 | Elapsed: 6.0s | ETA: 10.5s
  Batch 18/47 (38.3%) | Loss: 0.4665 | Acc: 0.814 | Elapsed: 6.3s | ETA: 10.1s
  Batch 19/47 (40.4%) | Loss: 0.4691 | Acc: 0.811 | Elapsed: 6.6s | ETA: 9.7s
  Batch 20/47 (42.6%) | Loss: 0.4877 | Acc: 0.803 | Elapsed: 6.9s | ETA: 9.4s
  Batch 21/47 (44.7%) | Loss: 0.4918 | Acc: 0.801 | Elapsed: 7.2s | ETA: 9.0s
  Batch 22/47 (46.8%) | Loss: 0.4920 | Acc: 0.800 | Elapsed: 7.6s | ETA: 8.6s
  Batch 23/47 (48.9%) | Loss: 0.4952 | Acc: 0.802 | Elapsed: 7.9s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 0.4959 | Acc: 0.802 | Elapsed: 8.2s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 0.4930 | Acc: 0.802 | Elapsed: 8.5s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.4865 | Acc: 0.805 | Elapsed: 8.9s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 0.4847 | Acc: 0.807 | Elapsed: 9.2s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.4825 | Acc: 0.807 | Elapsed: 9.5s | ETA: 6.5s
  Batch 29/47 (61.7%) | Loss: 0.4814 | Acc: 0.808 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.4883 | Acc: 0.804 | Elapsed: 10.2s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.4932 | Acc: 0.802 | Elapsed: 10.5s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.4960 | Acc: 0.801 | Elapsed: 10.8s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.4910 | Acc: 0.803 | Elapsed: 11.1s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.4853 | Acc: 0.806 | Elapsed: 11.4s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.4870 | Acc: 0.804 | Elapsed: 11.8s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 0.4840 | Acc: 0.806 | Elapsed: 12.1s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.4833 | Acc: 0.807 | Elapsed: 12.4s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.4807 | Acc: 0.808 | Elapsed: 12.7s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.4770 | Acc: 0.811 | Elapsed: 13.0s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.4796 | Acc: 0.810 | Elapsed: 13.4s | ETA: 2.3s
  Batch 41/47 (87.2%) | Loss: 0.4818 | Acc: 0.809 | Elapsed: 13.7s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.4847 | Acc: 0.807 | Elapsed: 14.0s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.4857 | Acc: 0.808 | Elapsed: 14.3s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.4932 | Acc: 0.806 | Elapsed: 14.7s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.4877 | Acc: 0.808 | Elapsed: 15.0s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.4879 | Acc: 0.808 | Elapsed: 15.3s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.4870 | Acc: 0.808 | Elapsed: 15.4s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 2.0947 | Acc: 0.444
Epoch 28 completed in 17.1s

--- Epoch 29/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.3408 | Acc: 0.875 | Elapsed: 0.8s | ETA: 35.9s
  Batch 2/47 (4.3%) | Loss: 0.3220 | Acc: 0.844 | Elapsed: 1.1s | ETA: 24.8s
  Batch 3/47 (6.4%) | Loss: 0.3582 | Acc: 0.844 | Elapsed: 1.5s | ETA: 21.3s
  Batch 4/47 (8.5%) | Loss: 0.4286 | Acc: 0.805 | Elapsed: 1.8s | ETA: 19.0s
  Batch 5/47 (10.6%) | Loss: 0.4097 | Acc: 0.812 | Elapsed: 2.1s | ETA: 17.5s
  Batch 6/47 (12.8%) | Loss: 0.4149 | Acc: 0.818 | Elapsed: 2.4s | ETA: 16.5s
  Batch 7/47 (14.9%) | Loss: 0.4224 | Acc: 0.817 | Elapsed: 2.7s | ETA: 15.7s
  Batch 8/47 (17.0%) | Loss: 0.4305 | Acc: 0.812 | Elapsed: 3.1s | ETA: 15.0s
  Batch 9/47 (19.1%) | Loss: 0.4295 | Acc: 0.812 | Elapsed: 3.4s | ETA: 14.3s
  Batch 10/47 (21.3%) | Loss: 0.4697 | Acc: 0.803 | Elapsed: 3.7s | ETA: 13.8s
  Batch 11/47 (23.4%) | Loss: 0.5057 | Acc: 0.793 | Elapsed: 4.0s | ETA: 13.2s
  Batch 12/47 (25.5%) | Loss: 0.5077 | Acc: 0.789 | Elapsed: 4.4s | ETA: 12.7s
  Batch 13/47 (27.7%) | Loss: 0.5013 | Acc: 0.796 | Elapsed: 4.7s | ETA: 12.3s
  Batch 14/47 (29.8%) | Loss: 0.4950 | Acc: 0.797 | Elapsed: 5.0s | ETA: 11.8s
  Batch 15/47 (31.9%) | Loss: 0.5010 | Acc: 0.796 | Elapsed: 5.3s | ETA: 11.4s
  Batch 16/47 (34.0%) | Loss: 0.5176 | Acc: 0.787 | Elapsed: 5.7s | ETA: 11.0s
  Batch 17/47 (36.2%) | Loss: 0.5397 | Acc: 0.781 | Elapsed: 6.0s | ETA: 10.6s
  Batch 18/47 (38.3%) | Loss: 0.5352 | Acc: 0.783 | Elapsed: 6.3s | ETA: 10.2s
  Batch 19/47 (40.4%) | Loss: 0.5374 | Acc: 0.780 | Elapsed: 6.6s | ETA: 9.8s
  Batch 20/47 (42.6%) | Loss: 0.5326 | Acc: 0.781 | Elapsed: 6.9s | ETA: 9.4s
  Batch 21/47 (44.7%) | Loss: 0.5295 | Acc: 0.783 | Elapsed: 7.3s | ETA: 9.0s
  Batch 22/47 (46.8%) | Loss: 0.5244 | Acc: 0.788 | Elapsed: 7.6s | ETA: 8.6s
  Batch 23/47 (48.9%) | Loss: 0.5221 | Acc: 0.789 | Elapsed: 7.9s | ETA: 8.3s
  Batch 24/47 (51.1%) | Loss: 0.5312 | Acc: 0.788 | Elapsed: 8.2s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 0.5358 | Acc: 0.785 | Elapsed: 8.6s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.5308 | Acc: 0.785 | Elapsed: 8.9s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 0.5306 | Acc: 0.785 | Elapsed: 9.2s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.5252 | Acc: 0.788 | Elapsed: 9.5s | ETA: 6.5s
  Batch 29/47 (61.7%) | Loss: 0.5167 | Acc: 0.793 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.5182 | Acc: 0.791 | Elapsed: 10.2s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.5191 | Acc: 0.790 | Elapsed: 10.5s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.5128 | Acc: 0.792 | Elapsed: 10.8s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.5104 | Acc: 0.792 | Elapsed: 11.1s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.5044 | Acc: 0.795 | Elapsed: 11.5s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.4986 | Acc: 0.799 | Elapsed: 11.8s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 0.4983 | Acc: 0.799 | Elapsed: 12.1s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.4961 | Acc: 0.802 | Elapsed: 12.4s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.4974 | Acc: 0.803 | Elapsed: 12.8s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.4982 | Acc: 0.804 | Elapsed: 13.1s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.4916 | Acc: 0.806 | Elapsed: 13.4s | ETA: 2.3s
  Batch 41/47 (87.2%) | Loss: 0.4934 | Acc: 0.805 | Elapsed: 13.7s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.4895 | Acc: 0.807 | Elapsed: 14.0s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.4917 | Acc: 0.808 | Elapsed: 14.4s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.4908 | Acc: 0.808 | Elapsed: 14.7s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.5022 | Acc: 0.805 | Elapsed: 15.0s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.5075 | Acc: 0.804 | Elapsed: 15.3s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.5094 | Acc: 0.803 | Elapsed: 15.4s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.4319 | Acc: 0.571
Epoch 29 completed in 17.6s

--- Epoch 30/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.3872 | Acc: 0.750 | Elapsed: 0.9s | ETA: 41.5s
  Batch 2/47 (4.3%) | Loss: 0.4047 | Acc: 0.797 | Elapsed: 1.2s | ETA: 27.4s
  Batch 3/47 (6.4%) | Loss: 0.3998 | Acc: 0.812 | Elapsed: 1.6s | ETA: 22.9s
  Batch 4/47 (8.5%) | Loss: 0.3961 | Acc: 0.812 | Elapsed: 1.9s | ETA: 20.1s
  Batch 5/47 (10.6%) | Loss: 0.3814 | Acc: 0.825 | Elapsed: 2.2s | ETA: 18.4s
  Batch 6/47 (12.8%) | Loss: 0.3871 | Acc: 0.812 | Elapsed: 2.5s | ETA: 17.2s
  Batch 7/47 (14.9%) | Loss: 0.4233 | Acc: 0.812 | Elapsed: 2.8s | ETA: 16.3s
  Batch 8/47 (17.0%) | Loss: 0.4196 | Acc: 0.820 | Elapsed: 3.2s | ETA: 15.4s
  Batch 9/47 (19.1%) | Loss: 0.4522 | Acc: 0.816 | Elapsed: 3.5s | ETA: 14.7s
  Batch 10/47 (21.3%) | Loss: 0.4873 | Acc: 0.809 | Elapsed: 3.8s | ETA: 14.1s
  Batch 11/47 (23.4%) | Loss: 0.4726 | Acc: 0.812 | Elapsed: 4.1s | ETA: 13.5s
  Batch 12/47 (25.5%) | Loss: 0.5367 | Acc: 0.792 | Elapsed: 4.4s | ETA: 13.0s
  Batch 13/47 (27.7%) | Loss: 0.5350 | Acc: 0.788 | Elapsed: 4.8s | ETA: 12.5s
  Batch 14/47 (29.8%) | Loss: 0.5420 | Acc: 0.783 | Elapsed: 5.1s | ETA: 12.0s
  Batch 15/47 (31.9%) | Loss: 0.5434 | Acc: 0.783 | Elapsed: 5.4s | ETA: 11.5s
  Batch 16/47 (34.0%) | Loss: 0.5321 | Acc: 0.789 | Elapsed: 5.7s | ETA: 11.1s
  Batch 17/47 (36.2%) | Loss: 0.5210 | Acc: 0.798 | Elapsed: 6.0s | ETA: 10.7s
  Batch 18/47 (38.3%) | Loss: 0.5092 | Acc: 0.806 | Elapsed: 6.4s | ETA: 10.3s
  Batch 19/47 (40.4%) | Loss: 0.5112 | Acc: 0.804 | Elapsed: 6.7s | ETA: 9.9s
  Batch 20/47 (42.6%) | Loss: 0.5031 | Acc: 0.808 | Elapsed: 7.0s | ETA: 9.5s
  Batch 21/47 (44.7%) | Loss: 0.4956 | Acc: 0.810 | Elapsed: 7.3s | ETA: 9.1s
  Batch 22/47 (46.8%) | Loss: 0.5023 | Acc: 0.805 | Elapsed: 7.7s | ETA: 8.7s
  Batch 23/47 (48.9%) | Loss: 0.4934 | Acc: 0.808 | Elapsed: 8.0s | ETA: 8.3s
  Batch 24/47 (51.1%) | Loss: 0.4920 | Acc: 0.810 | Elapsed: 8.3s | ETA: 8.0s
  Batch 25/47 (53.2%) | Loss: 0.4978 | Acc: 0.806 | Elapsed: 8.6s | ETA: 7.6s
  Batch 26/47 (55.3%) | Loss: 0.4945 | Acc: 0.806 | Elapsed: 8.9s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 0.4971 | Acc: 0.802 | Elapsed: 9.3s | ETA: 6.9s
  Batch 28/47 (59.6%) | Loss: 0.4898 | Acc: 0.806 | Elapsed: 9.6s | ETA: 6.5s
  Batch 29/47 (61.7%) | Loss: 0.4937 | Acc: 0.805 | Elapsed: 9.9s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.4890 | Acc: 0.807 | Elapsed: 10.2s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.4824 | Acc: 0.809 | Elapsed: 10.5s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.4812 | Acc: 0.810 | Elapsed: 10.9s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.4817 | Acc: 0.809 | Elapsed: 11.2s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.4785 | Acc: 0.811 | Elapsed: 11.5s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.4778 | Acc: 0.812 | Elapsed: 11.9s | ETA: 4.1s
  Batch 36/47 (76.6%) | Loss: 0.4778 | Acc: 0.812 | Elapsed: 12.2s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.4762 | Acc: 0.812 | Elapsed: 12.5s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.4784 | Acc: 0.812 | Elapsed: 12.9s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.4808 | Acc: 0.812 | Elapsed: 13.2s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.4831 | Acc: 0.812 | Elapsed: 13.5s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.4786 | Acc: 0.814 | Elapsed: 13.8s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.4841 | Acc: 0.810 | Elapsed: 14.2s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.4808 | Acc: 0.810 | Elapsed: 14.5s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.4820 | Acc: 0.809 | Elapsed: 14.8s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.4832 | Acc: 0.807 | Elapsed: 15.1s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.4875 | Acc: 0.806 | Elapsed: 15.4s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.4953 | Acc: 0.805 | Elapsed: 15.5s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.8421 | Acc: 0.517
Epoch 30 completed in 17.2s

--- Epoch 31/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.4925 | Acc: 0.844 | Elapsed: 0.8s | ETA: 36.2s
  Batch 2/47 (4.3%) | Loss: 0.4978 | Acc: 0.844 | Elapsed: 1.1s | ETA: 24.8s
  Batch 3/47 (6.4%) | Loss: 0.5396 | Acc: 0.802 | Elapsed: 1.4s | ETA: 21.2s
  Batch 4/47 (8.5%) | Loss: 0.5944 | Acc: 0.789 | Elapsed: 1.8s | ETA: 18.9s
  Batch 5/47 (10.6%) | Loss: 0.5462 | Acc: 0.800 | Elapsed: 2.1s | ETA: 17.4s
  Batch 6/47 (12.8%) | Loss: 0.5485 | Acc: 0.792 | Elapsed: 2.4s | ETA: 16.4s
  Batch 7/47 (14.9%) | Loss: 0.5382 | Acc: 0.790 | Elapsed: 2.7s | ETA: 15.6s
  Batch 8/47 (17.0%) | Loss: 0.5142 | Acc: 0.805 | Elapsed: 3.0s | ETA: 14.8s
  Batch 9/47 (19.1%) | Loss: 0.5356 | Acc: 0.785 | Elapsed: 3.4s | ETA: 14.2s
  Batch 10/47 (21.3%) | Loss: 0.5302 | Acc: 0.784 | Elapsed: 3.7s | ETA: 13.7s
  Batch 11/47 (23.4%) | Loss: 0.5485 | Acc: 0.776 | Elapsed: 4.0s | ETA: 13.1s
  Batch 12/47 (25.5%) | Loss: 0.5396 | Acc: 0.779 | Elapsed: 4.3s | ETA: 12.6s
  Batch 13/47 (27.7%) | Loss: 0.5496 | Acc: 0.774 | Elapsed: 4.7s | ETA: 12.2s
  Batch 14/47 (29.8%) | Loss: 0.5442 | Acc: 0.775 | Elapsed: 5.0s | ETA: 11.7s
  Batch 15/47 (31.9%) | Loss: 0.5342 | Acc: 0.777 | Elapsed: 5.3s | ETA: 11.3s
  Batch 16/47 (34.0%) | Loss: 0.5254 | Acc: 0.783 | Elapsed: 5.6s | ETA: 10.9s
  Batch 17/47 (36.2%) | Loss: 0.5295 | Acc: 0.781 | Elapsed: 5.9s | ETA: 10.5s
  Batch 18/47 (38.3%) | Loss: 0.5305 | Acc: 0.783 | Elapsed: 6.3s | ETA: 10.1s
  Batch 19/47 (40.4%) | Loss: 0.5405 | Acc: 0.778 | Elapsed: 6.6s | ETA: 9.7s
  Batch 20/47 (42.6%) | Loss: 0.5471 | Acc: 0.775 | Elapsed: 6.9s | ETA: 9.3s
  Batch 21/47 (44.7%) | Loss: 0.5464 | Acc: 0.774 | Elapsed: 7.2s | ETA: 9.0s
  Batch 22/47 (46.8%) | Loss: 0.5308 | Acc: 0.783 | Elapsed: 7.6s | ETA: 8.6s
  Batch 23/47 (48.9%) | Loss: 0.5393 | Acc: 0.779 | Elapsed: 7.9s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 0.5436 | Acc: 0.776 | Elapsed: 8.2s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 0.5405 | Acc: 0.779 | Elapsed: 8.5s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.5424 | Acc: 0.776 | Elapsed: 8.9s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 0.5409 | Acc: 0.777 | Elapsed: 9.2s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.5395 | Acc: 0.776 | Elapsed: 9.5s | ETA: 6.5s
  Batch 29/47 (61.7%) | Loss: 0.5337 | Acc: 0.779 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.5291 | Acc: 0.782 | Elapsed: 10.2s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.5298 | Acc: 0.782 | Elapsed: 10.5s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.5320 | Acc: 0.781 | Elapsed: 10.8s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.5299 | Acc: 0.781 | Elapsed: 11.1s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.5299 | Acc: 0.780 | Elapsed: 11.4s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.5268 | Acc: 0.782 | Elapsed: 11.8s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 0.5275 | Acc: 0.782 | Elapsed: 12.1s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.5334 | Acc: 0.778 | Elapsed: 12.4s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.5299 | Acc: 0.777 | Elapsed: 12.7s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.5257 | Acc: 0.780 | Elapsed: 13.0s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.5277 | Acc: 0.779 | Elapsed: 13.4s | ETA: 2.3s
  Batch 41/47 (87.2%) | Loss: 0.5237 | Acc: 0.780 | Elapsed: 13.7s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.5251 | Acc: 0.781 | Elapsed: 14.0s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.5241 | Acc: 0.784 | Elapsed: 14.3s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.5319 | Acc: 0.782 | Elapsed: 14.7s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.5332 | Acc: 0.781 | Elapsed: 15.0s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.5319 | Acc: 0.781 | Elapsed: 15.3s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.5342 | Acc: 0.780 | Elapsed: 15.3s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.3531 | Acc: 0.548
Epoch 31 completed in 17.1s

--- Epoch 32/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.5682 | Acc: 0.781 | Elapsed: 0.8s | ETA: 37.4s
  Batch 2/47 (4.3%) | Loss: 0.5895 | Acc: 0.781 | Elapsed: 1.1s | ETA: 25.4s
  Batch 3/47 (6.4%) | Loss: 0.6424 | Acc: 0.750 | Elapsed: 1.5s | ETA: 21.6s
  Batch 4/47 (8.5%) | Loss: 0.6195 | Acc: 0.766 | Elapsed: 1.8s | ETA: 19.2s
  Batch 5/47 (10.6%) | Loss: 0.5773 | Acc: 0.775 | Elapsed: 2.1s | ETA: 17.6s
  Batch 6/47 (12.8%) | Loss: 0.6326 | Acc: 0.755 | Elapsed: 2.4s | ETA: 16.5s
  Batch 7/47 (14.9%) | Loss: 0.6046 | Acc: 0.763 | Elapsed: 2.7s | ETA: 15.7s
  Batch 8/47 (17.0%) | Loss: 0.6083 | Acc: 0.758 | Elapsed: 3.1s | ETA: 14.9s
  Batch 9/47 (19.1%) | Loss: 0.5741 | Acc: 0.774 | Elapsed: 3.4s | ETA: 14.3s
  Batch 10/47 (21.3%) | Loss: 0.5709 | Acc: 0.775 | Elapsed: 3.7s | ETA: 13.8s
  Batch 11/47 (23.4%) | Loss: 0.5591 | Acc: 0.778 | Elapsed: 4.0s | ETA: 13.2s
  Batch 12/47 (25.5%) | Loss: 0.5643 | Acc: 0.779 | Elapsed: 4.4s | ETA: 12.7s
  Batch 13/47 (27.7%) | Loss: 0.5694 | Acc: 0.774 | Elapsed: 4.7s | ETA: 12.3s
  Batch 14/47 (29.8%) | Loss: 0.5776 | Acc: 0.772 | Elapsed: 5.0s | ETA: 11.8s
  Batch 15/47 (31.9%) | Loss: 0.5907 | Acc: 0.762 | Elapsed: 5.3s | ETA: 11.4s
  Batch 16/47 (34.0%) | Loss: 0.5864 | Acc: 0.764 | Elapsed: 5.7s | ETA: 11.0s
  Batch 17/47 (36.2%) | Loss: 0.5788 | Acc: 0.767 | Elapsed: 6.0s | ETA: 10.6s
  Batch 18/47 (38.3%) | Loss: 0.5692 | Acc: 0.771 | Elapsed: 6.3s | ETA: 10.2s
  Batch 19/47 (40.4%) | Loss: 0.5855 | Acc: 0.770 | Elapsed: 6.6s | ETA: 9.8s
  Batch 20/47 (42.6%) | Loss: 0.5886 | Acc: 0.764 | Elapsed: 7.0s | ETA: 9.4s
  Batch 21/47 (44.7%) | Loss: 0.6006 | Acc: 0.757 | Elapsed: 7.3s | ETA: 9.0s
  Batch 22/47 (46.8%) | Loss: 0.5922 | Acc: 0.763 | Elapsed: 7.6s | ETA: 8.6s
  Batch 23/47 (48.9%) | Loss: 0.5880 | Acc: 0.768 | Elapsed: 7.9s | ETA: 8.3s
  Batch 24/47 (51.1%) | Loss: 0.5826 | Acc: 0.768 | Elapsed: 8.2s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 0.5968 | Acc: 0.761 | Elapsed: 8.6s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.5932 | Acc: 0.761 | Elapsed: 8.9s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 0.6044 | Acc: 0.757 | Elapsed: 9.2s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.6078 | Acc: 0.753 | Elapsed: 9.5s | ETA: 6.5s
  Batch 29/47 (61.7%) | Loss: 0.6010 | Acc: 0.755 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.5976 | Acc: 0.757 | Elapsed: 10.2s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.5903 | Acc: 0.760 | Elapsed: 10.5s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.5906 | Acc: 0.761 | Elapsed: 10.8s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.5849 | Acc: 0.763 | Elapsed: 11.1s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.5847 | Acc: 0.764 | Elapsed: 11.5s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.5859 | Acc: 0.764 | Elapsed: 11.8s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 0.5889 | Acc: 0.764 | Elapsed: 12.1s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.5831 | Acc: 0.765 | Elapsed: 12.4s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.5817 | Acc: 0.766 | Elapsed: 12.7s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.5768 | Acc: 0.767 | Elapsed: 13.1s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.5779 | Acc: 0.768 | Elapsed: 13.4s | ETA: 2.3s
  Batch 41/47 (87.2%) | Loss: 0.5780 | Acc: 0.768 | Elapsed: 13.7s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.5763 | Acc: 0.769 | Elapsed: 14.0s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.5736 | Acc: 0.769 | Elapsed: 14.4s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.5743 | Acc: 0.769 | Elapsed: 14.7s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.5735 | Acc: 0.769 | Elapsed: 15.0s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.5780 | Acc: 0.766 | Elapsed: 15.3s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.5781 | Acc: 0.765 | Elapsed: 15.4s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.1532 | Acc: 0.606
Epoch 32 completed in 23.8s

--- Epoch 33/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.7918 | Acc: 0.719 | Elapsed: 0.9s | ETA: 39.8s
  Batch 2/47 (4.3%) | Loss: 0.7355 | Acc: 0.734 | Elapsed: 1.2s | ETA: 26.4s
  Batch 3/47 (6.4%) | Loss: 0.6348 | Acc: 0.771 | Elapsed: 1.5s | ETA: 21.7s
  Batch 4/47 (8.5%) | Loss: 0.6277 | Acc: 0.750 | Elapsed: 1.8s | ETA: 19.5s
  Batch 5/47 (10.6%) | Loss: 0.5829 | Acc: 0.775 | Elapsed: 2.1s | ETA: 17.8s
  Batch 6/47 (12.8%) | Loss: 0.5291 | Acc: 0.797 | Elapsed: 2.4s | ETA: 16.6s
  Batch 7/47 (14.9%) | Loss: 0.4997 | Acc: 0.808 | Elapsed: 2.8s | ETA: 15.7s
  Batch 8/47 (17.0%) | Loss: 0.4896 | Acc: 0.820 | Elapsed: 3.1s | ETA: 15.0s
  Batch 9/47 (19.1%) | Loss: 0.5139 | Acc: 0.806 | Elapsed: 3.4s | ETA: 14.3s
  Batch 10/47 (21.3%) | Loss: 0.5139 | Acc: 0.800 | Elapsed: 3.7s | ETA: 13.7s
  Batch 11/47 (23.4%) | Loss: 0.5121 | Acc: 0.801 | Elapsed: 4.0s | ETA: 13.1s
  Batch 12/47 (25.5%) | Loss: 0.5056 | Acc: 0.802 | Elapsed: 4.3s | ETA: 12.6s
  Batch 13/47 (27.7%) | Loss: 0.4972 | Acc: 0.800 | Elapsed: 4.6s | ETA: 12.1s
  Batch 14/47 (29.8%) | Loss: 0.5116 | Acc: 0.806 | Elapsed: 5.0s | ETA: 11.7s
  Batch 15/47 (31.9%) | Loss: 0.5075 | Acc: 0.808 | Elapsed: 5.3s | ETA: 11.3s
  Batch 16/47 (34.0%) | Loss: 0.4988 | Acc: 0.811 | Elapsed: 5.6s | ETA: 10.8s
  Batch 17/47 (36.2%) | Loss: 0.5005 | Acc: 0.811 | Elapsed: 5.9s | ETA: 10.4s
  Batch 18/47 (38.3%) | Loss: 0.5114 | Acc: 0.804 | Elapsed: 6.3s | ETA: 10.1s
  Batch 19/47 (40.4%) | Loss: 0.5030 | Acc: 0.808 | Elapsed: 6.6s | ETA: 9.7s
  Batch 20/47 (42.6%) | Loss: 0.4909 | Acc: 0.814 | Elapsed: 6.9s | ETA: 9.3s
  Batch 21/47 (44.7%) | Loss: 0.4992 | Acc: 0.811 | Elapsed: 7.2s | ETA: 8.9s
  Batch 22/47 (46.8%) | Loss: 0.5104 | Acc: 0.801 | Elapsed: 12.8s | ETA: 14.6s
  Batch 23/47 (48.9%) | Loss: 0.5195 | Acc: 0.796 | Elapsed: 13.1s | ETA: 13.7s
  Batch 24/47 (51.1%) | Loss: 0.5147 | Acc: 0.802 | Elapsed: 13.4s | ETA: 12.9s
  Batch 25/47 (53.2%) | Loss: 0.5191 | Acc: 0.801 | Elapsed: 13.8s | ETA: 12.1s
  Batch 26/47 (55.3%) | Loss: 0.5134 | Acc: 0.802 | Elapsed: 14.1s | ETA: 11.4s
  Batch 27/47 (57.4%) | Loss: 0.5136 | Acc: 0.800 | Elapsed: 14.4s | ETA: 10.7s
  Batch 28/47 (59.6%) | Loss: 0.5050 | Acc: 0.802 | Elapsed: 14.7s | ETA: 10.0s
  Batch 29/47 (61.7%) | Loss: 0.5042 | Acc: 0.804 | Elapsed: 15.0s | ETA: 9.3s
  Batch 30/47 (63.8%) | Loss: 0.5058 | Acc: 0.803 | Elapsed: 15.4s | ETA: 8.7s
  Batch 31/47 (66.0%) | Loss: 0.5050 | Acc: 0.804 | Elapsed: 15.7s | ETA: 8.1s
  Batch 32/47 (68.1%) | Loss: 0.5052 | Acc: 0.804 | Elapsed: 16.0s | ETA: 7.5s
  Batch 33/47 (70.2%) | Loss: 0.5048 | Acc: 0.800 | Elapsed: 16.3s | ETA: 6.9s
  Batch 34/47 (72.3%) | Loss: 0.5102 | Acc: 0.799 | Elapsed: 16.6s | ETA: 6.4s
  Batch 35/47 (74.5%) | Loss: 0.5110 | Acc: 0.802 | Elapsed: 17.0s | ETA: 5.8s
  Batch 36/47 (76.6%) | Loss: 0.5056 | Acc: 0.804 | Elapsed: 17.3s | ETA: 5.3s
  Batch 37/47 (78.7%) | Loss: 0.5018 | Acc: 0.806 | Elapsed: 17.6s | ETA: 4.8s
  Batch 38/47 (80.9%) | Loss: 0.4979 | Acc: 0.808 | Elapsed: 17.9s | ETA: 4.2s
  Batch 39/47 (83.0%) | Loss: 0.5046 | Acc: 0.801 | Elapsed: 18.2s | ETA: 3.7s
  Batch 40/47 (85.1%) | Loss: 0.5023 | Acc: 0.802 | Elapsed: 18.6s | ETA: 3.2s
  Batch 41/47 (87.2%) | Loss: 0.4988 | Acc: 0.805 | Elapsed: 18.9s | ETA: 2.8s
  Batch 42/47 (89.4%) | Loss: 0.5025 | Acc: 0.804 | Elapsed: 19.2s | ETA: 2.3s
  Batch 43/47 (91.5%) | Loss: 0.5057 | Acc: 0.802 | Elapsed: 19.5s | ETA: 1.8s
  Batch 44/47 (93.6%) | Loss: 0.5095 | Acc: 0.800 | Elapsed: 19.9s | ETA: 1.4s
  Batch 45/47 (95.7%) | Loss: 0.5099 | Acc: 0.799 | Elapsed: 20.2s | ETA: 0.9s
  Batch 46/47 (97.9%) | Loss: 0.5112 | Acc: 0.798 | Elapsed: 20.5s | ETA: 0.4s
  Batch 47/47 (100.0%) | Loss: 0.5121 | Acc: 0.797 | Elapsed: 20.6s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.5623 | Acc: 0.534
Epoch 33 completed in 22.7s

--- Epoch 34/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.6565 | Acc: 0.781 | Elapsed: 0.8s | ETA: 35.1s
  Batch 2/47 (4.3%) | Loss: 0.6099 | Acc: 0.766 | Elapsed: 1.1s | ETA: 24.3s
  Batch 3/47 (6.4%) | Loss: 0.5477 | Acc: 0.792 | Elapsed: 1.4s | ETA: 21.1s
  Batch 4/47 (8.5%) | Loss: 0.5662 | Acc: 0.797 | Elapsed: 1.8s | ETA: 18.8s
  Batch 5/47 (10.6%) | Loss: 0.5391 | Acc: 0.800 | Elapsed: 2.1s | ETA: 17.4s
  Batch 6/47 (12.8%) | Loss: 0.5742 | Acc: 0.792 | Elapsed: 2.4s | ETA: 16.5s
  Batch 7/47 (14.9%) | Loss: 0.5678 | Acc: 0.786 | Elapsed: 2.7s | ETA: 15.6s
  Batch 8/47 (17.0%) | Loss: 0.5962 | Acc: 0.766 | Elapsed: 3.1s | ETA: 14.9s
  Batch 9/47 (19.1%) | Loss: 0.5918 | Acc: 0.760 | Elapsed: 3.4s | ETA: 14.3s
  Batch 10/47 (21.3%) | Loss: 0.5973 | Acc: 0.753 | Elapsed: 3.7s | ETA: 13.8s
  Batch 11/47 (23.4%) | Loss: 0.5891 | Acc: 0.761 | Elapsed: 4.0s | ETA: 13.2s
  Batch 12/47 (25.5%) | Loss: 0.5683 | Acc: 0.768 | Elapsed: 4.4s | ETA: 12.8s
  Batch 13/47 (27.7%) | Loss: 0.5700 | Acc: 0.767 | Elapsed: 4.7s | ETA: 12.3s
  Batch 14/47 (29.8%) | Loss: 0.5626 | Acc: 0.770 | Elapsed: 5.0s | ETA: 11.9s
  Batch 15/47 (31.9%) | Loss: 0.5463 | Acc: 0.779 | Elapsed: 5.4s | ETA: 11.4s
  Batch 16/47 (34.0%) | Loss: 0.5443 | Acc: 0.775 | Elapsed: 5.7s | ETA: 11.0s
  Batch 17/47 (36.2%) | Loss: 0.5578 | Acc: 0.772 | Elapsed: 6.0s | ETA: 10.6s
  Batch 18/47 (38.3%) | Loss: 0.5445 | Acc: 0.780 | Elapsed: 6.3s | ETA: 10.2s
  Batch 19/47 (40.4%) | Loss: 0.5503 | Acc: 0.780 | Elapsed: 6.7s | ETA: 9.8s
  Batch 20/47 (42.6%) | Loss: 0.5505 | Acc: 0.778 | Elapsed: 7.0s | ETA: 9.5s
  Batch 21/47 (44.7%) | Loss: 0.5380 | Acc: 0.783 | Elapsed: 7.3s | ETA: 9.1s
  Batch 22/47 (46.8%) | Loss: 0.5300 | Acc: 0.786 | Elapsed: 7.7s | ETA: 8.7s
  Batch 23/47 (48.9%) | Loss: 0.5169 | Acc: 0.792 | Elapsed: 8.0s | ETA: 8.4s
  Batch 24/47 (51.1%) | Loss: 0.5101 | Acc: 0.798 | Elapsed: 8.4s | ETA: 8.0s
  Batch 25/47 (53.2%) | Loss: 0.5075 | Acc: 0.800 | Elapsed: 8.7s | ETA: 7.6s
  Batch 26/47 (55.3%) | Loss: 0.5049 | Acc: 0.799 | Elapsed: 9.0s | ETA: 7.3s
  Batch 27/47 (57.4%) | Loss: 0.5009 | Acc: 0.801 | Elapsed: 9.4s | ETA: 6.9s
  Batch 28/47 (59.6%) | Loss: 0.4998 | Acc: 0.802 | Elapsed: 9.7s | ETA: 6.6s
  Batch 29/47 (61.7%) | Loss: 0.5066 | Acc: 0.801 | Elapsed: 10.0s | ETA: 6.2s
  Batch 30/47 (63.8%) | Loss: 0.5035 | Acc: 0.801 | Elapsed: 10.4s | ETA: 5.9s
  Batch 31/47 (66.0%) | Loss: 0.5011 | Acc: 0.800 | Elapsed: 10.7s | ETA: 5.5s
  Batch 32/47 (68.1%) | Loss: 0.4953 | Acc: 0.805 | Elapsed: 11.0s | ETA: 5.2s
  Batch 33/47 (70.2%) | Loss: 0.4997 | Acc: 0.802 | Elapsed: 11.4s | ETA: 4.8s
  Batch 34/47 (72.3%) | Loss: 0.4985 | Acc: 0.802 | Elapsed: 11.7s | ETA: 4.5s
  Batch 35/47 (74.5%) | Loss: 0.4940 | Acc: 0.804 | Elapsed: 12.1s | ETA: 4.1s
  Batch 36/47 (76.6%) | Loss: 0.4914 | Acc: 0.806 | Elapsed: 12.4s | ETA: 3.8s
  Batch 37/47 (78.7%) | Loss: 0.4909 | Acc: 0.803 | Elapsed: 12.7s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.5042 | Acc: 0.798 | Elapsed: 13.1s | ETA: 3.1s
  Batch 39/47 (83.0%) | Loss: 0.5136 | Acc: 0.794 | Elapsed: 13.4s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.5220 | Acc: 0.791 | Elapsed: 13.7s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.5217 | Acc: 0.791 | Elapsed: 14.1s | ETA: 2.1s
  Batch 42/47 (89.4%) | Loss: 0.5219 | Acc: 0.791 | Elapsed: 14.4s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.5188 | Acc: 0.791 | Elapsed: 14.7s | ETA: 1.4s
  Batch 44/47 (93.6%) | Loss: 0.5160 | Acc: 0.790 | Elapsed: 15.1s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.5151 | Acc: 0.790 | Elapsed: 15.4s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.5175 | Acc: 0.789 | Elapsed: 15.7s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.5208 | Acc: 0.788 | Elapsed: 15.8s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 2.7119 | Acc: 0.446
Epoch 34 completed in 17.5s

--- Epoch 35/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.8448 | Acc: 0.688 | Elapsed: 0.8s | ETA: 36.4s
  Batch 2/47 (4.3%) | Loss: 0.7346 | Acc: 0.719 | Elapsed: 1.1s | ETA: 25.0s
  Batch 3/47 (6.4%) | Loss: 0.8130 | Acc: 0.688 | Elapsed: 1.5s | ETA: 21.6s
  Batch 4/47 (8.5%) | Loss: 0.7741 | Acc: 0.695 | Elapsed: 1.8s | ETA: 19.3s
  Batch 5/47 (10.6%) | Loss: 0.7374 | Acc: 0.681 | Elapsed: 2.1s | ETA: 17.8s
  Batch 6/47 (12.8%) | Loss: 0.7080 | Acc: 0.703 | Elapsed: 2.5s | ETA: 16.9s
  Batch 7/47 (14.9%) | Loss: 0.6872 | Acc: 0.714 | Elapsed: 2.8s | ETA: 15.9s
  Batch 8/47 (17.0%) | Loss: 0.6529 | Acc: 0.734 | Elapsed: 3.1s | ETA: 15.2s
  Batch 9/47 (19.1%) | Loss: 0.6672 | Acc: 0.733 | Elapsed: 3.4s | ETA: 14.6s
  Batch 10/47 (21.3%) | Loss: 0.6576 | Acc: 0.731 | Elapsed: 3.8s | ETA: 14.0s
  Batch 11/47 (23.4%) | Loss: 0.6454 | Acc: 0.736 | Elapsed: 4.1s | ETA: 13.4s
  Batch 12/47 (25.5%) | Loss: 0.6186 | Acc: 0.745 | Elapsed: 4.4s | ETA: 12.9s
  Batch 13/47 (27.7%) | Loss: 0.6105 | Acc: 0.748 | Elapsed: 4.8s | ETA: 12.4s
  Batch 14/47 (29.8%) | Loss: 0.6014 | Acc: 0.757 | Elapsed: 5.1s | ETA: 12.0s
  Batch 15/47 (31.9%) | Loss: 0.5930 | Acc: 0.760 | Elapsed: 5.4s | ETA: 11.6s
  Batch 16/47 (34.0%) | Loss: 0.5742 | Acc: 0.768 | Elapsed: 5.8s | ETA: 11.2s
  Batch 17/47 (36.2%) | Loss: 0.5661 | Acc: 0.767 | Elapsed: 6.1s | ETA: 10.7s
  Batch 18/47 (38.3%) | Loss: 0.5527 | Acc: 0.778 | Elapsed: 6.4s | ETA: 10.3s
  Batch 19/47 (40.4%) | Loss: 0.5546 | Acc: 0.778 | Elapsed: 6.7s | ETA: 9.9s
  Batch 20/47 (42.6%) | Loss: 0.5433 | Acc: 0.783 | Elapsed: 7.1s | ETA: 9.6s
  Batch 21/47 (44.7%) | Loss: 0.5484 | Acc: 0.780 | Elapsed: 7.4s | ETA: 9.2s
  Batch 22/47 (46.8%) | Loss: 0.5477 | Acc: 0.781 | Elapsed: 7.8s | ETA: 8.8s
  Batch 23/47 (48.9%) | Loss: 0.5432 | Acc: 0.785 | Elapsed: 8.1s | ETA: 8.4s
  Batch 24/47 (51.1%) | Loss: 0.5379 | Acc: 0.786 | Elapsed: 8.4s | ETA: 8.1s
  Batch 25/47 (53.2%) | Loss: 0.5402 | Acc: 0.786 | Elapsed: 8.7s | ETA: 7.7s
  Batch 26/47 (55.3%) | Loss: 0.5363 | Acc: 0.786 | Elapsed: 9.1s | ETA: 7.3s
  Batch 27/47 (57.4%) | Loss: 0.5379 | Acc: 0.782 | Elapsed: 9.4s | ETA: 7.0s
  Batch 28/47 (59.6%) | Loss: 0.5435 | Acc: 0.780 | Elapsed: 9.7s | ETA: 6.6s
  Batch 29/47 (61.7%) | Loss: 0.5404 | Acc: 0.781 | Elapsed: 10.0s | ETA: 6.2s
  Batch 30/47 (63.8%) | Loss: 0.5391 | Acc: 0.779 | Elapsed: 10.4s | ETA: 5.9s
  Batch 31/47 (66.0%) | Loss: 0.5348 | Acc: 0.781 | Elapsed: 10.7s | ETA: 5.5s
  Batch 32/47 (68.1%) | Loss: 0.5304 | Acc: 0.784 | Elapsed: 11.0s | ETA: 5.2s
  Batch 33/47 (70.2%) | Loss: 0.5313 | Acc: 0.784 | Elapsed: 11.3s | ETA: 4.8s
  Batch 34/47 (72.3%) | Loss: 0.5285 | Acc: 0.784 | Elapsed: 11.6s | ETA: 4.5s
  Batch 35/47 (74.5%) | Loss: 0.5267 | Acc: 0.787 | Elapsed: 12.0s | ETA: 4.1s
  Batch 36/47 (76.6%) | Loss: 0.5201 | Acc: 0.788 | Elapsed: 12.3s | ETA: 3.8s
  Batch 37/47 (78.7%) | Loss: 0.5152 | Acc: 0.790 | Elapsed: 12.6s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.5084 | Acc: 0.793 | Elapsed: 12.9s | ETA: 3.1s
  Batch 39/47 (83.0%) | Loss: 0.5067 | Acc: 0.792 | Elapsed: 13.3s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.5048 | Acc: 0.794 | Elapsed: 13.6s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.4992 | Acc: 0.796 | Elapsed: 13.9s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.5020 | Acc: 0.795 | Elapsed: 14.2s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.4983 | Acc: 0.797 | Elapsed: 14.5s | ETA: 1.4s
  Batch 44/47 (93.6%) | Loss: 0.4968 | Acc: 0.798 | Elapsed: 14.9s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.4998 | Acc: 0.796 | Elapsed: 15.2s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.5013 | Acc: 0.796 | Elapsed: 15.5s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.5010 | Acc: 0.796 | Elapsed: 15.6s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.3697 | Acc: 0.600
Epoch 35 completed in 17.3s

--- Epoch 36/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.2642 | Acc: 0.906 | Elapsed: 0.8s | ETA: 36.5s
  Batch 2/47 (4.3%) | Loss: 0.3661 | Acc: 0.891 | Elapsed: 1.1s | ETA: 25.0s
  Batch 3/47 (6.4%) | Loss: 0.4316 | Acc: 0.854 | Elapsed: 1.5s | ETA: 21.4s
  Batch 4/47 (8.5%) | Loss: 0.3928 | Acc: 0.867 | Elapsed: 1.8s | ETA: 19.2s
  Batch 5/47 (10.6%) | Loss: 0.4855 | Acc: 0.819 | Elapsed: 2.1s | ETA: 17.8s
  Batch 6/47 (12.8%) | Loss: 0.4947 | Acc: 0.812 | Elapsed: 2.4s | ETA: 16.7s
  Batch 7/47 (14.9%) | Loss: 0.4680 | Acc: 0.817 | Elapsed: 2.8s | ETA: 15.9s
  Batch 8/47 (17.0%) | Loss: 0.4917 | Acc: 0.812 | Elapsed: 3.1s | ETA: 15.1s
  Batch 9/47 (19.1%) | Loss: 0.4954 | Acc: 0.819 | Elapsed: 3.4s | ETA: 14.5s
  Batch 10/47 (21.3%) | Loss: 0.4807 | Acc: 0.825 | Elapsed: 3.8s | ETA: 13.9s
  Batch 11/47 (23.4%) | Loss: 0.4904 | Acc: 0.821 | Elapsed: 4.1s | ETA: 13.3s
  Batch 12/47 (25.5%) | Loss: 0.4839 | Acc: 0.823 | Elapsed: 4.4s | ETA: 12.8s
  Batch 13/47 (27.7%) | Loss: 0.4835 | Acc: 0.822 | Elapsed: 4.7s | ETA: 12.3s
  Batch 14/47 (29.8%) | Loss: 0.4718 | Acc: 0.826 | Elapsed: 5.0s | ETA: 11.9s
  Batch 15/47 (31.9%) | Loss: 0.4872 | Acc: 0.825 | Elapsed: 5.4s | ETA: 11.4s
  Batch 16/47 (34.0%) | Loss: 0.4887 | Acc: 0.822 | Elapsed: 5.7s | ETA: 11.0s
  Batch 17/47 (36.2%) | Loss: 0.4936 | Acc: 0.822 | Elapsed: 6.0s | ETA: 10.6s
  Batch 18/47 (38.3%) | Loss: 0.4867 | Acc: 0.825 | Elapsed: 6.3s | ETA: 10.2s
  Batch 19/47 (40.4%) | Loss: 0.4971 | Acc: 0.817 | Elapsed: 6.6s | ETA: 9.8s
  Batch 20/47 (42.6%) | Loss: 0.4869 | Acc: 0.822 | Elapsed: 7.0s | ETA: 9.4s
  Batch 21/47 (44.7%) | Loss: 0.4795 | Acc: 0.826 | Elapsed: 7.3s | ETA: 9.0s
  Batch 22/47 (46.8%) | Loss: 0.4937 | Acc: 0.818 | Elapsed: 7.6s | ETA: 8.6s
  Batch 23/47 (48.9%) | Loss: 0.4948 | Acc: 0.819 | Elapsed: 7.9s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 0.4914 | Acc: 0.820 | Elapsed: 8.2s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 0.4826 | Acc: 0.823 | Elapsed: 8.5s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.4884 | Acc: 0.822 | Elapsed: 8.8s | ETA: 7.1s
  Batch 27/47 (57.4%) | Loss: 0.4820 | Acc: 0.823 | Elapsed: 9.2s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.4798 | Acc: 0.825 | Elapsed: 9.5s | ETA: 6.4s
  Batch 29/47 (61.7%) | Loss: 0.4794 | Acc: 0.824 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.4756 | Acc: 0.825 | Elapsed: 10.1s | ETA: 5.7s
  Batch 31/47 (66.0%) | Loss: 0.4777 | Acc: 0.823 | Elapsed: 10.4s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.4720 | Acc: 0.825 | Elapsed: 10.7s | ETA: 5.0s
  Batch 33/47 (70.2%) | Loss: 0.4733 | Acc: 0.824 | Elapsed: 11.1s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.4677 | Acc: 0.827 | Elapsed: 11.4s | ETA: 4.3s
  Batch 35/47 (74.5%) | Loss: 0.4670 | Acc: 0.825 | Elapsed: 11.7s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 0.4680 | Acc: 0.825 | Elapsed: 12.0s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.4703 | Acc: 0.823 | Elapsed: 12.3s | ETA: 3.3s
  Batch 38/47 (80.9%) | Loss: 0.4714 | Acc: 0.821 | Elapsed: 12.6s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.4667 | Acc: 0.821 | Elapsed: 13.0s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.4723 | Acc: 0.820 | Elapsed: 13.3s | ETA: 2.3s
  Batch 41/47 (87.2%) | Loss: 0.4712 | Acc: 0.820 | Elapsed: 13.6s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.4712 | Acc: 0.820 | Elapsed: 13.9s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.4680 | Acc: 0.821 | Elapsed: 14.2s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.4649 | Acc: 0.824 | Elapsed: 14.6s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.4681 | Acc: 0.822 | Elapsed: 14.9s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.4733 | Acc: 0.819 | Elapsed: 15.2s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.4739 | Acc: 0.819 | Elapsed: 15.2s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.2328 | Acc: 0.608
Epoch 36 completed in 18.6s

--- Epoch 37/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.6676 | Acc: 0.719 | Elapsed: 1.1s | ETA: 50.2s
  Batch 2/47 (4.3%) | Loss: 0.5433 | Acc: 0.797 | Elapsed: 1.4s | ETA: 31.5s
  Batch 3/47 (6.4%) | Loss: 0.4608 | Acc: 0.833 | Elapsed: 1.7s | ETA: 25.1s
  Batch 4/47 (8.5%) | Loss: 0.4377 | Acc: 0.844 | Elapsed: 2.0s | ETA: 22.0s
  Batch 5/47 (10.6%) | Loss: 0.4155 | Acc: 0.844 | Elapsed: 2.4s | ETA: 19.9s
  Batch 6/47 (12.8%) | Loss: 0.4084 | Acc: 0.849 | Elapsed: 2.7s | ETA: 18.4s
  Batch 7/47 (14.9%) | Loss: 0.3900 | Acc: 0.853 | Elapsed: 3.0s | ETA: 17.3s
  Batch 8/47 (17.0%) | Loss: 0.4092 | Acc: 0.844 | Elapsed: 3.3s | ETA: 16.3s
  Batch 9/47 (19.1%) | Loss: 0.4048 | Acc: 0.847 | Elapsed: 3.7s | ETA: 15.4s
  Batch 10/47 (21.3%) | Loss: 0.4194 | Acc: 0.838 | Elapsed: 4.0s | ETA: 14.7s
  Batch 11/47 (23.4%) | Loss: 0.4229 | Acc: 0.838 | Elapsed: 4.3s | ETA: 14.0s
  Batch 12/47 (25.5%) | Loss: 0.4147 | Acc: 0.836 | Elapsed: 4.6s | ETA: 13.4s
  Batch 13/47 (27.7%) | Loss: 0.4316 | Acc: 0.825 | Elapsed: 4.9s | ETA: 12.8s
  Batch 14/47 (29.8%) | Loss: 0.4287 | Acc: 0.828 | Elapsed: 5.2s | ETA: 12.3s
  Batch 15/47 (31.9%) | Loss: 0.4566 | Acc: 0.815 | Elapsed: 5.5s | ETA: 11.8s
  Batch 16/47 (34.0%) | Loss: 0.4531 | Acc: 0.816 | Elapsed: 5.9s | ETA: 11.3s
  Batch 17/47 (36.2%) | Loss: 0.4502 | Acc: 0.818 | Elapsed: 6.2s | ETA: 10.9s
  Batch 18/47 (38.3%) | Loss: 0.4507 | Acc: 0.816 | Elapsed: 6.5s | ETA: 10.5s
  Batch 19/47 (40.4%) | Loss: 0.4502 | Acc: 0.817 | Elapsed: 6.8s | ETA: 10.0s
  Batch 20/47 (42.6%) | Loss: 0.4499 | Acc: 0.820 | Elapsed: 7.1s | ETA: 9.6s
  Batch 21/47 (44.7%) | Loss: 0.4551 | Acc: 0.818 | Elapsed: 7.4s | ETA: 9.2s
  Batch 22/47 (46.8%) | Loss: 0.4744 | Acc: 0.808 | Elapsed: 7.8s | ETA: 8.8s
  Batch 23/47 (48.9%) | Loss: 0.4659 | Acc: 0.814 | Elapsed: 8.1s | ETA: 8.4s
  Batch 24/47 (51.1%) | Loss: 0.4566 | Acc: 0.819 | Elapsed: 8.4s | ETA: 8.0s
  Batch 25/47 (53.2%) | Loss: 0.4527 | Acc: 0.820 | Elapsed: 8.7s | ETA: 7.7s
  Batch 26/47 (55.3%) | Loss: 0.4490 | Acc: 0.821 | Elapsed: 9.0s | ETA: 7.3s
  Batch 27/47 (57.4%) | Loss: 0.4491 | Acc: 0.822 | Elapsed: 9.3s | ETA: 6.9s
  Batch 28/47 (59.6%) | Loss: 0.4505 | Acc: 0.821 | Elapsed: 9.6s | ETA: 6.5s
  Batch 29/47 (61.7%) | Loss: 0.4482 | Acc: 0.822 | Elapsed: 10.0s | ETA: 6.2s
  Batch 30/47 (63.8%) | Loss: 0.4434 | Acc: 0.824 | Elapsed: 10.3s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.4445 | Acc: 0.823 | Elapsed: 10.6s | ETA: 5.5s
  Batch 32/47 (68.1%) | Loss: 0.4461 | Acc: 0.822 | Elapsed: 10.9s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.4437 | Acc: 0.822 | Elapsed: 11.3s | ETA: 4.8s
  Batch 34/47 (72.3%) | Loss: 0.4458 | Acc: 0.819 | Elapsed: 11.6s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.4465 | Acc: 0.819 | Elapsed: 11.9s | ETA: 4.1s
  Batch 36/47 (76.6%) | Loss: 0.4429 | Acc: 0.820 | Elapsed: 12.2s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.4468 | Acc: 0.818 | Elapsed: 12.6s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.4432 | Acc: 0.820 | Elapsed: 12.9s | ETA: 3.1s
  Batch 39/47 (83.0%) | Loss: 0.4423 | Acc: 0.821 | Elapsed: 13.2s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.4419 | Acc: 0.822 | Elapsed: 13.5s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.4400 | Acc: 0.823 | Elapsed: 13.9s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.4390 | Acc: 0.824 | Elapsed: 14.2s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.4440 | Acc: 0.820 | Elapsed: 14.5s | ETA: 1.4s
  Batch 44/47 (93.6%) | Loss: 0.4474 | Acc: 0.817 | Elapsed: 14.8s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.4511 | Acc: 0.816 | Elapsed: 15.2s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.4508 | Acc: 0.815 | Elapsed: 15.5s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.4520 | Acc: 0.814 | Elapsed: 15.5s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.4902 | Acc: 0.556
Epoch 37 completed in 19.7s

--- Epoch 38/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.3552 | Acc: 0.844 | Elapsed: 0.8s | ETA: 38.4s
  Batch 2/47 (4.3%) | Loss: 0.4887 | Acc: 0.812 | Elapsed: 1.2s | ETA: 26.0s
  Batch 3/47 (6.4%) | Loss: 0.5529 | Acc: 0.771 | Elapsed: 1.5s | ETA: 22.0s
  Batch 4/47 (8.5%) | Loss: 0.6154 | Acc: 0.758 | Elapsed: 1.8s | ETA: 19.5s
  Batch 5/47 (10.6%) | Loss: 0.5594 | Acc: 0.781 | Elapsed: 2.1s | ETA: 17.8s
  Batch 6/47 (12.8%) | Loss: 0.5301 | Acc: 0.786 | Elapsed: 2.4s | ETA: 16.7s
  Batch 7/47 (14.9%) | Loss: 0.5036 | Acc: 0.795 | Elapsed: 2.8s | ETA: 15.8s
  Batch 8/47 (17.0%) | Loss: 0.5387 | Acc: 0.781 | Elapsed: 3.1s | ETA: 15.0s
  Batch 9/47 (19.1%) | Loss: 0.5297 | Acc: 0.785 | Elapsed: 3.4s | ETA: 14.4s
  Batch 10/47 (21.3%) | Loss: 0.5399 | Acc: 0.778 | Elapsed: 3.7s | ETA: 13.8s
  Batch 11/47 (23.4%) | Loss: 0.5436 | Acc: 0.778 | Elapsed: 4.0s | ETA: 13.2s
  Batch 12/47 (25.5%) | Loss: 0.5386 | Acc: 0.781 | Elapsed: 4.4s | ETA: 12.8s
  Batch 13/47 (27.7%) | Loss: 0.5288 | Acc: 0.784 | Elapsed: 4.7s | ETA: 12.3s
  Batch 14/47 (29.8%) | Loss: 0.5385 | Acc: 0.779 | Elapsed: 5.0s | ETA: 11.9s
  Batch 15/47 (31.9%) | Loss: 0.5332 | Acc: 0.781 | Elapsed: 5.4s | ETA: 11.4s
  Batch 16/47 (34.0%) | Loss: 0.5197 | Acc: 0.789 | Elapsed: 5.7s | ETA: 11.0s
  Batch 17/47 (36.2%) | Loss: 0.5223 | Acc: 0.783 | Elapsed: 6.0s | ETA: 10.6s
  Batch 18/47 (38.3%) | Loss: 0.5405 | Acc: 0.781 | Elapsed: 6.3s | ETA: 10.2s
  Batch 19/47 (40.4%) | Loss: 0.5362 | Acc: 0.780 | Elapsed: 6.7s | ETA: 9.8s
  Batch 20/47 (42.6%) | Loss: 0.5299 | Acc: 0.783 | Elapsed: 7.0s | ETA: 9.4s
  Batch 21/47 (44.7%) | Loss: 0.5168 | Acc: 0.787 | Elapsed: 7.3s | ETA: 9.1s
  Batch 22/47 (46.8%) | Loss: 0.5133 | Acc: 0.786 | Elapsed: 7.6s | ETA: 8.7s
  Batch 23/47 (48.9%) | Loss: 0.5165 | Acc: 0.785 | Elapsed: 8.0s | ETA: 8.3s
  Batch 24/47 (51.1%) | Loss: 0.5070 | Acc: 0.790 | Elapsed: 8.3s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 0.5070 | Acc: 0.789 | Elapsed: 8.6s | ETA: 7.6s
  Batch 26/47 (55.3%) | Loss: 0.5052 | Acc: 0.788 | Elapsed: 8.9s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 0.5039 | Acc: 0.789 | Elapsed: 9.2s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.5016 | Acc: 0.792 | Elapsed: 9.6s | ETA: 6.5s
  Batch 29/47 (61.7%) | Loss: 0.5014 | Acc: 0.795 | Elapsed: 9.9s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.4980 | Acc: 0.797 | Elapsed: 10.2s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.4939 | Acc: 0.800 | Elapsed: 10.5s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.4901 | Acc: 0.803 | Elapsed: 10.9s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.4830 | Acc: 0.807 | Elapsed: 11.2s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.4857 | Acc: 0.804 | Elapsed: 11.5s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.4879 | Acc: 0.804 | Elapsed: 11.8s | ETA: 4.1s
  Batch 36/47 (76.6%) | Loss: 0.4918 | Acc: 0.804 | Elapsed: 12.1s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.4951 | Acc: 0.801 | Elapsed: 12.5s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.4906 | Acc: 0.802 | Elapsed: 12.8s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.5042 | Acc: 0.800 | Elapsed: 13.1s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.5054 | Acc: 0.801 | Elapsed: 13.4s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.5063 | Acc: 0.801 | Elapsed: 13.8s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.5044 | Acc: 0.801 | Elapsed: 14.1s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.5016 | Acc: 0.803 | Elapsed: 14.4s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.4990 | Acc: 0.804 | Elapsed: 14.7s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.4961 | Acc: 0.806 | Elapsed: 15.1s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.4914 | Acc: 0.808 | Elapsed: 15.4s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.4920 | Acc: 0.808 | Elapsed: 15.4s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.2670 | Acc: 0.563
Epoch 38 completed in 17.2s

--- Epoch 39/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.5148 | Acc: 0.781 | Elapsed: 1.1s | ETA: 48.6s
  Batch 2/47 (4.3%) | Loss: 0.4573 | Acc: 0.812 | Elapsed: 1.4s | ETA: 31.1s
  Batch 3/47 (6.4%) | Loss: 0.4829 | Acc: 0.812 | Elapsed: 1.7s | ETA: 25.4s
  Batch 4/47 (8.5%) | Loss: 0.4761 | Acc: 0.812 | Elapsed: 2.1s | ETA: 22.1s
  Batch 5/47 (10.6%) | Loss: 0.5025 | Acc: 0.794 | Elapsed: 2.4s | ETA: 20.0s
  Batch 6/47 (12.8%) | Loss: 0.4973 | Acc: 0.792 | Elapsed: 2.7s | ETA: 18.6s
  Batch 7/47 (14.9%) | Loss: 0.4562 | Acc: 0.812 | Elapsed: 3.1s | ETA: 17.5s
  Batch 8/47 (17.0%) | Loss: 0.4868 | Acc: 0.805 | Elapsed: 3.4s | ETA: 16.5s
  Batch 9/47 (19.1%) | Loss: 0.5432 | Acc: 0.781 | Elapsed: 3.7s | ETA: 15.6s
  Batch 10/47 (21.3%) | Loss: 0.5501 | Acc: 0.784 | Elapsed: 4.0s | ETA: 14.9s
  Batch 11/47 (23.4%) | Loss: 0.5262 | Acc: 0.798 | Elapsed: 4.4s | ETA: 14.2s
  Batch 12/47 (25.5%) | Loss: 0.5235 | Acc: 0.797 | Elapsed: 4.7s | ETA: 13.6s
  Batch 13/47 (27.7%) | Loss: 0.5183 | Acc: 0.793 | Elapsed: 5.0s | ETA: 13.1s
  Batch 14/47 (29.8%) | Loss: 0.5077 | Acc: 0.801 | Elapsed: 5.3s | ETA: 12.6s
  Batch 15/47 (31.9%) | Loss: 0.5164 | Acc: 0.798 | Elapsed: 5.6s | ETA: 12.0s
  Batch 16/47 (34.0%) | Loss: 0.5304 | Acc: 0.791 | Elapsed: 6.0s | ETA: 11.6s
  Batch 17/47 (36.2%) | Loss: 0.5287 | Acc: 0.790 | Elapsed: 6.3s | ETA: 11.1s
  Batch 18/47 (38.3%) | Loss: 0.5201 | Acc: 0.795 | Elapsed: 6.6s | ETA: 10.7s
  Batch 19/47 (40.4%) | Loss: 0.5059 | Acc: 0.799 | Elapsed: 6.9s | ETA: 10.2s
  Batch 20/47 (42.6%) | Loss: 0.5034 | Acc: 0.800 | Elapsed: 7.3s | ETA: 9.8s
  Batch 21/47 (44.7%) | Loss: 0.5052 | Acc: 0.802 | Elapsed: 7.6s | ETA: 9.4s
  Batch 22/47 (46.8%) | Loss: 0.4999 | Acc: 0.804 | Elapsed: 7.9s | ETA: 9.0s
  Batch 23/47 (48.9%) | Loss: 0.4968 | Acc: 0.807 | Elapsed: 8.2s | ETA: 8.6s
  Batch 24/47 (51.1%) | Loss: 0.4985 | Acc: 0.803 | Elapsed: 8.6s | ETA: 8.2s
  Batch 25/47 (53.2%) | Loss: 0.4930 | Acc: 0.806 | Elapsed: 8.9s | ETA: 7.8s
  Batch 26/47 (55.3%) | Loss: 0.4919 | Acc: 0.809 | Elapsed: 9.2s | ETA: 7.4s
  Batch 27/47 (57.4%) | Loss: 0.4873 | Acc: 0.811 | Elapsed: 9.5s | ETA: 7.1s
  Batch 28/47 (59.6%) | Loss: 0.4837 | Acc: 0.812 | Elapsed: 9.9s | ETA: 6.7s
  Batch 29/47 (61.7%) | Loss: 0.4795 | Acc: 0.814 | Elapsed: 10.2s | ETA: 6.3s
  Batch 30/47 (63.8%) | Loss: 0.4850 | Acc: 0.812 | Elapsed: 10.5s | ETA: 6.0s
  Batch 31/47 (66.0%) | Loss: 0.4806 | Acc: 0.815 | Elapsed: 10.8s | ETA: 5.6s
  Batch 32/47 (68.1%) | Loss: 0.4808 | Acc: 0.814 | Elapsed: 11.2s | ETA: 5.2s
  Batch 33/47 (70.2%) | Loss: 0.4792 | Acc: 0.813 | Elapsed: 11.5s | ETA: 4.9s
  Batch 34/47 (72.3%) | Loss: 0.4761 | Acc: 0.813 | Elapsed: 11.8s | ETA: 4.5s
  Batch 35/47 (74.5%) | Loss: 0.4776 | Acc: 0.813 | Elapsed: 12.1s | ETA: 4.2s
  Batch 36/47 (76.6%) | Loss: 0.4810 | Acc: 0.812 | Elapsed: 12.4s | ETA: 3.8s
  Batch 37/47 (78.7%) | Loss: 0.4857 | Acc: 0.811 | Elapsed: 12.8s | ETA: 3.5s
  Batch 38/47 (80.9%) | Loss: 0.4887 | Acc: 0.808 | Elapsed: 13.1s | ETA: 3.1s
  Batch 39/47 (83.0%) | Loss: 0.4819 | Acc: 0.811 | Elapsed: 13.4s | ETA: 2.8s
  Batch 40/47 (85.1%) | Loss: 0.4745 | Acc: 0.814 | Elapsed: 13.7s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.4782 | Acc: 0.812 | Elapsed: 14.1s | ETA: 2.1s
  Batch 42/47 (89.4%) | Loss: 0.4753 | Acc: 0.815 | Elapsed: 14.4s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.4723 | Acc: 0.815 | Elapsed: 14.7s | ETA: 1.4s
  Batch 44/47 (93.6%) | Loss: 0.4690 | Acc: 0.817 | Elapsed: 15.1s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.4730 | Acc: 0.815 | Elapsed: 15.4s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.4686 | Acc: 0.816 | Elapsed: 15.7s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.4680 | Acc: 0.816 | Elapsed: 15.8s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.5812 | Acc: 0.552
Epoch 39 completed in 17.7s

--- Epoch 40/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.5301 | Acc: 0.750 | Elapsed: 0.8s | ETA: 36.7s
  Batch 2/47 (4.3%) | Loss: 0.4703 | Acc: 0.797 | Elapsed: 1.1s | ETA: 25.0s
  Batch 3/47 (6.4%) | Loss: 0.4830 | Acc: 0.802 | Elapsed: 1.5s | ETA: 21.5s
  Batch 4/47 (8.5%) | Loss: 0.4660 | Acc: 0.820 | Elapsed: 1.8s | ETA: 19.1s
  Batch 5/47 (10.6%) | Loss: 0.4421 | Acc: 0.825 | Elapsed: 2.1s | ETA: 17.6s
  Batch 6/47 (12.8%) | Loss: 0.4466 | Acc: 0.812 | Elapsed: 2.4s | ETA: 16.6s
  Batch 7/47 (14.9%) | Loss: 0.4339 | Acc: 0.812 | Elapsed: 2.7s | ETA: 15.7s
  Batch 8/47 (17.0%) | Loss: 0.4371 | Acc: 0.809 | Elapsed: 3.1s | ETA: 14.9s
  Batch 9/47 (19.1%) | Loss: 0.4626 | Acc: 0.799 | Elapsed: 3.4s | ETA: 14.3s
  Batch 10/47 (21.3%) | Loss: 0.4402 | Acc: 0.806 | Elapsed: 3.7s | ETA: 13.8s
  Batch 11/47 (23.4%) | Loss: 0.4408 | Acc: 0.810 | Elapsed: 4.0s | ETA: 13.2s
  Batch 12/47 (25.5%) | Loss: 0.4351 | Acc: 0.810 | Elapsed: 4.4s | ETA: 12.7s
  Batch 13/47 (27.7%) | Loss: 0.4434 | Acc: 0.805 | Elapsed: 4.7s | ETA: 12.3s
  Batch 14/47 (29.8%) | Loss: 0.4457 | Acc: 0.808 | Elapsed: 5.0s | ETA: 11.8s
  Batch 15/47 (31.9%) | Loss: 0.4831 | Acc: 0.798 | Elapsed: 5.3s | ETA: 11.4s
  Batch 16/47 (34.0%) | Loss: 0.4806 | Acc: 0.797 | Elapsed: 5.6s | ETA: 10.9s
  Batch 17/47 (36.2%) | Loss: 0.4796 | Acc: 0.798 | Elapsed: 6.0s | ETA: 10.5s
  Batch 18/47 (38.3%) | Loss: 0.4809 | Acc: 0.800 | Elapsed: 6.3s | ETA: 10.1s
  Batch 19/47 (40.4%) | Loss: 0.4808 | Acc: 0.803 | Elapsed: 6.6s | ETA: 9.7s
  Batch 20/47 (42.6%) | Loss: 0.4816 | Acc: 0.800 | Elapsed: 6.9s | ETA: 9.4s
  Batch 21/47 (44.7%) | Loss: 0.4854 | Acc: 0.793 | Elapsed: 7.3s | ETA: 9.0s
  Batch 22/47 (46.8%) | Loss: 0.4882 | Acc: 0.791 | Elapsed: 7.6s | ETA: 8.6s
  Batch 23/47 (48.9%) | Loss: 0.4796 | Acc: 0.793 | Elapsed: 7.9s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 0.4811 | Acc: 0.794 | Elapsed: 8.2s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 0.4743 | Acc: 0.799 | Elapsed: 8.5s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.4695 | Acc: 0.799 | Elapsed: 8.9s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 0.4702 | Acc: 0.801 | Elapsed: 9.2s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.4718 | Acc: 0.802 | Elapsed: 9.5s | ETA: 6.4s
  Batch 29/47 (61.7%) | Loss: 0.4668 | Acc: 0.805 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.4634 | Acc: 0.808 | Elapsed: 10.1s | ETA: 5.7s
  Batch 31/47 (66.0%) | Loss: 0.4615 | Acc: 0.808 | Elapsed: 10.5s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.4597 | Acc: 0.807 | Elapsed: 10.8s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.4537 | Acc: 0.811 | Elapsed: 11.1s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.4506 | Acc: 0.812 | Elapsed: 11.4s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.4494 | Acc: 0.813 | Elapsed: 11.8s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 0.4451 | Acc: 0.816 | Elapsed: 12.1s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.4453 | Acc: 0.817 | Elapsed: 12.4s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.4444 | Acc: 0.818 | Elapsed: 12.7s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.4434 | Acc: 0.820 | Elapsed: 13.1s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.4482 | Acc: 0.818 | Elapsed: 13.4s | ETA: 2.3s
  Batch 41/47 (87.2%) | Loss: 0.4534 | Acc: 0.815 | Elapsed: 13.7s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.4513 | Acc: 0.815 | Elapsed: 14.0s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.4440 | Acc: 0.818 | Elapsed: 14.3s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.4426 | Acc: 0.819 | Elapsed: 14.6s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.4448 | Acc: 0.817 | Elapsed: 15.0s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.4457 | Acc: 0.817 | Elapsed: 15.3s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.4463 | Acc: 0.817 | Elapsed: 15.3s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.3772 | Acc: 0.554
Epoch 40 completed in 17.1s

--- Epoch 41/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.4634 | Acc: 0.812 | Elapsed: 0.8s | ETA: 36.8s
  Batch 2/47 (4.3%) | Loss: 0.4414 | Acc: 0.844 | Elapsed: 1.1s | ETA: 25.1s
  Batch 3/47 (6.4%) | Loss: 0.4700 | Acc: 0.812 | Elapsed: 1.4s | ETA: 21.2s
  Batch 4/47 (8.5%) | Loss: 0.4297 | Acc: 0.828 | Elapsed: 1.8s | ETA: 18.9s
  Batch 5/47 (10.6%) | Loss: 0.5867 | Acc: 0.775 | Elapsed: 2.1s | ETA: 17.4s
  Batch 6/47 (12.8%) | Loss: 0.5610 | Acc: 0.776 | Elapsed: 2.4s | ETA: 16.4s
  Batch 7/47 (14.9%) | Loss: 0.5430 | Acc: 0.790 | Elapsed: 2.7s | ETA: 15.5s
  Batch 8/47 (17.0%) | Loss: 0.5965 | Acc: 0.766 | Elapsed: 3.0s | ETA: 14.8s
  Batch 9/47 (19.1%) | Loss: 0.5670 | Acc: 0.781 | Elapsed: 3.3s | ETA: 14.1s
  Batch 10/47 (21.3%) | Loss: 0.5576 | Acc: 0.781 | Elapsed: 3.7s | ETA: 13.6s
  Batch 11/47 (23.4%) | Loss: 0.5376 | Acc: 0.790 | Elapsed: 4.0s | ETA: 13.0s
  Batch 12/47 (25.5%) | Loss: 0.5267 | Acc: 0.789 | Elapsed: 4.3s | ETA: 12.5s
  Batch 13/47 (27.7%) | Loss: 0.5295 | Acc: 0.796 | Elapsed: 4.6s | ETA: 12.1s
  Batch 14/47 (29.8%) | Loss: 0.5172 | Acc: 0.806 | Elapsed: 4.9s | ETA: 11.7s
  Batch 15/47 (31.9%) | Loss: 0.5085 | Acc: 0.810 | Elapsed: 5.3s | ETA: 11.2s
  Batch 16/47 (34.0%) | Loss: 0.5196 | Acc: 0.801 | Elapsed: 5.6s | ETA: 10.8s
  Batch 17/47 (36.2%) | Loss: 0.5075 | Acc: 0.805 | Elapsed: 5.9s | ETA: 10.4s
  Batch 18/47 (38.3%) | Loss: 0.4947 | Acc: 0.812 | Elapsed: 6.2s | ETA: 10.0s
  Batch 19/47 (40.4%) | Loss: 0.4967 | Acc: 0.812 | Elapsed: 6.6s | ETA: 9.7s
  Batch 20/47 (42.6%) | Loss: 0.4947 | Acc: 0.811 | Elapsed: 6.9s | ETA: 9.3s
  Batch 21/47 (44.7%) | Loss: 0.4818 | Acc: 0.817 | Elapsed: 7.2s | ETA: 8.9s
  Batch 22/47 (46.8%) | Loss: 0.4730 | Acc: 0.821 | Elapsed: 7.5s | ETA: 8.6s
  Batch 23/47 (48.9%) | Loss: 0.4736 | Acc: 0.822 | Elapsed: 7.8s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 0.4774 | Acc: 0.818 | Elapsed: 8.2s | ETA: 7.8s
  Batch 25/47 (53.2%) | Loss: 0.4818 | Acc: 0.815 | Elapsed: 8.5s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.4877 | Acc: 0.811 | Elapsed: 8.8s | ETA: 7.1s
  Batch 27/47 (57.4%) | Loss: 0.4836 | Acc: 0.810 | Elapsed: 9.1s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.4790 | Acc: 0.809 | Elapsed: 9.4s | ETA: 6.4s
  Batch 29/47 (61.7%) | Loss: 0.4752 | Acc: 0.810 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.4716 | Acc: 0.812 | Elapsed: 10.1s | ETA: 5.7s
  Batch 31/47 (66.0%) | Loss: 0.4732 | Acc: 0.812 | Elapsed: 10.4s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.4660 | Acc: 0.817 | Elapsed: 10.7s | ETA: 5.0s
  Batch 33/47 (70.2%) | Loss: 0.4632 | Acc: 0.819 | Elapsed: 11.0s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.4553 | Acc: 0.822 | Elapsed: 11.3s | ETA: 4.3s
  Batch 35/47 (74.5%) | Loss: 0.4516 | Acc: 0.822 | Elapsed: 11.7s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 0.4499 | Acc: 0.824 | Elapsed: 12.0s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.4483 | Acc: 0.824 | Elapsed: 12.3s | ETA: 3.3s
  Batch 38/47 (80.9%) | Loss: 0.4510 | Acc: 0.822 | Elapsed: 12.6s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.4536 | Acc: 0.821 | Elapsed: 12.9s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.4518 | Acc: 0.824 | Elapsed: 13.3s | ETA: 2.3s
  Batch 41/47 (87.2%) | Loss: 0.4483 | Acc: 0.826 | Elapsed: 13.6s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.4523 | Acc: 0.824 | Elapsed: 13.9s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.4538 | Acc: 0.822 | Elapsed: 14.2s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.4532 | Acc: 0.822 | Elapsed: 14.5s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.4555 | Acc: 0.823 | Elapsed: 14.9s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.4609 | Acc: 0.821 | Elapsed: 15.2s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.4605 | Acc: 0.822 | Elapsed: 15.2s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.6247 | Acc: 0.554
Epoch 41 completed in 17.0s

--- Epoch 42/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.3806 | Acc: 0.844 | Elapsed: 0.8s | ETA: 36.5s
  Batch 2/47 (4.3%) | Loss: 0.4471 | Acc: 0.797 | Elapsed: 1.1s | ETA: 24.8s
  Batch 3/47 (6.4%) | Loss: 0.3910 | Acc: 0.833 | Elapsed: 1.4s | ETA: 21.2s
  Batch 4/47 (8.5%) | Loss: 0.3804 | Acc: 0.852 | Elapsed: 1.8s | ETA: 18.9s
  Batch 5/47 (10.6%) | Loss: 0.4193 | Acc: 0.831 | Elapsed: 2.1s | ETA: 17.5s
  Batch 6/47 (12.8%) | Loss: 0.4180 | Acc: 0.839 | Elapsed: 2.4s | ETA: 16.4s
  Batch 7/47 (14.9%) | Loss: 0.4125 | Acc: 0.839 | Elapsed: 2.7s | ETA: 15.6s
  Batch 8/47 (17.0%) | Loss: 0.3997 | Acc: 0.844 | Elapsed: 3.0s | ETA: 14.8s
  Batch 9/47 (19.1%) | Loss: 0.3972 | Acc: 0.851 | Elapsed: 3.4s | ETA: 14.2s
  Batch 10/47 (21.3%) | Loss: 0.4224 | Acc: 0.834 | Elapsed: 3.7s | ETA: 13.7s
  Batch 11/47 (23.4%) | Loss: 0.4325 | Acc: 0.835 | Elapsed: 4.0s | ETA: 13.1s
  Batch 12/47 (25.5%) | Loss: 0.4259 | Acc: 0.836 | Elapsed: 4.3s | ETA: 12.7s
  Batch 13/47 (27.7%) | Loss: 0.4201 | Acc: 0.839 | Elapsed: 4.7s | ETA: 12.2s
  Batch 14/47 (29.8%) | Loss: 0.4206 | Acc: 0.837 | Elapsed: 5.0s | ETA: 11.7s
  Batch 15/47 (31.9%) | Loss: 0.4396 | Acc: 0.833 | Elapsed: 5.3s | ETA: 11.3s
  Batch 16/47 (34.0%) | Loss: 0.4410 | Acc: 0.832 | Elapsed: 5.6s | ETA: 10.9s
  Batch 17/47 (36.2%) | Loss: 0.4528 | Acc: 0.825 | Elapsed: 5.9s | ETA: 10.5s
  Batch 18/47 (38.3%) | Loss: 0.4392 | Acc: 0.832 | Elapsed: 6.3s | ETA: 10.1s
  Batch 19/47 (40.4%) | Loss: 0.4324 | Acc: 0.836 | Elapsed: 6.6s | ETA: 9.7s
  Batch 20/47 (42.6%) | Loss: 0.4340 | Acc: 0.838 | Elapsed: 6.9s | ETA: 9.3s
  Batch 21/47 (44.7%) | Loss: 0.4325 | Acc: 0.839 | Elapsed: 7.2s | ETA: 8.9s
  Batch 22/47 (46.8%) | Loss: 0.4302 | Acc: 0.838 | Elapsed: 7.5s | ETA: 8.6s
  Batch 23/47 (48.9%) | Loss: 0.4325 | Acc: 0.836 | Elapsed: 7.9s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 0.4372 | Acc: 0.832 | Elapsed: 8.2s | ETA: 7.8s
  Batch 25/47 (53.2%) | Loss: 0.4592 | Acc: 0.829 | Elapsed: 8.5s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.4572 | Acc: 0.827 | Elapsed: 8.8s | ETA: 7.1s
  Batch 27/47 (57.4%) | Loss: 0.4509 | Acc: 0.830 | Elapsed: 9.1s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.4470 | Acc: 0.831 | Elapsed: 9.5s | ETA: 6.4s
  Batch 29/47 (61.7%) | Loss: 0.4445 | Acc: 0.831 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.4383 | Acc: 0.832 | Elapsed: 10.1s | ETA: 5.7s
  Batch 31/47 (66.0%) | Loss: 0.4346 | Acc: 0.834 | Elapsed: 10.4s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.4293 | Acc: 0.837 | Elapsed: 10.7s | ETA: 5.0s
  Batch 33/47 (70.2%) | Loss: 0.4311 | Acc: 0.835 | Elapsed: 11.1s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.4291 | Acc: 0.835 | Elapsed: 11.4s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.4252 | Acc: 0.836 | Elapsed: 11.7s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 0.4237 | Acc: 0.836 | Elapsed: 12.0s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.4240 | Acc: 0.835 | Elapsed: 12.3s | ETA: 3.3s
  Batch 38/47 (80.9%) | Loss: 0.4248 | Acc: 0.836 | Elapsed: 12.7s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.4218 | Acc: 0.836 | Elapsed: 13.0s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.4236 | Acc: 0.836 | Elapsed: 13.3s | ETA: 2.3s
  Batch 41/47 (87.2%) | Loss: 0.4241 | Acc: 0.834 | Elapsed: 13.6s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.4231 | Acc: 0.834 | Elapsed: 13.9s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.4222 | Acc: 0.833 | Elapsed: 14.3s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.4237 | Acc: 0.832 | Elapsed: 14.6s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.4243 | Acc: 0.831 | Elapsed: 14.9s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.4248 | Acc: 0.831 | Elapsed: 15.2s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.4298 | Acc: 0.830 | Elapsed: 15.3s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.4348 | Acc: 0.585
Epoch 42 completed in 17.5s

--- Epoch 43/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.5012 | Acc: 0.812 | Elapsed: 1.0s | ETA: 46.1s
  Batch 2/47 (4.3%) | Loss: 0.4049 | Acc: 0.828 | Elapsed: 1.3s | ETA: 29.6s
  Batch 3/47 (6.4%) | Loss: 0.5084 | Acc: 0.802 | Elapsed: 1.7s | ETA: 24.3s
  Batch 4/47 (8.5%) | Loss: 0.5356 | Acc: 0.797 | Elapsed: 2.0s | ETA: 21.2s
  Batch 5/47 (10.6%) | Loss: 0.5200 | Acc: 0.806 | Elapsed: 2.3s | ETA: 19.2s
  Batch 6/47 (12.8%) | Loss: 0.5394 | Acc: 0.807 | Elapsed: 2.6s | ETA: 17.9s
  Batch 7/47 (14.9%) | Loss: 0.5349 | Acc: 0.804 | Elapsed: 2.9s | ETA: 16.8s
  Batch 8/47 (17.0%) | Loss: 0.5281 | Acc: 0.801 | Elapsed: 3.2s | ETA: 15.8s
  Batch 9/47 (19.1%) | Loss: 0.5052 | Acc: 0.809 | Elapsed: 3.6s | ETA: 15.1s
  Batch 10/47 (21.3%) | Loss: 0.4910 | Acc: 0.812 | Elapsed: 3.9s | ETA: 14.4s
  Batch 11/47 (23.4%) | Loss: 0.4706 | Acc: 0.821 | Elapsed: 4.2s | ETA: 13.8s
  Batch 12/47 (25.5%) | Loss: 0.4771 | Acc: 0.820 | Elapsed: 4.5s | ETA: 13.2s
  Batch 13/47 (27.7%) | Loss: 0.4666 | Acc: 0.825 | Elapsed: 4.9s | ETA: 12.7s
  Batch 14/47 (29.8%) | Loss: 0.4702 | Acc: 0.826 | Elapsed: 5.2s | ETA: 12.2s
  Batch 15/47 (31.9%) | Loss: 0.4627 | Acc: 0.827 | Elapsed: 5.5s | ETA: 11.7s
  Batch 16/47 (34.0%) | Loss: 0.4640 | Acc: 0.826 | Elapsed: 5.8s | ETA: 11.3s
  Batch 17/47 (36.2%) | Loss: 0.4532 | Acc: 0.833 | Elapsed: 6.1s | ETA: 10.8s
  Batch 18/47 (38.3%) | Loss: 0.4494 | Acc: 0.833 | Elapsed: 6.5s | ETA: 10.4s
  Batch 19/47 (40.4%) | Loss: 0.4569 | Acc: 0.827 | Elapsed: 6.8s | ETA: 10.0s
  Batch 20/47 (42.6%) | Loss: 0.4500 | Acc: 0.830 | Elapsed: 7.1s | ETA: 9.6s
  Batch 21/47 (44.7%) | Loss: 0.4409 | Acc: 0.835 | Elapsed: 7.4s | ETA: 9.2s
  Batch 22/47 (46.8%) | Loss: 0.4378 | Acc: 0.835 | Elapsed: 7.8s | ETA: 8.8s
  Batch 23/47 (48.9%) | Loss: 0.4336 | Acc: 0.834 | Elapsed: 8.1s | ETA: 8.4s
  Batch 24/47 (51.1%) | Loss: 0.4401 | Acc: 0.832 | Elapsed: 8.4s | ETA: 8.0s
  Batch 25/47 (53.2%) | Loss: 0.4476 | Acc: 0.826 | Elapsed: 8.7s | ETA: 7.7s
  Batch 26/47 (55.3%) | Loss: 0.4566 | Acc: 0.822 | Elapsed: 9.0s | ETA: 7.3s
  Batch 27/47 (57.4%) | Loss: 0.4481 | Acc: 0.824 | Elapsed: 9.4s | ETA: 6.9s
  Batch 28/47 (59.6%) | Loss: 0.4468 | Acc: 0.824 | Elapsed: 9.7s | ETA: 6.6s
  Batch 29/47 (61.7%) | Loss: 0.4438 | Acc: 0.823 | Elapsed: 10.0s | ETA: 6.2s
  Batch 30/47 (63.8%) | Loss: 0.4407 | Acc: 0.825 | Elapsed: 10.3s | ETA: 5.9s
  Batch 31/47 (66.0%) | Loss: 0.4457 | Acc: 0.822 | Elapsed: 10.6s | ETA: 5.5s
  Batch 32/47 (68.1%) | Loss: 0.4483 | Acc: 0.819 | Elapsed: 11.0s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.4497 | Acc: 0.820 | Elapsed: 11.3s | ETA: 4.8s
  Batch 34/47 (72.3%) | Loss: 0.4488 | Acc: 0.821 | Elapsed: 11.6s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.4449 | Acc: 0.822 | Elapsed: 12.0s | ETA: 4.1s
  Batch 36/47 (76.6%) | Loss: 0.4412 | Acc: 0.823 | Elapsed: 12.3s | ETA: 3.8s
  Batch 37/47 (78.7%) | Loss: 0.4407 | Acc: 0.822 | Elapsed: 12.6s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.4402 | Acc: 0.823 | Elapsed: 12.9s | ETA: 3.1s
  Batch 39/47 (83.0%) | Loss: 0.4429 | Acc: 0.823 | Elapsed: 13.3s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.4450 | Acc: 0.822 | Elapsed: 13.6s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.4407 | Acc: 0.825 | Elapsed: 13.9s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.4397 | Acc: 0.827 | Elapsed: 14.2s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.4389 | Acc: 0.827 | Elapsed: 14.6s | ETA: 1.4s
  Batch 44/47 (93.6%) | Loss: 0.4408 | Acc: 0.825 | Elapsed: 14.9s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.4404 | Acc: 0.824 | Elapsed: 15.2s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.4400 | Acc: 0.824 | Elapsed: 15.5s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.4417 | Acc: 0.823 | Elapsed: 15.6s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.5587 | Acc: 0.528
Epoch 43 completed in 17.3s

--- Epoch 44/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.4990 | Acc: 0.875 | Elapsed: 0.8s | ETA: 36.4s
  Batch 2/47 (4.3%) | Loss: 0.3982 | Acc: 0.875 | Elapsed: 1.1s | ETA: 24.9s
  Batch 3/47 (6.4%) | Loss: 0.3661 | Acc: 0.875 | Elapsed: 1.5s | ETA: 21.3s
  Batch 4/47 (8.5%) | Loss: 0.4019 | Acc: 0.852 | Elapsed: 1.8s | ETA: 18.9s
  Batch 5/47 (10.6%) | Loss: 0.4226 | Acc: 0.819 | Elapsed: 2.1s | ETA: 17.5s
  Batch 6/47 (12.8%) | Loss: 0.4384 | Acc: 0.802 | Elapsed: 2.4s | ETA: 16.5s
  Batch 7/47 (14.9%) | Loss: 0.4642 | Acc: 0.790 | Elapsed: 2.7s | ETA: 15.6s
  Batch 8/47 (17.0%) | Loss: 0.4455 | Acc: 0.801 | Elapsed: 3.0s | ETA: 14.9s
  Batch 9/47 (19.1%) | Loss: 0.4962 | Acc: 0.778 | Elapsed: 3.4s | ETA: 14.2s
  Batch 10/47 (21.3%) | Loss: 0.4753 | Acc: 0.787 | Elapsed: 3.7s | ETA: 13.7s
  Batch 11/47 (23.4%) | Loss: 0.4848 | Acc: 0.776 | Elapsed: 4.0s | ETA: 13.2s
  Batch 12/47 (25.5%) | Loss: 0.4773 | Acc: 0.781 | Elapsed: 4.3s | ETA: 12.7s
  Batch 13/47 (27.7%) | Loss: 0.4630 | Acc: 0.788 | Elapsed: 4.7s | ETA: 12.2s
  Batch 14/47 (29.8%) | Loss: 0.4838 | Acc: 0.772 | Elapsed: 5.0s | ETA: 11.8s
  Batch 15/47 (31.9%) | Loss: 0.4769 | Acc: 0.777 | Elapsed: 5.3s | ETA: 11.3s
  Batch 16/47 (34.0%) | Loss: 0.4757 | Acc: 0.779 | Elapsed: 5.6s | ETA: 10.9s
  Batch 17/47 (36.2%) | Loss: 0.4660 | Acc: 0.785 | Elapsed: 6.0s | ETA: 10.5s
  Batch 18/47 (38.3%) | Loss: 0.4655 | Acc: 0.783 | Elapsed: 6.3s | ETA: 10.1s
  Batch 19/47 (40.4%) | Loss: 0.4622 | Acc: 0.785 | Elapsed: 6.6s | ETA: 9.7s
  Batch 20/47 (42.6%) | Loss: 0.4781 | Acc: 0.781 | Elapsed: 6.9s | ETA: 9.4s
  Batch 21/47 (44.7%) | Loss: 0.4747 | Acc: 0.781 | Elapsed: 7.2s | ETA: 9.0s
  Batch 22/47 (46.8%) | Loss: 0.4678 | Acc: 0.784 | Elapsed: 7.6s | ETA: 8.6s
  Batch 23/47 (48.9%) | Loss: 0.4774 | Acc: 0.784 | Elapsed: 7.9s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 0.4759 | Acc: 0.786 | Elapsed: 8.2s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 0.4773 | Acc: 0.786 | Elapsed: 8.5s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.4712 | Acc: 0.788 | Elapsed: 8.9s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 0.4847 | Acc: 0.787 | Elapsed: 9.2s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.4904 | Acc: 0.783 | Elapsed: 9.5s | ETA: 6.5s
  Batch 29/47 (61.7%) | Loss: 0.4848 | Acc: 0.782 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.4824 | Acc: 0.787 | Elapsed: 10.2s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.4752 | Acc: 0.791 | Elapsed: 10.5s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.4763 | Acc: 0.793 | Elapsed: 10.8s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.4785 | Acc: 0.793 | Elapsed: 11.1s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.4736 | Acc: 0.796 | Elapsed: 11.5s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.4716 | Acc: 0.797 | Elapsed: 11.8s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 0.4711 | Acc: 0.798 | Elapsed: 12.1s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.4692 | Acc: 0.800 | Elapsed: 12.4s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.4672 | Acc: 0.801 | Elapsed: 12.7s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.4684 | Acc: 0.802 | Elapsed: 13.1s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.4678 | Acc: 0.802 | Elapsed: 13.4s | ETA: 2.3s
  Batch 41/47 (87.2%) | Loss: 0.4694 | Acc: 0.801 | Elapsed: 13.7s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.4715 | Acc: 0.802 | Elapsed: 14.0s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.4724 | Acc: 0.803 | Elapsed: 14.4s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.4776 | Acc: 0.800 | Elapsed: 14.7s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.4744 | Acc: 0.803 | Elapsed: 15.0s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.4806 | Acc: 0.801 | Elapsed: 15.3s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.4800 | Acc: 0.801 | Elapsed: 15.4s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.5491 | Acc: 0.602
Epoch 44 completed in 17.1s

--- Epoch 45/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.4534 | Acc: 0.750 | Elapsed: 0.8s | ETA: 36.7s
  Batch 2/47 (4.3%) | Loss: 0.4428 | Acc: 0.797 | Elapsed: 1.1s | ETA: 25.0s
  Batch 3/47 (6.4%) | Loss: 0.5225 | Acc: 0.802 | Elapsed: 1.5s | ETA: 21.5s
  Batch 4/47 (8.5%) | Loss: 0.5007 | Acc: 0.820 | Elapsed: 1.8s | ETA: 19.1s
  Batch 5/47 (10.6%) | Loss: 0.4621 | Acc: 0.838 | Elapsed: 2.1s | ETA: 17.6s
  Batch 6/47 (12.8%) | Loss: 0.4681 | Acc: 0.823 | Elapsed: 2.4s | ETA: 16.6s
  Batch 7/47 (14.9%) | Loss: 0.4616 | Acc: 0.817 | Elapsed: 2.7s | ETA: 15.7s
  Batch 8/47 (17.0%) | Loss: 0.4680 | Acc: 0.812 | Elapsed: 3.1s | ETA: 14.9s
  Batch 9/47 (19.1%) | Loss: 0.4502 | Acc: 0.823 | Elapsed: 3.4s | ETA: 14.3s
  Batch 10/47 (21.3%) | Loss: 0.4436 | Acc: 0.828 | Elapsed: 3.7s | ETA: 13.7s
  Batch 11/47 (23.4%) | Loss: 0.4527 | Acc: 0.827 | Elapsed: 4.0s | ETA: 13.2s
  Batch 12/47 (25.5%) | Loss: 0.4443 | Acc: 0.828 | Elapsed: 4.4s | ETA: 12.7s
  Batch 13/47 (27.7%) | Loss: 0.4326 | Acc: 0.832 | Elapsed: 4.7s | ETA: 12.2s
  Batch 14/47 (29.8%) | Loss: 0.4693 | Acc: 0.819 | Elapsed: 5.0s | ETA: 11.8s
  Batch 15/47 (31.9%) | Loss: 0.4555 | Acc: 0.825 | Elapsed: 5.3s | ETA: 11.3s
  Batch 16/47 (34.0%) | Loss: 0.4572 | Acc: 0.822 | Elapsed: 5.6s | ETA: 10.9s
  Batch 17/47 (36.2%) | Loss: 0.4651 | Acc: 0.816 | Elapsed: 6.0s | ETA: 10.5s
  Batch 18/47 (38.3%) | Loss: 0.4658 | Acc: 0.812 | Elapsed: 6.3s | ETA: 10.1s
  Batch 19/47 (40.4%) | Loss: 0.4718 | Acc: 0.809 | Elapsed: 6.6s | ETA: 9.7s
  Batch 20/47 (42.6%) | Loss: 0.4695 | Acc: 0.806 | Elapsed: 6.9s | ETA: 9.4s
  Batch 21/47 (44.7%) | Loss: 0.4610 | Acc: 0.811 | Elapsed: 7.3s | ETA: 9.0s
  Batch 22/47 (46.8%) | Loss: 0.4663 | Acc: 0.810 | Elapsed: 7.6s | ETA: 8.6s
  Batch 23/47 (48.9%) | Loss: 0.4626 | Acc: 0.811 | Elapsed: 7.9s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 0.4587 | Acc: 0.812 | Elapsed: 8.2s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 0.4527 | Acc: 0.818 | Elapsed: 8.5s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.4630 | Acc: 0.812 | Elapsed: 8.9s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 0.4643 | Acc: 0.814 | Elapsed: 9.2s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.4610 | Acc: 0.814 | Elapsed: 9.5s | ETA: 6.5s
  Batch 29/47 (61.7%) | Loss: 0.4570 | Acc: 0.816 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.4594 | Acc: 0.814 | Elapsed: 10.1s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.4569 | Acc: 0.817 | Elapsed: 10.5s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.4533 | Acc: 0.817 | Elapsed: 10.8s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.4490 | Acc: 0.818 | Elapsed: 11.1s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.4521 | Acc: 0.819 | Elapsed: 11.4s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.4490 | Acc: 0.820 | Elapsed: 11.8s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 0.4482 | Acc: 0.820 | Elapsed: 12.1s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.4485 | Acc: 0.822 | Elapsed: 12.4s | ETA: 3.3s
  Batch 38/47 (80.9%) | Loss: 0.4492 | Acc: 0.821 | Elapsed: 12.7s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.4466 | Acc: 0.822 | Elapsed: 13.0s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.4435 | Acc: 0.824 | Elapsed: 13.4s | ETA: 2.3s
  Batch 41/47 (87.2%) | Loss: 0.4450 | Acc: 0.824 | Elapsed: 13.7s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.4409 | Acc: 0.827 | Elapsed: 14.0s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.4382 | Acc: 0.828 | Elapsed: 14.3s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.4365 | Acc: 0.830 | Elapsed: 14.6s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.4399 | Acc: 0.828 | Elapsed: 15.0s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.4444 | Acc: 0.827 | Elapsed: 15.3s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.4440 | Acc: 0.827 | Elapsed: 15.3s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.2728 | Acc: 0.587
Epoch 45 completed in 17.1s

--- Epoch 46/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.3436 | Acc: 0.844 | Elapsed: 1.0s | ETA: 47.9s
  Batch 2/47 (4.3%) | Loss: 0.3618 | Acc: 0.797 | Elapsed: 1.4s | ETA: 30.5s
  Batch 3/47 (6.4%) | Loss: 0.3387 | Acc: 0.833 | Elapsed: 1.7s | ETA: 24.9s
  Batch 4/47 (8.5%) | Loss: 0.3442 | Acc: 0.836 | Elapsed: 2.0s | ETA: 21.7s
  Batch 5/47 (10.6%) | Loss: 0.3446 | Acc: 0.844 | Elapsed: 2.3s | ETA: 19.6s
  Batch 6/47 (12.8%) | Loss: 0.3648 | Acc: 0.833 | Elapsed: 2.7s | ETA: 18.2s
  Batch 7/47 (14.9%) | Loss: 0.3751 | Acc: 0.830 | Elapsed: 3.0s | ETA: 17.0s
  Batch 8/47 (17.0%) | Loss: 0.3880 | Acc: 0.824 | Elapsed: 3.3s | ETA: 16.1s
  Batch 9/47 (19.1%) | Loss: 0.3895 | Acc: 0.826 | Elapsed: 3.6s | ETA: 15.3s
  Batch 10/47 (21.3%) | Loss: 0.4048 | Acc: 0.825 | Elapsed: 3.9s | ETA: 14.6s
  Batch 11/47 (23.4%) | Loss: 0.3867 | Acc: 0.832 | Elapsed: 4.3s | ETA: 13.9s
  Batch 12/47 (25.5%) | Loss: 0.3973 | Acc: 0.828 | Elapsed: 4.6s | ETA: 13.4s
  Batch 13/47 (27.7%) | Loss: 0.3897 | Acc: 0.834 | Elapsed: 4.9s | ETA: 12.8s
  Batch 14/47 (29.8%) | Loss: 0.3792 | Acc: 0.842 | Elapsed: 5.2s | ETA: 12.3s
  Batch 15/47 (31.9%) | Loss: 0.3906 | Acc: 0.844 | Elapsed: 5.5s | ETA: 11.8s
  Batch 16/47 (34.0%) | Loss: 0.3878 | Acc: 0.844 | Elapsed: 5.9s | ETA: 11.3s
  Batch 17/47 (36.2%) | Loss: 0.4005 | Acc: 0.838 | Elapsed: 6.2s | ETA: 10.9s
  Batch 18/47 (38.3%) | Loss: 0.4036 | Acc: 0.839 | Elapsed: 6.5s | ETA: 10.5s
  Batch 19/47 (40.4%) | Loss: 0.4039 | Acc: 0.837 | Elapsed: 6.8s | ETA: 10.1s
  Batch 20/47 (42.6%) | Loss: 0.4016 | Acc: 0.834 | Elapsed: 7.1s | ETA: 9.6s
  Batch 21/47 (44.7%) | Loss: 0.3990 | Acc: 0.838 | Elapsed: 7.5s | ETA: 9.2s
  Batch 22/47 (46.8%) | Loss: 0.4057 | Acc: 0.832 | Elapsed: 7.8s | ETA: 8.8s
  Batch 23/47 (48.9%) | Loss: 0.4144 | Acc: 0.829 | Elapsed: 8.1s | ETA: 8.5s
  Batch 24/47 (51.1%) | Loss: 0.4089 | Acc: 0.829 | Elapsed: 8.4s | ETA: 8.1s
  Batch 25/47 (53.2%) | Loss: 0.4103 | Acc: 0.830 | Elapsed: 8.7s | ETA: 7.7s
  Batch 26/47 (55.3%) | Loss: 0.4094 | Acc: 0.834 | Elapsed: 9.1s | ETA: 7.3s
  Batch 27/47 (57.4%) | Loss: 0.4057 | Acc: 0.836 | Elapsed: 9.4s | ETA: 7.0s
  Batch 28/47 (59.6%) | Loss: 0.4009 | Acc: 0.838 | Elapsed: 9.7s | ETA: 6.6s
  Batch 29/47 (61.7%) | Loss: 0.3985 | Acc: 0.838 | Elapsed: 10.0s | ETA: 6.2s
  Batch 30/47 (63.8%) | Loss: 0.3995 | Acc: 0.840 | Elapsed: 10.3s | ETA: 5.9s
  Batch 31/47 (66.0%) | Loss: 0.4034 | Acc: 0.838 | Elapsed: 10.7s | ETA: 5.5s
  Batch 32/47 (68.1%) | Loss: 0.4013 | Acc: 0.838 | Elapsed: 11.0s | ETA: 5.2s
  Batch 33/47 (70.2%) | Loss: 0.4011 | Acc: 0.840 | Elapsed: 11.3s | ETA: 4.8s
  Batch 34/47 (72.3%) | Loss: 0.4004 | Acc: 0.840 | Elapsed: 11.6s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.3973 | Acc: 0.840 | Elapsed: 12.0s | ETA: 4.1s
  Batch 36/47 (76.6%) | Loss: 0.3963 | Acc: 0.841 | Elapsed: 12.3s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.3950 | Acc: 0.841 | Elapsed: 12.6s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.3926 | Acc: 0.842 | Elapsed: 12.9s | ETA: 3.1s
  Batch 39/47 (83.0%) | Loss: 0.3957 | Acc: 0.843 | Elapsed: 13.2s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.3974 | Acc: 0.842 | Elapsed: 13.6s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.4046 | Acc: 0.838 | Elapsed: 13.9s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.4006 | Acc: 0.839 | Elapsed: 14.2s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.3982 | Acc: 0.840 | Elapsed: 14.5s | ETA: 1.4s
  Batch 44/47 (93.6%) | Loss: 0.3948 | Acc: 0.842 | Elapsed: 14.8s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.3959 | Acc: 0.842 | Elapsed: 15.2s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.4009 | Acc: 0.839 | Elapsed: 15.5s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.4036 | Acc: 0.838 | Elapsed: 15.5s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.1659 | Acc: 0.614
Epoch 46 completed in 18.4s

--- Epoch 47/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.4092 | Acc: 0.781 | Elapsed: 0.8s | ETA: 36.8s
  Batch 2/47 (4.3%) | Loss: 0.3098 | Acc: 0.844 | Elapsed: 1.1s | ETA: 25.0s
  Batch 3/47 (6.4%) | Loss: 0.3475 | Acc: 0.854 | Elapsed: 1.5s | ETA: 21.4s
  Batch 4/47 (8.5%) | Loss: 0.3673 | Acc: 0.844 | Elapsed: 1.8s | ETA: 19.0s
  Batch 5/47 (10.6%) | Loss: 0.3672 | Acc: 0.838 | Elapsed: 2.1s | ETA: 17.5s
  Batch 6/47 (12.8%) | Loss: 0.4012 | Acc: 0.823 | Elapsed: 2.4s | ETA: 16.4s
  Batch 7/47 (14.9%) | Loss: 0.4432 | Acc: 0.812 | Elapsed: 2.7s | ETA: 15.6s
  Batch 8/47 (17.0%) | Loss: 0.4628 | Acc: 0.797 | Elapsed: 3.0s | ETA: 14.8s
  Batch 9/47 (19.1%) | Loss: 0.4774 | Acc: 0.792 | Elapsed: 3.4s | ETA: 14.2s
  Batch 10/47 (21.3%) | Loss: 0.4690 | Acc: 0.797 | Elapsed: 3.7s | ETA: 13.6s
  Batch 11/47 (23.4%) | Loss: 0.4988 | Acc: 0.787 | Elapsed: 4.0s | ETA: 13.1s
  Batch 12/47 (25.5%) | Loss: 0.4918 | Acc: 0.789 | Elapsed: 4.3s | ETA: 12.6s
  Batch 13/47 (27.7%) | Loss: 0.4908 | Acc: 0.788 | Elapsed: 4.6s | ETA: 12.1s
  Batch 14/47 (29.8%) | Loss: 0.5078 | Acc: 0.783 | Elapsed: 5.0s | ETA: 11.7s
  Batch 15/47 (31.9%) | Loss: 0.5062 | Acc: 0.777 | Elapsed: 5.3s | ETA: 11.3s
  Batch 16/47 (34.0%) | Loss: 0.4969 | Acc: 0.781 | Elapsed: 5.6s | ETA: 10.9s
  Batch 17/47 (36.2%) | Loss: 0.4981 | Acc: 0.779 | Elapsed: 5.9s | ETA: 10.5s
  Batch 18/47 (38.3%) | Loss: 0.5082 | Acc: 0.771 | Elapsed: 6.2s | ETA: 10.1s
  Batch 19/47 (40.4%) | Loss: 0.4970 | Acc: 0.778 | Elapsed: 6.6s | ETA: 9.7s
  Batch 20/47 (42.6%) | Loss: 0.4934 | Acc: 0.778 | Elapsed: 6.9s | ETA: 9.3s
  Batch 21/47 (44.7%) | Loss: 0.5003 | Acc: 0.774 | Elapsed: 7.2s | ETA: 8.9s
  Batch 22/47 (46.8%) | Loss: 0.4904 | Acc: 0.778 | Elapsed: 7.5s | ETA: 8.6s
  Batch 23/47 (48.9%) | Loss: 0.4893 | Acc: 0.776 | Elapsed: 7.8s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 0.4882 | Acc: 0.777 | Elapsed: 8.2s | ETA: 7.8s
  Batch 25/47 (53.2%) | Loss: 0.4830 | Acc: 0.780 | Elapsed: 8.5s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.4806 | Acc: 0.779 | Elapsed: 8.8s | ETA: 7.1s
  Batch 27/47 (57.4%) | Loss: 0.4782 | Acc: 0.781 | Elapsed: 9.2s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.4745 | Acc: 0.783 | Elapsed: 9.5s | ETA: 6.4s
  Batch 29/47 (61.7%) | Loss: 0.4729 | Acc: 0.783 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.4739 | Acc: 0.782 | Elapsed: 10.1s | ETA: 5.7s
  Batch 31/47 (66.0%) | Loss: 0.4777 | Acc: 0.779 | Elapsed: 10.5s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.4755 | Acc: 0.781 | Elapsed: 10.8s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.4750 | Acc: 0.780 | Elapsed: 11.1s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.4713 | Acc: 0.783 | Elapsed: 11.4s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.4741 | Acc: 0.784 | Elapsed: 11.8s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 0.4754 | Acc: 0.783 | Elapsed: 12.1s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.4768 | Acc: 0.782 | Elapsed: 12.4s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.4694 | Acc: 0.786 | Elapsed: 12.7s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.4672 | Acc: 0.786 | Elapsed: 13.1s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.4591 | Acc: 0.791 | Elapsed: 13.4s | ETA: 2.3s
  Batch 41/47 (87.2%) | Loss: 0.4656 | Acc: 0.787 | Elapsed: 13.7s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.4661 | Acc: 0.788 | Elapsed: 14.1s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.4613 | Acc: 0.790 | Elapsed: 14.4s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.4604 | Acc: 0.790 | Elapsed: 14.7s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.4619 | Acc: 0.788 | Elapsed: 15.0s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.4688 | Acc: 0.785 | Elapsed: 15.4s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.4697 | Acc: 0.785 | Elapsed: 15.4s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.5574 | Acc: 0.530
Epoch 47 completed in 19.1s

--- Epoch 48/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.4633 | Acc: 0.812 | Elapsed: 0.8s | ETA: 37.5s
  Batch 2/47 (4.3%) | Loss: 0.4294 | Acc: 0.828 | Elapsed: 1.1s | ETA: 25.4s
  Batch 3/47 (6.4%) | Loss: 0.4469 | Acc: 0.781 | Elapsed: 1.5s | ETA: 21.5s
  Batch 4/47 (8.5%) | Loss: 0.4365 | Acc: 0.789 | Elapsed: 1.8s | ETA: 19.2s
  Batch 5/47 (10.6%) | Loss: 0.5010 | Acc: 0.769 | Elapsed: 2.1s | ETA: 17.7s
  Batch 6/47 (12.8%) | Loss: 0.5423 | Acc: 0.755 | Elapsed: 2.4s | ETA: 16.6s
  Batch 7/47 (14.9%) | Loss: 0.5467 | Acc: 0.750 | Elapsed: 2.8s | ETA: 15.8s
  Batch 8/47 (17.0%) | Loss: 0.5358 | Acc: 0.750 | Elapsed: 3.1s | ETA: 15.0s
  Batch 9/47 (19.1%) | Loss: 0.6018 | Acc: 0.729 | Elapsed: 3.4s | ETA: 14.4s
  Batch 10/47 (21.3%) | Loss: 0.5892 | Acc: 0.738 | Elapsed: 3.7s | ETA: 13.8s
  Batch 11/47 (23.4%) | Loss: 0.5854 | Acc: 0.733 | Elapsed: 4.1s | ETA: 13.3s
  Batch 12/47 (25.5%) | Loss: 0.5647 | Acc: 0.740 | Elapsed: 4.4s | ETA: 12.8s
  Batch 13/47 (27.7%) | Loss: 0.5530 | Acc: 0.745 | Elapsed: 4.7s | ETA: 12.3s
  Batch 14/47 (29.8%) | Loss: 0.5565 | Acc: 0.746 | Elapsed: 5.0s | ETA: 11.9s
  Batch 15/47 (31.9%) | Loss: 0.5568 | Acc: 0.746 | Elapsed: 5.4s | ETA: 11.5s
  Batch 16/47 (34.0%) | Loss: 0.5657 | Acc: 0.746 | Elapsed: 5.7s | ETA: 11.0s
  Batch 17/47 (36.2%) | Loss: 0.5900 | Acc: 0.743 | Elapsed: 6.0s | ETA: 10.6s
  Batch 18/47 (38.3%) | Loss: 0.5663 | Acc: 0.753 | Elapsed: 6.3s | ETA: 10.2s
  Batch 19/47 (40.4%) | Loss: 0.5596 | Acc: 0.757 | Elapsed: 6.7s | ETA: 9.8s
  Batch 20/47 (42.6%) | Loss: 0.5535 | Acc: 0.758 | Elapsed: 7.0s | ETA: 9.4s
  Batch 21/47 (44.7%) | Loss: 0.5464 | Acc: 0.762 | Elapsed: 7.3s | ETA: 9.0s
  Batch 22/47 (46.8%) | Loss: 0.5469 | Acc: 0.760 | Elapsed: 7.6s | ETA: 8.7s
  Batch 23/47 (48.9%) | Loss: 0.5814 | Acc: 0.753 | Elapsed: 7.9s | ETA: 8.3s
  Batch 24/47 (51.1%) | Loss: 0.5977 | Acc: 0.747 | Elapsed: 8.3s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 0.5912 | Acc: 0.751 | Elapsed: 8.6s | ETA: 7.6s
  Batch 26/47 (55.3%) | Loss: 0.5965 | Acc: 0.755 | Elapsed: 8.9s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 0.5872 | Acc: 0.759 | Elapsed: 9.2s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.5857 | Acc: 0.761 | Elapsed: 9.5s | ETA: 6.5s
  Batch 29/47 (61.7%) | Loss: 0.5829 | Acc: 0.762 | Elapsed: 9.9s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.5761 | Acc: 0.766 | Elapsed: 10.2s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.5705 | Acc: 0.768 | Elapsed: 10.5s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.5640 | Acc: 0.772 | Elapsed: 10.8s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.5752 | Acc: 0.770 | Elapsed: 11.1s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.5752 | Acc: 0.768 | Elapsed: 11.5s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.5684 | Acc: 0.771 | Elapsed: 11.8s | ETA: 4.0s
  Batch 36/47 (76.6%) | Loss: 0.5659 | Acc: 0.773 | Elapsed: 12.1s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.5601 | Acc: 0.774 | Elapsed: 12.4s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.5576 | Acc: 0.775 | Elapsed: 12.8s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.5537 | Acc: 0.776 | Elapsed: 13.1s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.5547 | Acc: 0.776 | Elapsed: 13.4s | ETA: 2.3s
  Batch 41/47 (87.2%) | Loss: 0.5518 | Acc: 0.777 | Elapsed: 13.7s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.5446 | Acc: 0.781 | Elapsed: 14.0s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.5521 | Acc: 0.781 | Elapsed: 14.4s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.5534 | Acc: 0.778 | Elapsed: 14.7s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.5490 | Acc: 0.778 | Elapsed: 15.0s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.5511 | Acc: 0.775 | Elapsed: 15.3s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.5544 | Acc: 0.774 | Elapsed: 15.4s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.1861 | Acc: 0.585
Epoch 48 completed in 17.4s

--- Epoch 49/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.4690 | Acc: 0.781 | Elapsed: 1.0s | ETA: 45.2s
  Batch 2/47 (4.3%) | Loss: 0.4117 | Acc: 0.812 | Elapsed: 1.3s | ETA: 29.3s
  Batch 3/47 (6.4%) | Loss: 0.4371 | Acc: 0.802 | Elapsed: 1.6s | ETA: 24.1s
  Batch 4/47 (8.5%) | Loss: 0.4313 | Acc: 0.789 | Elapsed: 2.0s | ETA: 21.1s
  Batch 5/47 (10.6%) | Loss: 0.4210 | Acc: 0.794 | Elapsed: 2.3s | ETA: 19.1s
  Batch 6/47 (12.8%) | Loss: 0.4513 | Acc: 0.781 | Elapsed: 2.6s | ETA: 17.8s
  Batch 7/47 (14.9%) | Loss: 0.4458 | Acc: 0.790 | Elapsed: 2.9s | ETA: 16.7s
  Batch 8/47 (17.0%) | Loss: 0.4525 | Acc: 0.789 | Elapsed: 3.2s | ETA: 15.8s
  Batch 9/47 (19.1%) | Loss: 0.4750 | Acc: 0.781 | Elapsed: 3.6s | ETA: 15.1s
  Batch 10/47 (21.3%) | Loss: 0.5500 | Acc: 0.775 | Elapsed: 3.9s | ETA: 14.4s
  Batch 11/47 (23.4%) | Loss: 0.5364 | Acc: 0.781 | Elapsed: 4.2s | ETA: 13.8s
  Batch 12/47 (25.5%) | Loss: 0.5338 | Acc: 0.786 | Elapsed: 4.5s | ETA: 13.2s
  Batch 13/47 (27.7%) | Loss: 0.5559 | Acc: 0.769 | Elapsed: 4.9s | ETA: 12.7s
  Batch 14/47 (29.8%) | Loss: 0.5513 | Acc: 0.770 | Elapsed: 5.2s | ETA: 12.2s
  Batch 15/47 (31.9%) | Loss: 0.5480 | Acc: 0.775 | Elapsed: 5.5s | ETA: 11.8s
  Batch 16/47 (34.0%) | Loss: 0.5543 | Acc: 0.770 | Elapsed: 5.8s | ETA: 11.3s
  Batch 17/47 (36.2%) | Loss: 0.5600 | Acc: 0.767 | Elapsed: 6.2s | ETA: 10.9s
  Batch 18/47 (38.3%) | Loss: 0.5568 | Acc: 0.769 | Elapsed: 6.5s | ETA: 10.4s
  Batch 19/47 (40.4%) | Loss: 0.5553 | Acc: 0.768 | Elapsed: 6.8s | ETA: 10.0s
  Batch 20/47 (42.6%) | Loss: 0.5574 | Acc: 0.770 | Elapsed: 7.1s | ETA: 9.6s
  Batch 21/47 (44.7%) | Loss: 0.5486 | Acc: 0.775 | Elapsed: 7.4s | ETA: 9.2s
  Batch 22/47 (46.8%) | Loss: 0.5363 | Acc: 0.783 | Elapsed: 7.8s | ETA: 8.8s
  Batch 23/47 (48.9%) | Loss: 0.5358 | Acc: 0.783 | Elapsed: 8.1s | ETA: 8.4s
  Batch 24/47 (51.1%) | Loss: 0.5373 | Acc: 0.783 | Elapsed: 8.4s | ETA: 8.1s
  Batch 25/47 (53.2%) | Loss: 0.5289 | Acc: 0.786 | Elapsed: 8.7s | ETA: 7.7s
  Batch 26/47 (55.3%) | Loss: 0.5192 | Acc: 0.791 | Elapsed: 9.1s | ETA: 7.3s
  Batch 27/47 (57.4%) | Loss: 0.5150 | Acc: 0.795 | Elapsed: 9.4s | ETA: 7.0s
  Batch 28/47 (59.6%) | Loss: 0.5146 | Acc: 0.794 | Elapsed: 9.7s | ETA: 6.6s
  Batch 29/47 (61.7%) | Loss: 0.5100 | Acc: 0.794 | Elapsed: 10.0s | ETA: 6.2s
  Batch 30/47 (63.8%) | Loss: 0.5056 | Acc: 0.796 | Elapsed: 10.4s | ETA: 5.9s
  Batch 31/47 (66.0%) | Loss: 0.5016 | Acc: 0.796 | Elapsed: 10.7s | ETA: 5.5s
  Batch 32/47 (68.1%) | Loss: 0.4972 | Acc: 0.800 | Elapsed: 11.0s | ETA: 5.2s
  Batch 33/47 (70.2%) | Loss: 0.4904 | Acc: 0.804 | Elapsed: 11.3s | ETA: 4.8s
  Batch 34/47 (72.3%) | Loss: 0.4882 | Acc: 0.807 | Elapsed: 11.6s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.4864 | Acc: 0.806 | Elapsed: 12.0s | ETA: 4.1s
  Batch 36/47 (76.6%) | Loss: 0.4926 | Acc: 0.803 | Elapsed: 12.3s | ETA: 3.8s
  Batch 37/47 (78.7%) | Loss: 0.4898 | Acc: 0.803 | Elapsed: 12.6s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.4836 | Acc: 0.805 | Elapsed: 12.9s | ETA: 3.1s
  Batch 39/47 (83.0%) | Loss: 0.4820 | Acc: 0.807 | Elapsed: 13.3s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.4839 | Acc: 0.806 | Elapsed: 13.6s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.4757 | Acc: 0.810 | Elapsed: 13.9s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.4883 | Acc: 0.807 | Elapsed: 14.2s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.4836 | Acc: 0.808 | Elapsed: 14.6s | ETA: 1.4s
  Batch 44/47 (93.6%) | Loss: 0.4786 | Acc: 0.810 | Elapsed: 14.9s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.4793 | Acc: 0.810 | Elapsed: 15.2s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.4804 | Acc: 0.809 | Elapsed: 15.5s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.4804 | Acc: 0.809 | Elapsed: 15.6s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.9671 | Acc: 0.493
Epoch 49 completed in 17.3s

--- Epoch 50/50 ---
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
  Batch 1/47 (2.1%) | Loss: 0.5584 | Acc: 0.656 | Elapsed: 0.8s | ETA: 37.6s
  Batch 2/47 (4.3%) | Loss: 0.4470 | Acc: 0.781 | Elapsed: 1.1s | ETA: 25.4s
  Batch 3/47 (6.4%) | Loss: 0.4321 | Acc: 0.792 | Elapsed: 1.5s | ETA: 21.6s
  Batch 4/47 (8.5%) | Loss: 0.4485 | Acc: 0.805 | Elapsed: 1.8s | ETA: 19.2s
  Batch 5/47 (10.6%) | Loss: 0.4529 | Acc: 0.794 | Elapsed: 2.1s | ETA: 17.7s
  Batch 6/47 (12.8%) | Loss: 0.4191 | Acc: 0.812 | Elapsed: 2.4s | ETA: 16.6s
  Batch 7/47 (14.9%) | Loss: 0.4578 | Acc: 0.804 | Elapsed: 2.7s | ETA: 15.7s
  Batch 8/47 (17.0%) | Loss: 0.4592 | Acc: 0.805 | Elapsed: 3.1s | ETA: 14.9s
  Batch 9/47 (19.1%) | Loss: 0.4691 | Acc: 0.806 | Elapsed: 3.4s | ETA: 14.3s
  Batch 10/47 (21.3%) | Loss: 0.4584 | Acc: 0.816 | Elapsed: 3.7s | ETA: 13.7s
  Batch 11/47 (23.4%) | Loss: 0.4456 | Acc: 0.810 | Elapsed: 4.0s | ETA: 13.2s
  Batch 12/47 (25.5%) | Loss: 0.4445 | Acc: 0.807 | Elapsed: 4.4s | ETA: 12.7s
  Batch 13/47 (27.7%) | Loss: 0.4401 | Acc: 0.810 | Elapsed: 4.7s | ETA: 12.2s
  Batch 14/47 (29.8%) | Loss: 0.4324 | Acc: 0.812 | Elapsed: 5.0s | ETA: 11.8s
  Batch 15/47 (31.9%) | Loss: 0.4343 | Acc: 0.815 | Elapsed: 5.3s | ETA: 11.3s
  Batch 16/47 (34.0%) | Loss: 0.4466 | Acc: 0.812 | Elapsed: 5.6s | ETA: 10.9s
  Batch 17/47 (36.2%) | Loss: 0.4461 | Acc: 0.812 | Elapsed: 6.0s | ETA: 10.5s
  Batch 18/47 (38.3%) | Loss: 0.4535 | Acc: 0.811 | Elapsed: 6.3s | ETA: 10.1s
  Batch 19/47 (40.4%) | Loss: 0.4576 | Acc: 0.809 | Elapsed: 6.6s | ETA: 9.7s
  Batch 20/47 (42.6%) | Loss: 0.4505 | Acc: 0.814 | Elapsed: 6.9s | ETA: 9.3s
  Batch 21/47 (44.7%) | Loss: 0.4485 | Acc: 0.814 | Elapsed: 7.2s | ETA: 9.0s
  Batch 22/47 (46.8%) | Loss: 0.4476 | Acc: 0.817 | Elapsed: 7.6s | ETA: 8.6s
  Batch 23/47 (48.9%) | Loss: 0.4575 | Acc: 0.812 | Elapsed: 7.9s | ETA: 8.2s
  Batch 24/47 (51.1%) | Loss: 0.4486 | Acc: 0.816 | Elapsed: 8.2s | ETA: 7.9s
  Batch 25/47 (53.2%) | Loss: 0.4514 | Acc: 0.816 | Elapsed: 8.5s | ETA: 7.5s
  Batch 26/47 (55.3%) | Loss: 0.4578 | Acc: 0.814 | Elapsed: 8.9s | ETA: 7.2s
  Batch 27/47 (57.4%) | Loss: 0.4576 | Acc: 0.811 | Elapsed: 9.2s | ETA: 6.8s
  Batch 28/47 (59.6%) | Loss: 0.4560 | Acc: 0.810 | Elapsed: 9.5s | ETA: 6.5s
  Batch 29/47 (61.7%) | Loss: 0.4739 | Acc: 0.805 | Elapsed: 9.8s | ETA: 6.1s
  Batch 30/47 (63.8%) | Loss: 0.4650 | Acc: 0.810 | Elapsed: 10.2s | ETA: 5.8s
  Batch 31/47 (66.0%) | Loss: 0.4627 | Acc: 0.811 | Elapsed: 10.5s | ETA: 5.4s
  Batch 32/47 (68.1%) | Loss: 0.4603 | Acc: 0.812 | Elapsed: 10.8s | ETA: 5.1s
  Batch 33/47 (70.2%) | Loss: 0.4750 | Acc: 0.807 | Elapsed: 11.2s | ETA: 4.7s
  Batch 34/47 (72.3%) | Loss: 0.4798 | Acc: 0.805 | Elapsed: 11.5s | ETA: 4.4s
  Batch 35/47 (74.5%) | Loss: 0.4762 | Acc: 0.804 | Elapsed: 11.8s | ETA: 4.1s
  Batch 36/47 (76.6%) | Loss: 0.4852 | Acc: 0.801 | Elapsed: 12.1s | ETA: 3.7s
  Batch 37/47 (78.7%) | Loss: 0.4822 | Acc: 0.802 | Elapsed: 12.5s | ETA: 3.4s
  Batch 38/47 (80.9%) | Loss: 0.4817 | Acc: 0.803 | Elapsed: 12.8s | ETA: 3.0s
  Batch 39/47 (83.0%) | Loss: 0.4810 | Acc: 0.804 | Elapsed: 13.1s | ETA: 2.7s
  Batch 40/47 (85.1%) | Loss: 0.4761 | Acc: 0.808 | Elapsed: 13.4s | ETA: 2.4s
  Batch 41/47 (87.2%) | Loss: 0.4700 | Acc: 0.811 | Elapsed: 13.8s | ETA: 2.0s
  Batch 42/47 (89.4%) | Loss: 0.4712 | Acc: 0.810 | Elapsed: 14.1s | ETA: 1.7s
  Batch 43/47 (91.5%) | Loss: 0.4674 | Acc: 0.812 | Elapsed: 14.4s | ETA: 1.3s
  Batch 44/47 (93.6%) | Loss: 0.4672 | Acc: 0.811 | Elapsed: 14.7s | ETA: 1.0s
  Batch 45/47 (95.7%) | Loss: 0.4648 | Acc: 0.812 | Elapsed: 15.0s | ETA: 0.7s
  Batch 46/47 (97.9%) | Loss: 0.4585 | Acc: 0.814 | Elapsed: 15.4s | ETA: 0.3s
  Batch 47/47 (100.0%) | Loss: 0.4586 | Acc: 0.814 | Elapsed: 15.4s | ETA: 0.0s
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val -> Loss: 1.6349 | Acc: 0.585
Epoch 50 completed in 17.1s

✅ Training per 'supervised' completato. Best @ epoch 46 -> 0.614